In [1]:
import os
import asyncio
from metagpt.tools.libs.web_scraping import view_page_element_to_scrape
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# 创建项目目录结构
terminal = Terminal()
await terminal.run("mkdir -p langchain_tutorial/docs")
await terminal.run("mkdir -p langchain_tutorial/code_examples")
await terminal.run("mkdir -p langchain_tutorial/images")

# 获取LangChain文档主页内容
introduction_url = "https://python.langchain.com/docs/introduction/"
introduction_html = await view_page_element_to_scrape(
    url=introduction_url,
    requirement="获取页面内部导航结构和主要内容",
    keep_links=True
)

# 提取文档结构
import re
from bs4 import BeautifulSoup

soup = BeautifulSoup(introduction_html, 'html.parser')

# 提取主导航菜单
nav_items = soup.find_all('nav')
sidebar_content = None
for nav in nav_items:
    if nav.get('class') and 'sidebar' in ' '.join(nav.get('class')):
        sidebar_content = nav
        break

# 创建文档结构索引文件
with open('langchain_tutorial/docs/document_structure.md', 'w', encoding='utf-8') as f:
    f.write("# LangChain 文档结构\n\n")
    f.write("## 基本架构和核心组件\n\n")
    
    # 提取主要内容
    main_content = soup.find('main')
    if main_content:
        headers = main_content.find_all(['h1', 'h2', 'h3'])
        for header in headers:
            header_text = header.get_text(strip=True)
            if header.name == 'h1':
                f.write(f"# {header_text}\n\n")
            elif header.name == 'h2':
                f.write(f"## {header_text}\n\n")
            else:
                f.write(f"### {header_text}\n\n")
            
            # 获取段落内容
            next_elem = header.find_next_sibling()
            while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                if next_elem.name == 'p':
                    f.write(f"{next_elem.get_text(strip=True)}\n\n")
                next_elem = next_elem.find_next_sibling()
    
    # 添加侧边栏导航结构
    f.write("## 文档导航结构\n\n")
    if sidebar_content:
        links = sidebar_content.find_all('a')
        for link in links:
            href = link.get('href')
            text = link.get_text(strip=True)
            if href and text:
                f.write(f"- [{text}]({href})\n")

# 获取模块页面的内容
modules_url = "https://python.langchain.com/docs/modules/"
modules_html = await view_page_element_to_scrape(
    url=modules_url,
    requirement="获取LangChain各个模块的信息",
    keep_links=True
)

# 分析模块内容并保存
soup_modules = BeautifulSoup(modules_html, 'html.parser')
main_content = soup_modules.find('main')

with open('langchain_tutorial/docs/components.md', 'w', encoding='utf-8') as f:
    f.write("# LangChain 框架组件\n\n")
    
    if main_content:
        headers = main_content.find_all(['h1', 'h2', 'h3'])
        for header in headers:
            header_text = header.get_text(strip=True)
            if header.name == 'h1':
                f.write(f"# {header_text}\n\n")
            elif header.name == 'h2':
                f.write(f"## {header_text}\n\n")
            else:
                f.write(f"### {header_text}\n\n")
            
            # 获取段落内容
            next_elem = header.find_next_sibling()
            while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                if next_elem.name == 'p':
                    f.write(f"{next_elem.get_text(strip=True)}\n\n")
                next_elem = next_elem.find_next_sibling()

# 检查LangChain的agents部分
agents_url = "https://python.langchain.com/docs/modules/agents/"
agents_html = await view_page_element_to_scrape(
    url=agents_url,
    requirement="获取LangChain智能体相关信息",
    keep_links=True
)

# 保存agents文档
with open('langchain_tutorial/docs/agents.md', 'w', encoding='utf-8') as f:
    f.write("# LangChain 智能体 (Agents)\n\n")
    soup_agents = BeautifulSoup(agents_html, 'html.parser')
    main_content = soup_agents.find('main')
    
    if main_content:
        content_text = main_content.get_text(strip=True)
        paragraphs = re.split(r'\n\s*\n', content_text)
        for paragraph in paragraphs:
            f.write(f"{paragraph.strip()}\n\n")

# 显示项目结构
await terminal.run("find langchain_tutorial -type f | sort")
print("成功创建LangChain文档结构索引文件")

成功创建LangChain文档结构索引文件


2025-04-06 20:33:49.559 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:33:52.839 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:33:56.121 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.


In [2]:
import asyncio
from bs4 import BeautifulSoup
from metagpt.tools.libs.web_scraping import view_page_element_to_scrape
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# 创建目录用于保存不同模块的文档
terminal = Terminal()
await terminal.run("mkdir -p langchain_tutorial/docs/components")
await terminal.run("mkdir -p langchain_tutorial/docs/agents")
await terminal.run("mkdir -p langchain_tutorial/docs/search")

# 核心组件页面URL列表
component_urls = [
    ("model_io", "https://python.langchain.com/docs/modules/model_io/"),
    ("memory", "https://python.langchain.com/docs/modules/memory/"),
    ("prompt_templates", "https://python.langchain.com/docs/modules/prompts/"),
    ("output_parsers", "https://python.langchain.com/docs/modules/model_io/output_parsers"),
    ("chains", "https://python.langchain.com/docs/modules/chains/"),
    ("retrieval", "https://python.langchain.com/docs/modules/data_connection/"),
]

# 获取每个核心组件的文档并保存
for component_name, url in component_urls:
    print(f"正在获取 {component_name} 组件文档...")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"获取 {component_name} 组件的详细信息和示例代码",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    if main_content:
        with open(f"langchain_tutorial/docs/components/{component_name}.md", 'w', encoding='utf-8') as f:
            f.write(f"# LangChain {component_name} 组件\n\n")
            f.write(f"来源: {url}\n\n")
            
            # 提取标题和内容
            headers = main_content.find_all(['h1', 'h2', 'h3'])
            for header in headers:
                header_text = header.get_text(strip=True)
                if header.name == 'h1':
                    f.write(f"# {header_text}\n\n")
                elif header.name == 'h2':
                    f.write(f"## {header_text}\n\n")
                else:
                    f.write(f"### {header_text}\n\n")
                
                # 获取段落内容
                next_elem = header.find_next_sibling()
                while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                    if next_elem.name == 'p':
                        f.write(f"{next_elem.get_text(strip=True)}\n\n")
                    elif next_elem.name == 'pre':
                        # 获取代码示例
                        code = next_elem.get_text()
                        f.write("```python\n")
                        f.write(code)
                        f.write("\n```\n\n")
                    elif next_elem.name in ['ul', 'ol']:
                        # 处理列表
                        items = next_elem.find_all('li')
                        for item in items:
                            f.write(f"- {item.get_text(strip=True)}\n")
                        f.write("\n")
                    next_elem = next_elem.find_next_sibling()

# 获取智能体相关文档
agent_urls = [
    ("agent_overview", "https://python.langchain.com/docs/modules/agents/"),
    ("agent_types", "https://python.langchain.com/docs/modules/agents/agent_types/"),
    ("agent_executors", "https://python.langchain.com/docs/modules/agents/executor/"),
    ("tools", "https://python.langchain.com/docs/modules/tools/"),
    ("toolkits", "https://python.langchain.com/docs/modules/agents/toolkits/")
]

for agent_name, url in agent_urls:
    print(f"正在获取 {agent_name} 文档...")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"获取 {agent_name} 的详细信息和示例代码",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    if main_content:
        with open(f"langchain_tutorial/docs/agents/{agent_name}.md", 'w', encoding='utf-8') as f:
            f.write(f"# LangChain {agent_name}\n\n")
            f.write(f"来源: {url}\n\n")
            
            # 提取标题和内容
            headers = main_content.find_all(['h1', 'h2', 'h3'])
            for header in headers:
                header_text = header.get_text(strip=True)
                if header.name == 'h1':
                    f.write(f"# {header_text}\n\n")
                elif header.name == 'h2':
                    f.write(f"## {header_text}\n\n")
                else:
                    f.write(f"### {header_text}\n\n")
                
                # 获取段落内容和代码示例
                next_elem = header.find_next_sibling()
                while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                    if next_elem.name == 'p':
                        f.write(f"{next_elem.get_text(strip=True)}\n\n")
                    elif next_elem.name == 'pre':
                        code = next_elem.get_text()
                        f.write("```python\n")
                        f.write(code)
                        f.write("\n```\n\n")
                    elif next_elem.name in ['ul', 'ol']:
                        items = next_elem.find_all('li')
                        for item in items:
                            f.write(f"- {item.get_text(strip=True)}\n")
                        f.write("\n")
                    next_elem = next_elem.find_next_sibling()

# 获取搜索结合智能体相关文档
search_urls = [
    ("retrieval", "https://python.langchain.com/docs/modules/data_connection/retrievers/"),
    ("rag_overview", "https://python.langchain.com/docs/use_cases/question_answering/"),
    ("rag_retrieval", "https://python.langchain.com/docs/use_cases/question_answering/retrieval")
]

for search_name, url in search_urls:
    print(f"正在获取 {search_name} 文档...")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"获取 {search_name} 的详细信息和示例代码",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    if main_content:
        with open(f"langchain_tutorial/docs/search/{search_name}.md", 'w', encoding='utf-8') as f:
            f.write(f"# LangChain {search_name}\n\n")
            f.write(f"来源: {url}\n\n")
            
            # 提取标题和内容
            headers = main_content.find_all(['h1', 'h2', 'h3'])
            for header in headers:
                header_text = header.get_text(strip=True)
                if header.name == 'h1':
                    f.write(f"# {header_text}\n\n")
                elif header.name == 'h2':
                    f.write(f"## {header_text}\n\n")
                else:
                    f.write(f"### {header_text}\n\n")
                
                # 获取段落内容和代码示例
                next_elem = header.find_next_sibling()
                while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                    if next_elem.name == 'p':
                        f.write(f"{next_elem.get_text(strip=True)}\n\n")
                    elif next_elem.name == 'pre':
                        code = next_elem.get_text()
                        f.write("```python\n")
                        f.write(code)
                        f.write("\n```\n\n")
                    elif next_elem.name in ['ul', 'ol']:
                        items = next_elem.find_all('li')
                        for item in items:
                            f.write(f"- {item.get_text(strip=True)}\n")
                        f.write("\n")
                    next_elem = next_elem.find_next_sibling()

# 获取多智能体编排相关文档
multi_agent_urls = [
    ("agent_supervision", "https://python.langchain.com/docs/modules/agents/agent_supervision/"),
    ("executor", "https://python.langchain.com/docs/modules/agents/executor/"),
    ("tools", "https://python.langchain.com/docs/modules/tools/")
]

for agent_name, url in multi_agent_urls:
    print(f"正在获取 {agent_name} 文档...")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"获取 {agent_name} 的详细信息和示例代码",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    if main_content:
        with open(f"langchain_tutorial/docs/agents/{agent_name}_multi.md", 'w', encoding='utf-8') as f:
            f.write(f"# LangChain 多智能体 {agent_name}\n\n")
            f.write(f"来源: {url}\n\n")
            
            # 提取标题和内容
            headers = main_content.find_all(['h1', 'h2', 'h3'])
            for header in headers:
                header_text = header.get_text(strip=True)
                if header.name == 'h1':
                    f.write(f"# {header_text}\n\n")
                elif header.name == 'h2':
                    f.write(f"## {header_text}\n\n")
                else:
                    f.write(f"### {header_text}\n\n")
                
                # 获取段落内容和代码示例
                next_elem = header.find_next_sibling()
                while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                    if next_elem.name == 'p':
                        f.write(f"{next_elem.get_text(strip=True)}\n\n")
                    elif next_elem.name == 'pre':
                        code = next_elem.get_text()
                        f.write("```python\n")
                        f.write(code)
                        f.write("\n```\n\n")
                    elif next_elem.name in ['ul', 'ol']:
                        items = next_elem.find_all('li')
                        for item in items:
                            f.write(f"- {item.get_text(strip=True)}\n")
                        f.write("\n")
                    next_elem = next_elem.find_next_sibling()

# 获取表达式语言(LCEL)相关文档，这对多智能体编排很重要
lcel_urls = [
    ("lcel_intro", "https://python.langchain.com/docs/expression_language/"),
    ("lcel_interface", "https://python.langchain.com/docs/expression_language/interface"),
    ("lcel_cookbook", "https://python.langchain.com/docs/expression_language/cookbook")
]

for lcel_name, url in lcel_urls:
    print(f"正在获取 {lcel_name} 文档...")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"获取 {lcel_name} 的详细信息和示例代码",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    if main_content:
        with open(f"langchain_tutorial/docs/{lcel_name}.md", 'w', encoding='utf-8') as f:
            f.write(f"# LangChain 表达式语言 {lcel_name}\n\n")
            f.write(f"来源: {url}\n\n")
            
            # 提取标题和内容
            headers = main_content.find_all(['h1', 'h2', 'h3'])
            for header in headers:
                header_text = header.get_text(strip=True)
                if header.name == 'h1':
                    f.write(f"# {header_text}\n\n")
                elif header.name == 'h2':
                    f.write(f"## {header_text}\n\n")
                else:
                    f.write(f"### {header_text}\n\n")
                
                # 获取段落内容和代码示例
                next_elem = header.find_next_sibling()
                while next_elem and next_elem.name not in ['h1', 'h2', 'h3']:
                    if next_elem.name == 'p':
                        f.write(f"{next_elem.get_text(strip=True)}\n\n")
                    elif next_elem.name == 'pre':
                        code = next_elem.get_text()
                        f.write("```python\n")
                        f.write(code)
                        f.write("\n```\n\n")
                    elif next_elem.name in ['ul', 'ol']:
                        items = next_elem.find_all('li')
                        for item in items:
                            f.write(f"- {item.get_text(strip=True)}\n")
                        f.write("\n")
                    next_elem = next_elem.find_next_sibling()

# 创建框架总览文档
with open('langchain_tutorial/docs/overview.md', 'w', encoding='utf-8') as f:
    f.write("# LangChain 框架总览\n\n")
    f.write("## 基本组件\n\n")
    f.write("LangChain 框架由以下核心组件构成：\n\n")
    f.write("1. **模型 I/O (Model I/O)** - 处理与语言模型的交互\n")
    f.write("2. **提示模板 (Prompt Templates)** - 管理和优化发送给模型的提示\n")
    f.write("3. **输出解析器 (Output Parsers)** - 处理模型响应转换为结构化格式\n")
    f.write("4. **链 (Chains)** - 组合模型调用与其他组件\n")
    f.write("5. **记忆 (Memory)** - 维护对话状态\n")
    f.write("6. **检索 (Retrieval)** - 与外部数据源交互\n")
    f.write("7. **智能体 (Agents)** - 允许模型确定执行的操作\n\n")
    
    f.write("## 表达式语言 (LCEL)\n\n")
    f.write("LangChain 表达式语言 (LCEL) 是一种声明式方式，用于组合链和智能体。它通过简洁的操作符实现组件间的连接，并支持异步操作。\n\n")
    
    f.write("## 智能体类型\n\n")
    f.write("LangChain 支持多种智能体类型：\n\n")
    f.write("1. **ReAct 智能体** - 结合推理和行动的智能体\n")
    f.write("2. **工具调用智能体** - 专门设计用于工具调用的智能体\n")
    f.write("3. **计划与执行智能体** - 先规划后执行的智能体\n")
    f.write("4. **OpenAI 助手智能体** - 基于 OpenAI 助手 API 的智能体\n")
    f.write("5. **自定义智能体** - 根据特定需求创建的智能体\n\n")
    
    f.write("## 多智能体编排\n\n")
    f.write("LangChain 支持多智能体系统的编排，包括：\n\n")
    f.write("1. **智能体监督** - 管理多个智能体的协作\n")
    f.write("2. **智能体执行器** - 控制智能体执行流程\n")
    f.write("3. **工具共享** - 在多个智能体间共享工具\n\n")
    
    f.write("## 搜索与检索增强生成 (RAG)\n\n")
    f.write("LangChain 提供了强大的检索功能，支持：\n\n")
    f.write("1. **多种检索器** - 向量存储、关键词搜索等\n")
    f.write("2. **RAG 管道** - 结合检索与生成的端到端系统\n")
    f.write("3. **上下文增强** - 使用外部知识增强模型响应\n\n")

# 显示创建的文档结构
await terminal.run("find langchain_tutorial -type f | grep -v __pycache__ | sort")

正在获取 model_io 组件文档...
正在获取 memory 组件文档...
正在获取 prompt_templates 组件文档...
正在获取 output_parsers 组件文档...
正在获取 chains 组件文档...
正在获取 retrieval 组件文档...
正在获取 agent_overview 文档...
正在获取 agent_types 文档...
正在获取 agent_executors 文档...
正在获取 tools 文档...
正在获取 toolkits 文档...
正在获取 retrieval 文档...
正在获取 rag_overview 文档...
正在获取 rag_retrieval 文档...
正在获取 agent_supervision 文档...
正在获取 executor 文档...
正在获取 tools 文档...
正在获取 lcel_intro 文档...
正在获取 lcel_interface 文档...
正在获取 lcel_cookbook 文档...


2025-04-06 20:35:39.406 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:35:42.743 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:35:45.903 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:35:49.067 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:35:52.407 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:35:55.796 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:35:59.002 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:36:02.339 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 20:36:05.799 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06

'\nlangchain_tutorial/docs/agents.md\nlangchain_tutorial/docs/agents/agent_executors.md\nlangchain_tutorial/docs/agents/agent_overview.md\nlangchain_tutorial/docs/agents/agent_supervision_multi.md\nlangchain_tutorial/docs/agents/agent_types.md\nlangchain_tutorial/docs/agents/executor_multi.md\nlangchain_tutorial/docs/agents/toolkits.md\nlangchain_tutorial/docs/agents/tools.md\nlangchain_tutorial/docs/agents/tools_multi.md\nlangchain_tutorial/docs/components.md\nlangchain_tutorial/docs/components/chains.md\nlangchain_tutorial/docs/components/memory.md\nlangchain_tutorial/docs/components/model_io.md\nlangchain_tutorial/docs/components/output_parsers.md\nlangchain_tutorial/docs/components/prompt_templates.md\nlangchain_tutorial/docs/components/retrieval.md\nlangchain_tutorial/docs/document_structure.md\nlangchain_tutorial/docs/lcel_cookbook.md\nlangchain_tutorial/docs/lcel_interface.md\nlangchain_tutorial/docs/lcel_intro.md\nlangchain_tutorial/docs/overview.md\nlangchain_tutorial/docs/sea

In [3]:
# Create a comprehensive code example showcasing LangChain's components and multi-agent workflow
import os
from typing import List, Dict, Any, Optional, Tuple
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import create_async_playwright_browser
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.callbacks import get_openai_callback
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# 创建项目目录
terminal = Terminal()
await terminal.run("mkdir -p langchain_tutorial/code_examples/multi_agent_demo")

# Create a README file to explain the example
readme_content = """# LangChain 多智能体工作流示例

本示例展示了 LangChain 框架的核心组件和多智能体工作流程。示例包括：

1. 基本组件演示：提示模板、语言模型、链、记忆组件
2. RAG (检索增强生成) 示例：向量数据库、文档加载与分割、查询
3. 单智能体工作流：使用工具增强智能体能力
4. 多智能体协作系统：执行复杂任务的多智能体编排

## 环境设置

需要以下环境变量：
- OPENAI_API_KEY：OpenAI API 密钥

## 文件结构

- `multi_agent_workflow.py`：主要示例代码
- `data/`：示例数据文件
- `agents/`：定义的智能体
- `tools/`：可用工具

## 运行方式


SyntaxError: incomplete input (2285967862.py, line 31)

In [4]:
import os
from typing import List, Dict, Any, Optional, Tuple
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# 创建项目目录
terminal = Terminal()
await terminal.run("mkdir -p langchain_tutorial/code_examples/multi_agent_demo")

# Create a README file to explain the example
readme_content = """# LangChain 多智能体工作流示例

本示例展示了 LangChain 框架的核心组件和多智能体工作流程。示例包括：

1. 基本组件演示：提示模板、语言模型、链、记忆组件
2. RAG (检索增强生成) 示例：向量数据库、文档加载与分割、查询
3. 单智能体工作流：使用工具增强智能体能力
4. 多智能体协作系统：执行复杂任务的多智能体编排

## 环境设置

需要以下环境变量：
- OPENAI_API_KEY：OpenAI API 密钥

## 文件结构

- `multi_agent_workflow.py`：主要示例代码
- `data/`：示例数据文件
- `agents/`：定义的智能体
- `tools/`：可用工具

## 运行方式
python multi_agent_workflow.py
"""

# 将README写入到项目目录
editor = Editor()
editor.write("langchain_tutorial/code_examples/multi_agent_demo/README.md", readme_content)

# 创建示例代码文件
multi_agent_code = """# LangChain 多智能体工作流示例代码
import os
import sys
from typing import List, Dict, Any
from pathlib import Path

# 请先设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 1. 基础组件演示
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema import SystemMessage, HumanMessage, AIMessage

def demonstrate_basic_components():
    print("\\n1. 演示 LangChain 基础组件")
    
    # 创建一个提示模板
    template = "你是一个{role}。用户问题: {question}"
    prompt = PromptTemplate(
        input_variables=["role", "question"],
        template=template,
    )
    
    # 初始化LLM
    llm = ChatOpenAI(temperature=0.7)
    
    # 创建一个带有记忆的链
    memory = ConversationBufferMemory()
    chain = LLMChain(
        llm=llm, 
        prompt=prompt, 
        memory=memory,
        verbose=True
    )
    
    # 执行链
    response = chain.run(role="友好的AI助手", question="什么是LangChain框架?")
    print(f"\\n回答: {response}\\n")
    
    # 展示记忆功能
    response = chain.run(role="友好的AI助手", question="它的主要组件有哪些?")
    print(f"\\n回答: {response}\\n")
    
    return chain

# 2. 检索增强生成 (RAG) 示例
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

def create_rag_system():
    print("\\n2. 创建检索增强生成 (RAG) 系统")
    
    # 创建示例文档
    Path("data").mkdir(exist_ok=True)
    with open("data/langchain_info.txt", "w", encoding="utf-8") as f:
        f.write('''
LangChain 是一个用于开发由语言模型驱动的应用程序的框架。它有以下核心组件：

1. 模型 I/O: 处理与语言模型的交互，包括提示模板和输出解析。
2. 数据连接: 连接语言模型与各种数据源。
3. 链: 将多个组件组合成一个应用程序。
4. 记忆: 在链的运行之间持久化状态。
5. 智能体: 允许语言模型根据目标选择要使用的工具。
6. 回调: 在链执行期间钩入中间步骤。

LangChain 表达式语言 (LCEL) 是一种声明式语言，用于组合这些组件。它支持同步和异步操作，可以实现复杂的工作流程，如检索增强生成 (RAG)。

LangChain 还支持多种类型的智能体，如 ReAct、工具使用、规划与执行等。多智能体系统可以通过智能体监督功能进行协作。
        ''')
    
    # 加载并处理文档
    loader = TextLoader("data/langchain_info.txt")
    documents = loader.load()
    
    # 文本分块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=50
    )
    docs = text_splitter.split_documents(documents)
    
    # 创建向量存储
    embeddings = OpenAIEmbeddings()
    vectordb = FAISS.from_documents(docs, embeddings)
    
    # 创建一个检索链
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    retriever = vectordb.as_retriever()
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(),
        retriever=retriever,
        memory=memory,
        verbose=True
    )
    
    # 执行检索问答
    result = qa_chain({"question": "LangChain有哪些核心组件?"})
    print(f"\\nRAG回答: {result['answer']}\\n")
    
    return qa_chain

# 3. 单智能体工作流
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools.python.tool import PythonREPLTool

def create_single_agent():
    print("\\n3. 创建带工具的单智能体系统")
    
    # 定义工具
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="Wikipedia",
            func=wikipedia.run,
            description="查询维基百科以获取信息"
        ),
        Tool(
            name="Python REPL",
            func=python_repl.run,
            description="执行Python代码"
        )
    ]
    
    # 初始化智能体
    llm = ChatOpenAI(temperature=0)
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )
    
    # 运行智能体
    response = agent.run("LangChain框架是什么时候发布的? 然后计算从发布到现在经过了多少天。")
    print(f"\\n智能体回答: {response}\\n")
    
    return agent

# 4. 多智能体系统
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import BaseChatPromptTemplate
from langchain import LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re

# 定义自定义智能体
class MultiAgentSystem:
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.research_agent = self._create_research_agent()
        self.planning_agent = self._create_planning_agent()
        self.execution_agent = self._create_execution_agent()
    
    def _create_research_agent(self):
        # 创建研究智能体
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        
        tools = [
            Tool(
                name="Wikipedia",
                func=wikipedia.run,
                description="查询维基百科以获取信息"
            )
        ]
        
        agent = initialize_agent(
            tools=tools,
            llm=self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        return agent
    
    def _create_planning_agent(self):
        # 创建规划智能体
        prompt = PromptTemplate(
            template="你是一个规划专家。根据下面的任务描述创建一个分步计划:\\n{task}",
            input_variables=["task"]
        )
        
        planning_chain = LLMChain(llm=self.llm, prompt=prompt)
        return planning_chain
    
    def _create_execution_agent(self):
        # 创建执行智能体
        python_repl = PythonREPLTool()
        
        tools = [
            Tool(
                name="Python REPL",
                func=python_repl.run,
                description="执行Python代码"
            )
        ]
        
        agent = initialize_agent(
            tools=tools,
            llm=self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        return agent
    
    def run(self, task):
        print(f"\\n4. 执行多智能体工作流 - 任务: {task}")
        
        # 1. 研究阶段
        print("\\n步骤1: 研究阶段")
        research_result = self.research_agent.run(f"研究关于: {task}")
        print(f"研究结果: {research_result}")
        
        # 2. 规划阶段
        print("\\n步骤2: 规划阶段")
        plan = self.planning_chain.run(task=f"{task}\\n背景信息: {research_result}")
        print(f"执行计划: {plan}")
        
        # 3. 执行阶段
        print("\\n步骤3: 执行阶段")
        final_result = self.execution_agent.run(
            f"根据以下计划执行任务: {plan}\\n任务: {task}"
        )
        print(f"最终结果: {final_result}")
        
        return {
            "research": research_result,
            "plan": plan,
            "result": final_result
        }

def main():
    print("LangChain 多智能体工作流示例")
    
    # 运行基础组件演示
    basic_chain = demonstrate_basic_components()
    
    # 创建RAG系统
    rag_chain = create_rag_system()
    
    # 创建单智能体系统
    agent = create_single_agent()
    
    # 创建多智能体系统
    multi_agent_system = MultiAgentSystem()
    result = multi_agent_system.run("创建一个简单的网站访问量分析程序")
    
    print("\\n演示完成!")

if __name__ == "__main__":
    main()
"""

# 写入主代码文件
editor.write("langchain_tutorial/code_examples/multi_agent_demo/multi_agent_workflow.py", multi_agent_code)

# 创建一个简单的工具库，用于多智能体系统
tools_code = """# 自定义工具库
from langchain.tools import BaseTool
from langchain.agents import Tool
from typing import Dict, List, Any

class WebSearchTool(BaseTool):
    name = "web_search"
    description = "在网络上搜索信息"
    
    def _run(self, query: str) -> str:
        # 模拟网络搜索
        return f"关于'{query}'的搜索结果: 这里是一些相关信息..."
    
    async def _arun(self, query: str) -> str:
        return self._run(query)

class DataAnalysisTool(BaseTool):
    name = "data_analysis"
    description = "分析数据集并提供洞察"
    
    def _run(self, data_description: str) -> str:
        # 模拟数据分析
        return f"对'{data_description}'的分析结果: 数据显示以下模式..."
    
    async def _arun(self, data_description: str) -> str:
        return self._run(data_description)

class CodeGenerationTool(BaseTool):
    name = "code_generation"
    description = "生成Python代码来解决特定问题"
    
    def _run(self, task: str) -> str:
        # 模拟代码生成
        return f"为'{task}'生成的代码:\\n```python\\ndef solution():\\n    # 实现代码\\n    print('解决方案')\\n```"
    
    async def _arun(self, task: str) -> str:
        return self._run(task)

def get_all_tools() -> List[Tool]:
    \"\"\"获取所有可用工具\"\"\"
    return [
        Tool(
            name="Web搜索",
            func=WebSearchTool()._run,
            description="在网络上搜索信息"
        ),
        Tool(
            name="数据分析",
            func=DataAnalysisTool()._run,
            description="分析数据集并提供洞察"
        ),
        Tool(
            name="代码生成",
            func=CodeGenerationTool()._run,
            description="生成Python代码来解决特定问题"
        )
    ]
"""

# 写入工具库文件
await terminal.run("mkdir -p langchain_tutorial/code_examples/multi_agent_demo/tools")
editor.write("langchain_tutorial/code_examples/multi_agent_demo/tools/custom_tools.py", tools_code)

# 创建多智能体定义文件
agents_code = """# 定义多智能体系统中的各类智能体
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish
from typing import List, Union, Dict, Any
import re

# 基础智能体定义
class BaseSpecialistAgent:
    def __init__(self, name, role_description, tools=None):
        self.name = name
        self.role_description = role_description
        self.tools = tools or []
        self.llm = ChatOpenAI(temperature=0)
        self.agent_executor = self._create_agent_executor()
    
    def _create_agent_executor(self):
        # 这里实现具体的智能体执行器
        # 简化示例，实际中需根据不同智能体类型定制
        pass
        
    def run(self, task):
        return f"{self.name} 处理任务: {task}"

# 研究员智能体
class ResearcherAgent(BaseSpecialistAgent):
    def __init__(self, tools=None):
        super().__init__(
            name="Researcher",
            role_description="你是一名研究专家，擅长收集和分析信息",
            tools=tools
        )
    
    def run(self, query):
        # 简化的研究流程
        return f"研究结果: 关于'{query}'的详细信息和分析"

# 规划师智能体
class PlannerAgent(BaseSpecialistAgent):
    def __init__(self, tools=None):
        super().__init__(
            name="Planner",
            role_description="你是一名规划专家，擅长制定详细的执行计划",
            tools=tools
        )
    
    def run(self, task, context=None):
        context_str = f"\\n背景信息: {context}" if context else ""
        return f"执行计划:\\n1. 分析问题\\n2. 设计解决方案\\n3. 实施方案\\n4. 测试结果"

# 编码员智能体
class CoderAgent(BaseSpecialistAgent):
    def __init__(self, tools=None):
        super().__init__(
            name="Coder",
            role_description="你是一名编程专家，擅长编写和调试代码",
            tools=tools
        )
    
    def run(self, task, plan=None):
        plan_str = f"\\n按照计划: {plan}" if plan else ""
        return f"代码实现:\\n```python\\n# {task}的代码实现\\nprint('Hello world!')\\n```"

# 测试员智能体
class TesterAgent(BaseSpecialistAgent):
    def __init__(self, tools=None):
        super().__init__(
            name="Tester",
            role_description="你是一名测试专家，擅长验证解决方案和提供反馈",
            tools=tools
        )
    
    def run(self, solution):
        return f"测试结果: 解决方案已验证，运行正常"

# 多智能体协调器
class AgentCoordinator:
    def __init__(self):
        self.researcher = ResearcherAgent()
        self.planner = PlannerAgent()
        self.coder = CoderAgent()
        self.tester = TesterAgent()
    
    def run_workflow(self, task):
        results = {}
        
        # 1. 研究阶段
        research_result = self.researcher.run(task)
        results["research"] = research_result
        
        # 2. 规划阶段
        plan = self.planner.run(task, context=research_result)
        results["plan"] = plan
        
        # 3. 编码阶段
        code = self.coder.run(task, plan=plan)
        results["code"] = code
        
        # 4. 测试阶段
        test_result = self.tester.run(code)
        results["test"] = test_result
        
        return results
"""

# 写入智能体文件
await terminal.run("mkdir -p langchain_tutorial/code_examples/multi_agent_demo/agents")
editor.write("langchain_tutorial/code_examples/multi_agent_demo/agents/specialist_agents.py", agents_code)

# 创建一个使用LCEL的示例
lcel_example = """# LangChain 表达式语言 (LCEL) 示例
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from typing import Dict, List, Any

# 基础链
def simple_chain_example():
    # 1. 定义提示模板
    prompt = PromptTemplate.from_template("向我介绍{topic}")
    
    # 2. 定义语言模型
    model = ChatOpenAI()
    
    # 3. 定义解析器
    output_parser = StrOutputParser()
    
    # 4. 使用LCEL组合成一个链
    chain = prompt | model | output_parser
    
    # 5. 执行链
    response = chain.invoke({"topic": "LangChain表达式语言"})
    print("\\n简单链示例:")
    print(response)
    
    return chain

# RAG 链
def rag_chain_example():
    # 1. 准备文档
    with open("data/rag_example.txt", "w", encoding="utf-8") as f:
        f.write('''
LangChain 表达式语言 (LCEL) 是一种声明式语言，设计用于组合链和其他组件。
LCEL 的主要优势包括：
1. 简洁的接口: 使用 | 操作符连接组件
2. 异步支持: 提供同步和异步执行选项
3. 流媒体支持: 支持流式响应
4. 批处理支持: 可以高效处理多个输入
5. 可重试性: 内置的重试机制

LCEL 支持多种组件，包括 Runnable, Chain, PromptTemplate, ChatModel 等。
        ''')
    
    # 2. 加载和分割文档
    loader = TextLoader("data/rag_example.txt")
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
    splits = text_splitter.split_documents(documents)
    
    # 3. 创建向量存储
    vectorstore = FAISS.from_documents(splits, OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()
    
    # 4. 定义RAG提示模板
    template = \"\"\"使用以下上下文回答问题。如果你不知道答案，就说你不知道。

上下文:
{context}

问题: {question}
\"\"\"
    prompt = PromptTemplate.from_template(template)
    
    # 5. 使用LCEL组合成RAG链
    model = ChatOpenAI(temperature=0)
    
    def format_docs(docs):
        return "\\n\\n".join(doc.page_content for doc in docs)
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )
    
    # 6. 执行RAG链
    response = rag_chain.invoke("什么是LCEL的主要优势?")
    print("\\nRAG链示例:")
    print(response)
    
    return rag_chain

def main():
    # 执行简单链示例
    simple_chain = simple_chain_example()
    
    # 执行RAG链示例
    rag_chain = rag_chain_example()

if __name__ == "__main__":
    main()
"""

# 写入LCEL示例文件
editor.write("langchain_tutorial/code_examples/multi_agent_demo/lcel_example.py", lcel_example)

# 列出创建的文件
file_structure = await terminal.run("find langchain_tutorial/code_examples/multi_agent_demo -type f | sort")
print(f"创建的示例文件结构:\n{file_structure}")

# 总结创建的内容
print("\n已创建LangChain多智能体工作流程示例代码，包含以下内容:")
print("1. 基础组件示例: 提示模板、语言模型、链、记忆组件")
print("2. RAG系统实现: 文档加载、分块、向量存储和检索问答")
print("3. 单智能体工作流: 使用工具增强的智能体")
print("4. 多智能体协作系统: 研究、规划、编码和测试的专家智能体组合")
print("5. LangChain表达式语言(LCEL)示例: 声明式链组合")

创建的示例文件结构:

langchain_tutorial/code_examples/multi_agent_demo/README.md
langchain_tutorial/code_examples/multi_agent_demo/agents/specialist_agents.py
langchain_tutorial/code_examples/multi_agent_demo/lcel_example.py
langchain_tutorial/code_examples/multi_agent_demo/multi_agent_workflow.py
langchain_tutorial/code_examples/multi_agent_demo/tools/custom_tools.py


已创建LangChain多智能体工作流程示例代码，包含以下内容:
1. 基础组件示例: 提示模板、语言模型、链、记忆组件
2. RAG系统实现: 文档加载、分块、向量存储和检索问答
3. 单智能体工作流: 使用工具增强的智能体
4. 多智能体协作系统: 研究、规划、编码和测试的专家智能体组合
5. LangChain表达式语言(LCEL)示例: 声明式链组合


In [5]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os
import asyncio

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for the comprehensive documentation
await terminal.run("mkdir -p langchain_tutorial/documentation")

# Create the comprehensive summary document
summary_content = """# LangChain 框架综合指南

## 1. 框架概述

LangChain 是一个用于开发由大型语言模型（LLM）驱动的应用程序的框架，其设计目标是让开发者能够轻松构建结合外部数据、上下文感知和推理能力的智能应用。

### 1.1 核心理念

LangChain 的核心理念是将语言模型与其他计算资源和知识来源相结合，使 AI 应用不仅能够生成文本，还能够：
- 访问和处理特定领域的知识
- 与数据库和 API 交互
- 链接多个推理步骤
- 根据实际情况做出决策

### 1.2 应用场景

LangChain 框架适用于各种应用场景，包括但不限于：
- 聊天机器人和对话系统
- 问答系统（基于文档的问答）
- 文本摘要与分析
- 智能助手和代理
- 内容生成与创意写作
- 数据分析与解释

## 2. 架构与核心组件

LangChain 框架的架构由以下核心组件构成：

### 2.1 模型 I/O (Model I/O)

负责处理与语言模型的交互，包括：
- **语言模型 (LLMs/Chat Models)**: 与各种 LLM 提供商（如 OpenAI, Anthropic, Hugging Face 等）的集成
- **提示模板 (Prompt Templates)**: 创建和管理提示
- **输出解析器 (Output Parsers)**: 将模型输出转换为结构化格式

### 2.2 数据连接 (Data Connection)

用于连接语言模型与各种数据源：
- **文档加载器 (Document Loaders)**: 从各种源加载文档
- **文档转换器 (Document Transformers)**: 分割和转换文档
- **文本嵌入模型 (Text Embedding Models)**: 将文本转换为向量表示
- **向量存储 (Vector Stores)**: 存储和检索向量嵌入
- **检索器 (Retrievers)**: 检索相关信息

### 2.3 链 (Chains)

将多个组件组合成应用程序的序列：
- **基础链 (Basic Chains)**: 简单的串行操作
- **顺序链 (Sequential Chains)**: 多个链按顺序执行
- **转换链 (Transform Chains)**: 在链之间转换输入/输出
- **用例特定链 (Use-case Specific Chains)**: 针对特定场景优化的链

### 2.4 记忆 (Memory)

在链的执行之间维护状态：
- **对话记忆 (Conversation Memory)**: 记录对话历史
- **实体记忆 (Entity Memory)**: 记录特定实体的信息
- **长期记忆 (Long-term Memory)**: 跨会话持久化信息
- **总结记忆 (Summary Memory)**: 维护对话摘要

### 2.5 智能体 (Agents)

允许语言模型决定采取的行动：
- **智能体类型 (Agent Types)**: 如 ReAct, 工具使用, 规划与执行
- **智能体工具 (Agent Tools)**: 智能体可以使用的功能
- **智能体执行器 (Agent Executors)**: 控制智能体执行流程
- **智能体工具包 (Agent Toolkits)**: 预定义的工具集合

### 2.6 回调 (Callbacks)

允许在链执行期间钩入中间步骤：
- **记录 (Logging)**: 跟踪执行过程
- **流式处理 (Streaming)**: 实时处理输出
- **监控 (Monitoring)**: 监控执行状态

## 3. LangChain 表达式语言 (LCEL)

LangChain 表达式语言是一种声明式方法，用于组合链和智能体。

### 3.1 基本概念

- **Runnable 接口**: 所有组件统一接口
- **管道操作符**: 使用 `|` 连接组件
- **异步支持**: 提供同步和异步执行选项
- **流式传输**: 支持流式响应
- **批处理**: 高效处理多个输入

### 3.2 优势

- **简洁的接口**: 更清晰的组件连接方式
- **一致的调用模式**: invoke, batch, stream 等统一方法
- **内置错误处理**: 包含重试和回退机制
- **扩展性**: 易于添加新组件

## 4. 单智能体实现

### 4.1 智能体类型

1. **ReAct 智能体**: 结合推理（Reasoning）和行动（Acting）的智能体
2. **工具调用智能体**: 专为工具调用优化的智能体
3. **规划与执行智能体**: 先制定计划再执行的智能体
4. **OpenAI 助手智能体**: 基于 OpenAI 助手 API 的智能体
5. **自定义智能体**: 根据特定需求定制的智能体

### 4.2 实现步骤

1. **定义工具**: 创建智能体可以使用的工具
2. **选择智能体类型**: 根据需求选择适当的智能体类型
3. **配置智能体**: 设置提示模板和响应格式
4. **创建执行器**: 配置智能体执行环境
5. **运行智能体**: 处理用户查询并执行操作

## 5. 多智能体编排

### 5.1 多智能体架构

- **协作模式**: 多个智能体协同工作的方式
- **通信机制**: 智能体之间的信息交换
- **角色分配**: 不同专业领域的智能体

### 5.2 智能体监督

- **主管智能体**: 协调其他智能体工作
- **任务分配**: 将复杂任务分解为子任务
- **结果整合**: 汇总各个智能体的输出

### 5.3 实现方法

1. **定义专家智能体**: 创建具有特定专长的智能体
2. **设置监督机制**: 配置主管智能体
3. **建立通信渠道**: 实现智能体之间的信息交换
4. **制定协作流程**: 定义工作流和决策机制
5. **执行与监控**: 启动系统并监控运行情况

## 6. 搜索与检索增强生成 (RAG)

### 6.1 基本概念

检索增强生成 (Retrieval-Augmented Generation, RAG) 通过结合检索系统和生成模型，使语言模型能够访问外部知识。

### 6.2 组件

- **文档处理**: 加载、分割和处理文档
- **嵌入生成**: 创建文本的向量表示
- **向量存储**: 存储和索引嵌入
- **检索系统**: 根据查询获取相关内容
- **上下文增强**: 将检索到的内容与查询结合
- **生成系统**: 基于增强上下文生成回答

### 6.3 实现步骤

1. **准备数据**: 加载和预处理文档
2. **创建向量存储**: 生成嵌入并存储
3. **构建检索器**: 配置检索策略
4. **设计提示模板**: 将检索结果融入提示
5. **配置生成模型**: 设置语言模型参数
6. **组装 RAG 链**: 使用 LCEL 连接各组件
7. **优化与评估**: 测试和改进系统性能

## 7. 多智能体工作流设计

### 7.1 工作流组件

- **专家智能体**: 具有特定领域知识的智能体
- **工具集成**: 与外部系统和数据源的连接
- **协作机制**: 智能体之间的互动规则
- **决策流程**: 如何做出和执行决策
- **监控系统**: 追踪工作流执行情况

### 7.2 常见模式

1. **研究-规划-执行**: 先研究问题，制定计划，再执行
2. **分解-征询-整合**: 将问题分解，征求专家意见，整合结果
3. **反馈循环**: 执行-评估-调整的迭代过程
4. **层级决策**: 不同层级的智能体负责不同抽象级别的决策

### 7.3 实现案例

多智能体系统可以包含如下角色：
- **研究员智能体**: 收集和分析信息
- **规划师智能体**: 制定执行计划
- **编码员智能体**: 编写和调试代码
- **测试员智能体**: 验证解决方案
- **协调器**: 管理整个工作流程

## 8. 最佳实践与优化

### 8.1 性能优化

- **批量处理**: 使用批处理减少 API 调用
- **缓存机制**: 缓存常用查询和结果
- **并行执行**: 并行化独立任务
- **模型选择**: 针对不同任务使用适当的模型

### 8.2 提示工程

- **任务分解**: 将复杂任务分解为简单步骤
- **上下文增强**: 提供充分的背景信息
- **指令明确性**: 给出清晰的指令
- **示例驱动**: 使用少样本学习

### 8.3 错误处理

- **异常捕获**: 处理可能的错误情况
- **重试机制**: 为不稳定操作实现重试
- **回退策略**: 当首选方法失败时的替代方案
- **用户反馈**: 适当时寻求用户输入

## 9. 资源与参考

- **官方文档**: https://python.langchain.com/docs/introduction/
- **GitHub 仓库**: https://github.com/langchain-ai/langchain
- **示例集**: https://python.langchain.com/docs/use_cases/
"""

# Write the comprehensive guide
editor.write("langchain_tutorial/documentation/langchain_comprehensive_guide.md", summary_content)

# Create a practical code examples library document
code_examples = """# LangChain 实用代码示例集

## 1. 基础组件示例

### 1.1 语言模型和提示模板


SyntaxError: incomplete input (387482405.py, line 233)

In [6]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os
import asyncio

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for the comprehensive documentation
await terminal.run("mkdir -p langchain_tutorial/documentation")

# Create the comprehensive summary document
summary_content = """# LangChain 框架综合指南

## 1. 框架概述

LangChain 是一个用于开发由大型语言模型（LLM）驱动的应用程序的框架，其设计目标是让开发者能够轻松构建结合外部数据、上下文感知和推理能力的智能应用。

### 1.1 核心理念

LangChain 的核心理念是将语言模型与其他计算资源和知识来源相结合，使 AI 应用不仅能够生成文本，还能够：
- 访问和处理特定领域的知识
- 与数据库和 API 交互
- 链接多个推理步骤
- 根据实际情况做出决策

### 1.2 应用场景

LangChain 框架适用于各种应用场景，包括但不限于：
- 聊天机器人和对话系统
- 问答系统（基于文档的问答）
- 文本摘要与分析
- 智能助手和代理
- 内容生成与创意写作
- 数据分析与解释

## 2. 架构与核心组件

LangChain 框架的架构由以下核心组件构成：

### 2.1 模型 I/O (Model I/O)

负责处理与语言模型的交互，包括：
- **语言模型 (LLMs/Chat Models)**: 与各种 LLM 提供商（如 OpenAI, Anthropic, Hugging Face 等）的集成
- **提示模板 (Prompt Templates)**: 创建和管理提示
- **输出解析器 (Output Parsers)**: 将模型输出转换为结构化格式

### 2.2 数据连接 (Data Connection)

用于连接语言模型与各种数据源：
- **文档加载器 (Document Loaders)**: 从各种源加载文档
- **文档转换器 (Document Transformers)**: 分割和转换文档
- **文本嵌入模型 (Text Embedding Models)**: 将文本转换为向量表示
- **向量存储 (Vector Stores)**: 存储和检索向量嵌入
- **检索器 (Retrievers)**: 检索相关信息

### 2.3 链 (Chains)

将多个组件组合成应用程序的序列：
- **基础链 (Basic Chains)**: 简单的串行操作
- **顺序链 (Sequential Chains)**: 多个链按顺序执行
- **转换链 (Transform Chains)**: 在链之间转换输入/输出
- **用例特定链 (Use-case Specific Chains)**: 针对特定场景优化的链

### 2.4 记忆 (Memory)

在链的执行之间维护状态：
- **对话记忆 (Conversation Memory)**: 记录对话历史
- **实体记忆 (Entity Memory)**: 记录特定实体的信息
- **长期记忆 (Long-term Memory)**: 跨会话持久化信息
- **总结记忆 (Summary Memory)**: 维护对话摘要

### 2.5 智能体 (Agents)

允许语言模型决定采取的行动：
- **智能体类型 (Agent Types)**: 如 ReAct, 工具使用, 规划与执行
- **智能体工具 (Agent Tools)**: 智能体可以使用的功能
- **智能体执行器 (Agent Executors)**: 控制智能体执行流程
- **智能体工具包 (Agent Toolkits)**: 预定义的工具集合

### 2.6 回调 (Callbacks)

允许在链执行期间钩入中间步骤：
- **记录 (Logging)**: 跟踪执行过程
- **流式处理 (Streaming)**: 实时处理输出
- **监控 (Monitoring)**: 监控执行状态

## 3. LangChain 表达式语言 (LCEL)

LangChain 表达式语言是一种声明式方法，用于组合链和智能体。

### 3.1 基本概念

- **Runnable 接口**: 所有组件统一接口
- **管道操作符**: 使用 `|` 连接组件
- **异步支持**: 提供同步和异步执行选项
- **流式传输**: 支持流式响应
- **批处理**: 高效处理多个输入

### 3.2 优势

- **简洁的接口**: 更清晰的组件连接方式
- **一致的调用模式**: invoke, batch, stream 等统一方法
- **内置错误处理**: 包含重试和回退机制
- **扩展性**: 易于添加新组件

## 4. 单智能体实现

### 4.1 智能体类型

1. **ReAct 智能体**: 结合推理（Reasoning）和行动（Acting）的智能体
2. **工具调用智能体**: 专为工具调用优化的智能体
3. **规划与执行智能体**: 先制定计划再执行的智能体
4. **OpenAI 助手智能体**: 基于 OpenAI 助手 API 的智能体
5. **自定义智能体**: 根据特定需求定制的智能体

### 4.2 实现步骤

1. **定义工具**: 创建智能体可以使用的工具
2. **选择智能体类型**: 根据需求选择适当的智能体类型
3. **配置智能体**: 设置提示模板和响应格式
4. **创建执行器**: 配置智能体执行环境
5. **运行智能体**: 处理用户查询并执行操作

## 5. 多智能体编排

### 5.1 多智能体架构

- **协作模式**: 多个智能体协同工作的方式
- **通信机制**: 智能体之间的信息交换
- **角色分配**: 不同专业领域的智能体

### 5.2 智能体监督

- **主管智能体**: 协调其他智能体工作
- **任务分配**: 将复杂任务分解为子任务
- **结果整合**: 汇总各个智能体的输出

### 5.3 实现方法

1. **定义专家智能体**: 创建具有特定专长的智能体
2. **设置监督机制**: 配置主管智能体
3. **建立通信渠道**: 实现智能体之间的信息交换
4. **制定协作流程**: 定义工作流和决策机制
5. **执行与监控**: 启动系统并监控运行情况

## 6. 搜索与检索增强生成 (RAG)

### 6.1 基本概念

检索增强生成 (Retrieval-Augmented Generation, RAG) 通过结合检索系统和生成模型，使语言模型能够访问外部知识。

### 6.2 组件

- **文档处理**: 加载、分割和处理文档
- **嵌入生成**: 创建文本的向量表示
- **向量存储**: 存储和索引嵌入
- **检索系统**: 根据查询获取相关内容
- **上下文增强**: 将检索到的内容与查询结合
- **生成系统**: 基于增强上下文生成回答

### 6.3 实现步骤

1. **准备数据**: 加载和预处理文档
2. **创建向量存储**: 生成嵌入并存储
3. **构建检索器**: 配置检索策略
4. **设计提示模板**: 将检索结果融入提示
5. **配置生成模型**: 设置语言模型参数
6. **组装 RAG 链**: 使用 LCEL 连接各组件
7. **优化与评估**: 测试和改进系统性能

## 7. 多智能体工作流设计

### 7.1 工作流组件

- **专家智能体**: 具有特定领域知识的智能体
- **工具集成**: 与外部系统和数据源的连接
- **协作机制**: 智能体之间的互动规则
- **决策流程**: 如何做出和执行决策
- **监控系统**: 追踪工作流执行情况

### 7.2 常见模式

1. **研究-规划-执行**: 先研究问题，制定计划，再执行
2. **分解-征询-整合**: 将问题分解，征求专家意见，整合结果
3. **反馈循环**: 执行-评估-调整的迭代过程
4. **层级决策**: 不同层级的智能体负责不同抽象级别的决策

### 7.3 实现案例

多智能体系统可以包含如下角色：
- **研究员智能体**: 收集和分析信息
- **规划师智能体**: 制定执行计划
- **编码员智能体**: 编写和调试代码
- **测试员智能体**: 验证解决方案
- **协调器**: 管理整个工作流程

## 8. 最佳实践与优化

### 8.1 性能优化

- **批量处理**: 使用批处理减少 API 调用
- **缓存机制**: 缓存常用查询和结果
- **并行执行**: 并行化独立任务
- **模型选择**: 针对不同任务使用适当的模型

### 8.2 提示工程

- **任务分解**: 将复杂任务分解为简单步骤
- **上下文增强**: 提供充分的背景信息
- **指令明确性**: 给出清晰的指令
- **示例驱动**: 使用少样本学习

### 8.3 错误处理

- **异常捕获**: 处理可能的错误情况
- **重试机制**: 为不稳定操作实现重试
- **回退策略**: 当首选方法失败时的替代方案
- **用户反馈**: 适当时寻求用户输入

## 9. 资源与参考

- **官方文档**: https://python.langchain.com/docs/introduction/
- **GitHub 仓库**: https://github.com/langchain-ai/langchain
- **示例集**: https://python.langchain.com/docs/use_cases/
"""

# Write the comprehensive guide
editor.write("langchain_tutorial/documentation/langchain_comprehensive_guide.md", summary_content)

# Create a practical code examples library document
code_examples = """# LangChain 实用代码示例集

## 1. 基础组件示例

### 1.1 语言模型和提示模板


SyntaxError: incomplete input (387482405.py, line 233)

In [7]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os
import asyncio

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for the comprehensive documentation
await terminal.run("mkdir -p langchain_tutorial/documentation")

# Create the comprehensive summary document
summary_content = """# LangChain 框架综合指南

## 1. 框架概述

LangChain 是一个用于开发由大型语言模型（LLM）驱动的应用程序的框架，其设计目标是让开发者能够轻松构建结合外部数据、上下文感知和推理能力的智能应用。

### 1.1 核心理念

LangChain 的核心理念是将语言模型与其他计算资源和知识来源相结合，使 AI 应用不仅能够生成文本，还能够：
- 访问和处理特定领域的知识
- 与数据库和 API 交互
- 链接多个推理步骤
- 根据实际情况做出决策

### 1.2 应用场景

LangChain 框架适用于各种应用场景，包括但不限于：
- 聊天机器人和对话系统
- 问答系统（基于文档的问答）
- 文本摘要与分析
- 智能助手和代理
- 内容生成与创意写作
- 数据分析与解释

## 2. 架构与核心组件

LangChain 框架的架构由以下核心组件构成：

### 2.1 模型 I/O (Model I/O)

负责处理与语言模型的交互，包括：
- **语言模型 (LLMs/Chat Models)**: 与各种 LLM 提供商（如 OpenAI, Anthropic, Hugging Face 等）的集成
- **提示模板 (Prompt Templates)**: 创建和管理提示
- **输出解析器 (Output Parsers)**: 将模型输出转换为结构化格式

### 2.2 数据连接 (Data Connection)

用于连接语言模型与各种数据源：
- **文档加载器 (Document Loaders)**: 从各种源加载文档
- **文档转换器 (Document Transformers)**: 分割和转换文档
- **文本嵌入模型 (Text Embedding Models)**: 将文本转换为向量表示
- **向量存储 (Vector Stores)**: 存储和检索向量嵌入
- **检索器 (Retrievers)**: 检索相关信息

### 2.3 链 (Chains)

将多个组件组合成应用程序的序列：
- **基础链 (Basic Chains)**: 简单的串行操作
- **顺序链 (Sequential Chains)**: 多个链按顺序执行
- **转换链 (Transform Chains)**: 在链之间转换输入/输出
- **用例特定链 (Use-case Specific Chains)**: 针对特定场景优化的链

### 2.4 记忆 (Memory)

在链的执行之间维护状态：
- **对话记忆 (Conversation Memory)**: 记录对话历史
- **实体记忆 (Entity Memory)**: 记录特定实体的信息
- **长期记忆 (Long-term Memory)**: 跨会话持久化信息
- **总结记忆 (Summary Memory)**: 维护对话摘要

### 2.5 智能体 (Agents)

允许语言模型决定采取的行动：
- **智能体类型 (Agent Types)**: 如 ReAct, 工具使用, 规划与执行
- **智能体工具 (Agent Tools)**: 智能体可以使用的功能
- **智能体执行器 (Agent Executors)**: 控制智能体执行流程
- **智能体工具包 (Agent Toolkits)**: 预定义的工具集合

### 2.6 回调 (Callbacks)

允许在链执行期间钩入中间步骤：
- **记录 (Logging)**: 跟踪执行过程
- **流式处理 (Streaming)**: 实时处理输出
- **监控 (Monitoring)**: 监控执行状态

## 3. LangChain 表达式语言 (LCEL)

LangChain 表达式语言是一种声明式方法，用于组合链和智能体。

### 3.1 基本概念

- **Runnable 接口**: 所有组件统一接口
- **管道操作符**: 使用 `|` 连接组件
- **异步支持**: 提供同步和异步执行选项
- **流式传输**: 支持流式响应
- **批处理**: 高效处理多个输入

### 3.2 优势

- **简洁的接口**: 更清晰的组件连接方式
- **一致的调用模式**: invoke, batch, stream 等统一方法
- **内置错误处理**: 包含重试和回退机制
- **扩展性**: 易于添加新组件

## 4. 单智能体实现

### 4.1 智能体类型

1. **ReAct 智能体**: 结合推理（Reasoning）和行动（Acting）的智能体
2. **工具调用智能体**: 专为工具调用优化的智能体
3. **规划与执行智能体**: 先制定计划再执行的智能体
4. **OpenAI 助手智能体**: 基于 OpenAI 助手 API 的智能体
5. **自定义智能体**: 根据特定需求定制的智能体

### 4.2 实现步骤

1. **定义工具**: 创建智能体可以使用的工具
2. **选择智能体类型**: 根据需求选择适当的智能体类型
3. **配置智能体**: 设置提示模板和响应格式
4. **创建执行器**: 配置智能体执行环境
5. **运行智能体**: 处理用户查询并执行操作

## 5. 多智能体编排

### 5.1 多智能体架构

- **协作模式**: 多个智能体协同工作的方式
- **通信机制**: 智能体之间的信息交换
- **角色分配**: 不同专业领域的智能体

### 5.2 智能体监督

- **主管智能体**: 协调其他智能体工作
- **任务分配**: 将复杂任务分解为子任务
- **结果整合**: 汇总各个智能体的输出

### 5.3 实现方法

1. **定义专家智能体**: 创建具有特定专长的智能体
2. **设置监督机制**: 配置主管智能体
3. **建立通信渠道**: 实现智能体之间的信息交换
4. **制定协作流程**: 定义工作流和决策机制
5. **执行与监控**: 启动系统并监控运行情况

## 6. 搜索与检索增强生成 (RAG)

### 6.1 基本概念

检索增强生成 (Retrieval-Augmented Generation, RAG) 通过结合检索系统和生成模型，使语言模型能够访问外部知识。

### 6.2 组件

- **文档处理**: 加载、分割和处理文档
- **嵌入生成**: 创建文本的向量表示
- **向量存储**: 存储和索引嵌入
- **检索系统**: 根据查询获取相关内容
- **上下文增强**: 将检索到的内容与查询结合
- **生成系统**: 基于增强上下文生成回答

### 6.3 实现步骤

1. **准备数据**: 加载和预处理文档
2. **创建向量存储**: 生成嵌入并存储
3. **构建检索器**: 配置检索策略
4. **设计提示模板**: 将检索结果融入提示
5. **配置生成模型**: 设置语言模型参数
6. **组装 RAG 链**: 使用 LCEL 连接各组件
7. **优化与评估**: 测试和改进系统性能

## 7. 多智能体工作流设计

### 7.1 工作流组件

- **专家智能体**: 具有特定领域知识的智能体
- **工具集成**: 与外部系统和数据源的连接
- **协作机制**: 智能体之间的互动规则
- **决策流程**: 如何做出和执行决策
- **监控系统**: 追踪工作流执行情况

### 7.2 常见模式

1. **研究-规划-执行**: 先研究问题，制定计划，再执行
2. **分解-征询-整合**: 将问题分解，征求专家意见，整合结果
3. **反馈循环**: 执行-评估-调整的迭代过程
4. **层级决策**: 不同层级的智能体负责不同抽象级别的决策

### 7.3 实现案例

多智能体系统可以包含如下角色：
- **研究员智能体**: 收集和分析信息
- **规划师智能体**: 制定执行计划
- **编码员智能体**: 编写和调试代码
- **测试员智能体**: 验证解决方案
- **协调器**: 管理整个工作流程

## 8. 最佳实践与优化

### 8.1 性能优化

- **批量处理**: 使用批处理减少 API 调用
- **缓存机制**: 缓存常用查询和结果
- **并行执行**: 并行化独立任务
- **模型选择**: 针对不同任务使用适当的模型

### 8.2 提示工程

- **任务分解**: 将复杂任务分解为简单步骤
- **上下文增强**: 提供充分的背景信息
- **指令明确性**: 给出清晰的指令
- **示例驱动**: 使用少样本学习

### 8.3 错误处理

- **异常捕获**: 处理可能的错误情况
- **重试机制**: 为不稳定操作实现重试
- **回退策略**: 当首选方法失败时的替代方案
- **用户反馈**: 适当时寻求用户输入

## 9. 资源与参考

- **官方文档**: https://python.langchain.com/docs/introduction/
- **GitHub 仓库**: https://github.com/langchain-ai/langchain
- **示例集**: https://python.langchain.com/docs/use_cases/
"""

# Write the comprehensive guide
editor.write("langchain_tutorial/documentation/langchain_comprehensive_guide.md", summary_content)

# Create a practical code examples library document
code_examples = """# LangChain 实用代码示例集

## 1. 基础组件示例

### 1.1 语言模型和提示模板


SyntaxError: incomplete input (387482405.py, line 233)

In [8]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os
import asyncio

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for the code examples document
await terminal.run("mkdir -p langchain_tutorial/documentation")

# Create the comprehensive code examples document focusing on agents
code_examples_md = """# LangChain 实用代码示例集

## 1. 单智能体实现模式

### 1.1 基础 ReAct 智能体

ReAct (Reasoning + Acting) 智能体结合了推理和行动的能力，是最常用的智能体类型。


SyntaxError: incomplete input (1008979211.py, line 14)

In [9]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for the code examples document
await terminal.run("mkdir -p langchain_tutorial/documentation")

# Create the comprehensive code examples document focusing on agents
code_examples_md = """# LangChain 实用代码示例集

## 1. 单智能体实现模式

### 1.1 基础 ReAct 智能体

ReAct (Reasoning + Acting) 智能体结合了推理和行动的能力，是最常用的智能体类型。


SyntaxError: incomplete input (4277677237.py, line 13)

In [10]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for the code examples document
await terminal.run("mkdir -p langchain_tutorial/documentation")

# Create the comprehensive code examples document focusing on agents
code_examples_md = """# LangChain 实用代码示例集

## 1. 单智能体实现模式

### 1.1 基础 ReAct 智能体

ReAct (Reasoning + Acting) 智能体结合了推理和行动的能力，是最常用的智能体类型。


SyntaxError: incomplete input (4277677237.py, line 13)

In [11]:
import asyncio
from bs4 import BeautifulSoup
from metagpt.tools.libs.web_scraping import view_page_element_to_scrape
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create dedicated directories for agent documentation
await terminal.run("mkdir -p langchain_tutorial/docs/single_agent")
await terminal.run("mkdir -p langchain_tutorial/docs/multi_agent")

# Target URLs focused on agent orchestration
single_agent_urls = [
    ("agent_executor", "https://python.langchain.com/docs/modules/agents/executor/"),
    ("react_agent", "https://python.langchain.com/docs/modules/agents/agent_types/react/"),
    ("openai_functions", "https://python.langchain.com/docs/modules/agents/agent_types/openai_functions/"),
    ("structured_chat", "https://python.langchain.com/docs/modules/agents/agent_types/structured_chat/"),
    ("plan_execute", "https://python.langchain.com/docs/modules/agents/agent_types/plan_and_execute/")
]

multi_agent_urls = [
    ("agent_supervision", "https://python.langchain.com/docs/modules/agents/agent_supervision/"),
    ("team_supervisor", "https://python.langchain.com/docs/modules/agents/agent_supervision/team_supervisor/"),
    ("manager_worker", "https://python.langchain.com/docs/modules/agents/agent_supervision/manager_worker/"), 
    ("plan_executor", "https://python.langchain.com/docs/modules/agents/agent_supervision/planexecutor/"),
    ("crewai_integration", "https://python.langchain.com/docs/integrations/platforms/crewai/")
]

# Function to extract content with better handling of code examples and structure
async def extract_doc_content(url, doc_type):
    print(f"Fetching {doc_type} documentation from: {url}")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"Extract all information about {doc_type} implementation, including examples and explanations",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    content = ""
    if main_content:
        # Extract title
        title = main_content.find('h1')
        if title:
            content += f"# {title.get_text(strip=True)}\n\n"
        
        # Process all elements sequentially to maintain structure and code examples
        for element in main_content.find_all(['h2', 'h3', 'h4', 'p', 'pre', 'ul', 'ol', 'li', 'div']):
            if element.name in ['h2', 'h3', 'h4']:
                level = int(element.name[1])
                content += f"{'#' * level} {element.get_text(strip=True)}\n\n"
            elif element.name == 'p':
                content += f"{element.get_text(strip=True)}\n\n"
            elif element.name == 'pre':
                # Check if it contains code
                code = element.find('code')
                if code:
                    language = "python" if "python" in code.get('class', [''])[0] else ""
                    content += f"```{language}\n{code.get_text()}\n```\n\n"
                else:
                    content += f"```\n{element.get_text()}\n```\n\n"
            elif element.name in ['ul', 'ol']:
                for li in element.find_all('li', recursive=False):
                    content += f"- {li.get_text(strip=True)}\n"
                content += "\n"
    
    return content

# Process single agent documentation
for agent_type, url in single_agent_urls:
    content = await extract_doc_content(url, f"single agent ({agent_type})")
    with open(f"langchain_tutorial/docs/single_agent/{agent_type}.md", 'w', encoding='utf-8') as f:
        f.write(content)
        f.write(f"\n\nSource: {url}\n")

# Process multi-agent documentation
for agent_type, url in multi_agent_urls:
    content = await extract_doc_content(url, f"multi-agent orchestration ({agent_type})")
    with open(f"langchain_tutorial/docs/multi_agent/{agent_type}.md", 'w', encoding='utf-8') as f:
        f.write(content)
        f.write(f"\n\nSource: {url}\n")

# Create a comprehensive guide document specifically about agent orchestration
comprehensive_guide = """# LangChain 智能体编排指南

## 1. 单一智能体架构

单一智能体架构通过将语言模型与工具和执行逻辑结合，使模型能够通过工具与外部环境交互，以解决复杂任务。

### 1.1 智能体执行器 (Agent Executor)

智能体执行器是实现智能体架构的核心组件，负责：
- 管理智能体与工具之间的交互
- 解析智能体输出
- 执行工具调用
- 将结果返回给智能体
- 循环直到任务完成

### 1.2 常见智能体类型

LangChain提供多种智能体类型，根据不同场景选择适当的智能体类型：

1. **ReAct 智能体**
   - 结合推理和行动的智能体
   - 适用于需要复杂推理的任务
   - 支持链式思考过程

2. **OpenAI 函数智能体**
   - 基于 OpenAI 函数调用能力
   - 更易于解析工具调用
   - 更高的可靠性和更低的幻觉

3. **结构化聊天智能体**
   - 处理复杂的人类指令
   - 增强了聊天功能
   - 可以使用多种工具

4. **计划与执行智能体**
   - 先制定计划再执行
   - 将任务分解为步骤
   - 适用于复杂任务的分解

## 2. 多智能体编排

多智能体编排通过结合多个智能体的专业知识和能力，以协作方式解决复杂问题。

### 2.1 多智能体架构模式

1. **团队监督模式 (Team Supervisor)**
   - 一个监督者智能体协调多个专家智能体
   - 专家智能体各自解决子问题
   - 监督者合并和整理结果

2. **经理-员工模式 (Manager-Worker)**
   - 经理智能体分配任务
   - 员工智能体执行具体工作
   - 层级结构的工作分配

3. **计划执行模式 (Plan Executor)**
   - 一个智能体制定详细计划
   - 另一个智能体执行计划中的步骤
   - 适用于需要规划的复杂任务

### 2.2 智能体监督 (Agent Supervision)

智能体监督是实现多智能体协作的核心机制，包括：
- 任务分配与跟踪
- 结果整合
- 冲突解决
- 错误处理和恢复

### 2.3 工具共享与通信

多智能体系统中工具共享和通信的关键考虑因素：
- 工具访问权限管理
- 智能体间通信协议
- 中间结果共享机制
- 知识库与上下文管理

## 3. 智能体与搜索集成

将搜索和检索功能集成到智能体架构中，增强智能体访问和处理信息的能力。

### 3.1 智能体与RAG结合

检索增强生成（RAG）与智能体结合可以：
- 让智能体主动检索相关信息
- 增加回答的准确性和相关性
- 减少幻觉和错误信息

### 3.2 实现模式

1. **检索作为工具**
   - 将检索器封装为智能体可用的工具
   - 智能体主动决定何时检索信息

2. **检索增强上下文**
   - 在智能体处理前先进行检索
   - 将检索结果作为上下文提供给智能体

3. **混合方法**
   - 结合以上两种方法
   - 提供基础上下文并允许主动检索

## 4. 多智能体工作流设计

### 4.1 工作流设计原则

设计多智能体工作流时应考虑：
- 任务分解粒度
- 智能体角色和专长定义
- 通信频率和方式
- 决策机制
- 错误处理策略

### 4.2 常见工作流模式

1. **研究-规划-执行模式**
   - 研究智能体收集信息
   - 规划智能体制定计划
   - 执行智能体实施计划

2. **分析-创建-评估模式**
   - 分析智能体理解问题
   - 创建智能体生成解决方案
   - 评估智能体验证解决方案

3. **协作迭代模式**
   - 多个智能体并行工作
   - 定期同步和更新
   - 迭代改进解决方案

### 4.3 外部工具集成

多智能体系统可以集成的外部工具类型：
- 代码执行环境
- 数据分析工具
- 网络搜索工具
- 文档处理工具
- API 调用工具
"""

# Write the comprehensive guide
editor.write("langchain_tutorial/docs/agent_orchestration_guide.md", comprehensive_guide)

# Check the files created
files = await terminal.run("find langchain_tutorial/docs -name '*.md' | sort")
print("\nCreated documentation files:")
print(files)

# Print summary
print("\n====== Documentation Collection Complete ======")
print("Collected information about:")
print("1. Single agent implementations - various agent types")
print("2. Multi-agent orchestration - supervision patterns")
print("3. Created a comprehensive guide on agent orchestration")

Fetching single agent (agent_executor) documentation from: https://python.langchain.com/docs/modules/agents/executor/
Fetching single agent (react_agent) documentation from: https://python.langchain.com/docs/modules/agents/agent_types/react/
Fetching single agent (openai_functions) documentation from: https://python.langchain.com/docs/modules/agents/agent_types/openai_functions/
Fetching single agent (structured_chat) documentation from: https://python.langchain.com/docs/modules/agents/agent_types/structured_chat/
Fetching single agent (plan_execute) documentation from: https://python.langchain.com/docs/modules/agents/agent_types/plan_and_execute/
Fetching multi-agent orchestration (agent_supervision) documentation from: https://python.langchain.com/docs/modules/agents/agent_supervision/
Fetching multi-agent orchestration (team_supervisor) documentation from: https://python.langchain.com/docs/modules/agents/agent_supervision/team_supervisor/
Fetching multi-agent orchestration (manager_

2025-04-06 21:02:05.222 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:08.434 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:11.678 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:15.022 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:18.408 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:21.477 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:24.816 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:27.609 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06 21:02:30.937 | INFO     | metagpt.configs.supabase_config:initialize:36 - Supabase is not enabled.
2025-04-06

In [12]:
I'll create practical code examples that demonstrate both single agent and multi-agent orchestration patterns in LangChain. Let me focus on creating clear examples that showcase different orchestration patterns.

```python
import asyncio
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Create directory for code examples focusing on agent orchestration
terminal = Terminal()
await terminal.run("mkdir -p langchain_tutorial/code_examples/agent_examples")

# First, let's create a comprehensive single agent example that demonstrates different agent types
single_agent_example = """# LangChain 单智能体模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.tools.python.tool import PythonREPLTool
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper, WikipediaAPIWrapper
from langchain_experimental.tools import PythonAstREPLTool

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key" 
# os.environ["SERPAPI_API_KEY"] = "your-serpapi-key"  # 如果使用搜索功能

# 1. ReAct 智能体示例 - 结合推理和行动的智能体
def create_react_agent():
    """创建基本的ReAct智能体"""
    # 定义工具集
    search = SerpAPIWrapper()
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="搜索",
            func=search.run,
            description="用于在网络上搜索最新信息的工具"
        ),
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="用于查询维基百科文章的工具"
        ),
        Tool(
            name="Python解释器",
            func=python_repl.run,
            description="用于执行Python代码的工具，可以进行计算或数据分析"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
    
    # 创建记忆组件
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # 初始化ReAct智能体
    agent = initialize_agent(
        tools, 
        llm, 
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        memory=memory,
        handle_parsing_errors=True
    )
    
    return agent

# 2. OpenAI函数智能体示例 - 专为函数调用设计的智能体
def create_openai_functions_agent():
    """创建基于OpenAI函数调用的智能体"""
    # 定义工具集
    python_repl = PythonAstREPLTool()
    wikipedia = WikipediaAPIWrapper()
    
    tools = [
        Tool(
            name="Python执行器",
            func=python_repl.run,
            description="执行Python代码的安全工具，适合进行计算、数据处理"
        ),
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="搜索维基百科文章的工具，适合查询事实性信息"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
    
    # 初始化OpenAI函数智能体
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True
    )
    
    return agent

# 3. 计划与执行智能体 - 先规划后执行的智能体
from langchain.agents import create_plan_and_execute_agent
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import create_async_playwright_browser

def create_planner_agent():
    """创建计划与执行智能体"""
    # 创建LLM
    llm = ChatOpenAI(temperature=0, model="gpt-4")
    
    # 定义简单工具集
    wikipedia = WikipediaAPIWrapper()
    tools = [
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="搜索维基百科文章的工具"
        ),
        PythonREPLTool()
    ]
    
    # 创建计划与执行智能体
    agent = create_plan_and_execute_agent(
        llm=llm,
        tools=tools,
        verbose=True
    )
    
    return agent

# 4. 自定义智能体工具示例
class WeatherTool(BaseTool):
    name = "天气查询"
    description = "查询指定城市的天气情况"
    
    def _run(self, city: str) -> str:
        # 模拟天气API调用
        return f"{city}的天气: 晴朗, 25°C, 湿度50%"
        
    async def _arun(self, city: str) -> str:
        return self._run(city)

class CalculatorTool(BaseTool):
    name = "计算器"
    description = "进行数学计算，输入应为数学表达式"
    
    def _run(self, expression: str) -> str:
        try:
            result = eval(expression)
            return f"计算结果: {result}"
        except Exception as e:
            return f"计算错误: {str(e)}"
            
    async def _arun(self, expression: str) -> str:
        return self._run(expression)

def create_custom_tool_agent():
    """创建带有自定义工具的智能体"""
    tools = [
        WeatherTool(),
        CalculatorTool(),
        PythonREPLTool()
    ]
    
    llm = ChatOpenAI(temperature=0)
    
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    return agent

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 单智能体模式示例 ===")
    
    # 选择要演示的智能体类型 
    agent_type = "react"  # 可选: "react", "openai_functions", "planner", "custom"
    
    if agent_type == "react":
        agent = create_react_agent()
        response = agent.run(
            "谁是阿尔伯特·爱因斯坦? 他出生于哪一年? 计算从他出生到现在过了多少年。"
        )
    elif agent_type == "openai_functions":
        agent = create_openai_functions_agent() 
        response = agent.run(
            "计算 2345 + 5678 的结果，并解释这两个数字的数学特性。"
        )
    elif agent_type == "planner":
        agent = create_planner_agent()
        response = agent.run(
            "查找关于人工智能的信息，然后编写一个Python函数来总结这些信息的关键点。"
        )
    elif agent_type == "custom":
        agent = create_custom_tool_agent()
        response = agent.run(
            "北京的天气如何？然后计算25乘以4的结果。"
        )
        
    print(f"\\n最终回答: {response}")
"""

# Now, let's create a multi-agent orchestration example showing different patterns
multi_agent_example = """# LangChain 多智能体编排模式示例

import os
from typing import List, Dict, Any, Optional
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.python.tool import PythonREPLTool
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 1. 团队监督者模式 (Team Supervisor)
class TeamSupervisorSystem:
    """
    团队监督者模式 - 一个监督者智能体协调多个专家智能体
    """
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        
        # 创建专家智能体
        self.researcher = self._create_researcher_agent()
        self.coder = self._create_coder_agent()
        self.critic = self._create_critic_agent()
        
        # 创建监督者
        self.supervisor = self._create_supervisor_agent()
    
    def _create_researcher_agent(self):
        """创建研究员智能体"""
        wikipedia = WikipediaAPIWrapper()
        tools = [
            Tool(
                name="维基百科",
                func=wikipedia.run,
                description="用于查询事实性信息的维基百科工具"
            )
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_coder_agent(self):
        """创建编码员智能体"""
        tools = [PythonREPLTool()]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_critic_agent(self):
        """创建评论员智能体"""
        prompt = PromptTemplate(
            template="你是一位严格的评论家。请评估以下解决方案的质量、正确性和效率：\\n{solution}\\n请提供详细的改进建议。",
            input_variables=["solution"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt, verbose=True)
    
    def _create_supervisor_agent(self):
        """创建监督者智能体"""
        prompt = PromptTemplate(
            template="你是一个团队领导，负责协调其他专家完成任务。你需要将复杂问题分解成子任务，分配给适当的专家，并整合他们的工作。\\n\\n任务描述: {task}\\n\\n请制定一个计划，说明如何将这个任务分解并分配给团队成员（研究员、编码员、评论员）。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行团队监督者系统"""
        print("\\n===== 团队监督者模式 =====")
        print(f"任务: {task}")
        
        # 1. 监督者分配任务
        print("\\n1. 监督者制定计划")
        plan = self.supervisor.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 研究员收集信息
        print("\\n2. 研究员收集信息")
        research_query = f"请查询关于以下内容的事实性信息: {task}"
        research_result = self.researcher.run(research_query)
        print(f"\\n研究结果:\\n{research_result}")
        
        # 3. 编码员根据研究结果实现解决方案
        print("\\n3. 编码员实现解决方案")
        coding_task = f"根据以下研究结果，编写Python代码解决问题: {task}\\n\\n研究信息: {research_result}"
        code_solution = self.coder.run(coding_task)
        print(f"\\n代码解决方案:\\n{code_solution}")
        
        # 4. 评论员评估解决方案
        print("\\n4. 评论员评估解决方案")
        critique = self.critic.run(solution=code_solution)
        print(f"\\n评估意见:\\n{critique}")
        
        # 5. 监督者整合结果
        print("\\n5. 监督者整合最终结果")
        final_prompt = f"请整合以下信息，提供最终的解决方案和总结：\\n\\n原始任务: {task}\\n\\n研究结果: {research_result}\\n\\n代码解决方案: {code_solution}\\n\\n评估意见: {critique}"
        final_solution = self.llm.predict(final_prompt)
        print(f"\\n最终解决方案:\\n{final_solution}")
        
        return {
            "task": task,
            "plan": plan,
            "research": research_result,
            "code": code_solution,
            "critique": critique,
            "final_solution": final_solution
        }

# 2. 经理-员工模式 (Manager-Worker)
class ManagerWorkerSystem:
    """
    经理-员工模式 - 经理智能体分配任务给员工智能体
    """
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建经理智能体
        self.manager = self._create_manager_agent()
        
        # 创建员工智能体
        self.workers = {
            "researcher": self._create_worker("researcher", "执行研究任务，查找和分析信息"),
            "analyst": self._create_worker("analyst", "分析数据并提供洞察"),
            "coder": self._create_worker("coder", "编写和调试代码"),
            "writer": self._create_worker("writer", "撰写报告和文档")
        }
    
    def _create_manager_agent(self):
        """创建经理智能体"""
        prompt = PromptTemplate(
            template="你是一位项目经理，负责将大型任务分解为子任务并分配给适当的员工。\\n\\n可用员工: researcher(研究员), analyst(分析师), coder(程序员), writer(作家)\\n\\n任务: {task}\\n\\n请将此任务分解成1-4个子任务，每个子任务分配给一个员工，并提供清晰的指示。格式为JSON:\\n{{\\n  \"assignments\": [\\n    {{\\n      \"worker\": \"worker_name\",\\n      \"task\": \"task_description\"\\n    }}\\n  ]\\n}}",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def _create_worker(self, role: str, description: str):
        """创建员工智能体"""
        tools = []
        
        # 根据角色分配不同的工具
        if role == "researcher":
            tools.append(
                Tool(
                    name="维基百科",
                    func=WikipediaAPIWrapper().run,
                    description="查询事实性信息的工具"
                )
            )
        elif role == "coder":
            tools.append(PythonREPLTool())
        
        # 创建员工智能体
        worker_prompt = f"你是一名{role}({description})。你需要完成分配给你的任务并提供详细的结果。"
        
        if tools:
            return initialize_agent(
                tools,
                self.llm,
                agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                verbose=True,
                agent_kwargs={"system_message": worker_prompt}
            )
        else:
            # 对于不需要特殊工具的角色，使用简单链
            prompt = PromptTemplate(
                template=f"{worker_prompt}\\n\\n任务: {{task}}\\n\\n请提供你的解决方案:",
                input_variables=["task"]
            )
            return LLMChain(llm=self.llm, prompt=prompt)
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行经理-员工系统"""
        print("\\n===== 经理-员工模式 =====")
        print(f"任务: {task}")
        
        # 1. 经理分配任务
        print("\\n1. 经理分配任务")
        assignments_text = self.manager.run(task=task)
        print(f"任务分配:\\n{assignments_text}")
        
        # 简单解析JSON (实际应用中应使用更健壮的方法)
        import json
        try:
            assignments = json.loads(assignments_text)["assignments"]
        except:
            # 如果解析失败，使用备用方案
            print("无法解析任务分配，使用备用方案")
            assignments = [
                {"worker": "researcher", "task": f"研究关于: {task}"},
                {"worker": "coder", "task": f"编写代码解决: {task}"},
                {"worker": "writer", "task": f"撰写关于{task}的报告"}
            ]
        
        # 2. 员工执行任务
        results = {}
        for idx, assignment in enumerate(assignments):
            worker_name = assignment["worker"]
            worker_task = assignment["task"]
            
            if worker_name in self.workers:
                print(f"\\n{idx+2}. {worker_name}执行任务: {worker_task}")
                worker = self.workers[worker_name]
                result = worker.run(task=worker_task)
                results[worker_name] = result
                print(f"\\n{worker_name}的结果:\\n{result}")
            else:
                print(f"未找到{worker_name}角色的员工")
        
        # 3. 经理整合结果
        print("\\n最后. 经理整合结果")
        summary_prompt = f"以下是各员工完成的工作成果，请整合这些结果并提供最终的总结和解决方案：\\n\\n原始任务: {task}\\n\\n"
        
        for worker_name, result in results.items():
            summary_prompt += f"{worker_name}的工作成果: {result}\\n\\n"
        
        final_summary = self.llm.predict(summary_prompt)
        print(f"\\n最终总结:\\n{final_summary}")
        
        return {
            "task": task,
            "assignments": assignments,
            "results": results,
            "summary": final_summary
        }

# 3. 计划执行模式 (Plan Executor)
class PlanExecutorSystem:
    """
    计划执行模式 - 一个智能体制定计划，另一个智能体执行计划
    """
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建规划者智能体
        self.planner = self._create_planner()
        
        # 创建执行者智能体
        self.executor = self._create_executor()
    
    def _create_planner(self):
        """创建规划者智能体"""
        prompt = PromptTemplate(
            template="你是一位专业的规划者。请为以下任务制定一个详细的、分步骤的计划：\\n\\n任务: {task}\\n\\n请提供一个清晰的、有序的计划，每个步骤应该具体且可执行。确保计划涵盖任务的所有方面，并考虑可能的障碍和解决方案。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def _create_executor(self):
        """创建执行者智能体"""
        tools = [
            Tool(
                name="维基百科",
                func=WikipediaAPIWrapper().run,
                description="用于查询信息的工具"
            ),
            PythonREPLTool()
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行计划执行系统"""
        print("\\n===== 计划执行模式 =====")
        print(f"任务: {task}")
        
        # 1. 制定计划
        print("\\n1. 规划者制定计划")
        plan = self.planner.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 执行计划
        print("\\n2. 执行者执行计划")
        execution_task = f"根据以下计划执行任务：\\n\\n任务: {task}\\n\\n计划:\\n{plan}\\n\\n执行每个步骤并报告结果。"
        execution_result = self.executor.run(execution_task)
        print(f"\\n执行结果:\\n{execution_result}")
        
        # 3. 总结结果
        summary = f"任务: {task}\\n\\n计划: {plan}\\n\\n执行结果: {execution_result}"
        
        return {
            "task": task,
            "plan": plan,
            "execution_result": execution_result,
            "summary": summary
        }

# 4. RAG与智能体结合示例
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

class RAGAgentSystem:
    """
    将RAG（检索增强生成）与智能体结合的系统
    """
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.retriever = None
        self.rag_chain = None
        self.agent = None
        
    def setup_knowledge_base(self, texts: List[str]):
        """设置知识库"""
        print("\\n设置RAG知识库...")
        
        # 创建临时文本文件
        import tempfile
        import os
        
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"document_{i}.txt")
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(text)
            file_paths.append(file_path)
        
        # 加载文档
        documents = []
        for file_path in file_paths:
            loader = TextLoader(file_path)
            documents.extend(loader.load())
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        chunks = text_splitter.split_documents(documents)
        
        # 创建检索器
        embeddings = OpenAIEmbeddings()
        self.vectordb = FAISS.from_documents(chunks, embeddings)
        self.retriever = self.vectordb.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # 创建RAG链
        self._create_rag_chain()
        
        # 创建智能体工具
        self._create_agent_with_rag_tool()
        
        print("知识库设置完成!")
        
    def _create_rag_chain(self):
        """创建RAG链"""
        # 创建RAG提示模板
        template = """使用以下检索的上下文来回答问题。如果你无法从上下文中找到答案，请说明你不知道。
        
上下文:
{context}

问题: {question}
"""
        prompt = PromptTemplate.from_template(template)
        
        # 定义文档格式化函数
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建RAG链
        self.rag_chain = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | self.llm
        )
    
    def _create_agent_with_rag_tool(self):
        """创建带有RAG工具的智能体"""
        # 定义RAG工具
        def rag_tool(query):
            """使用RAG系统查询知识库"""
            result = self.rag_chain.invoke(query)
            return result.content if hasattr(result, 'content') else str(result)
        
        tools = [
            Tool(
                name="知识库查询",
                func=rag_tool,
                description="当你需要查询特定领域知识时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        self.agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True
        )
    
    def run(self, query: str) -> str:
        """运行RAG智能体系统"""
        print("\\n===== RAG智能体系统 =====")
        print(f"查询: {query}")
        
        if not self.agent:
            raise ValueError("请先通过setup_knowledge_base方法设置知识库")
        
        result = self.agent.run(query)
        return result

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 多智能体编排模式示例 ===")
    
    # 选择要演示的编排模式
    orchestration_type = "team_supervisor"  # 可选: "team_supervisor", "manager_worker", "plan_executor", "rag_agent"
    
    if orchestration_type == "team_supervisor":
        system = TeamSupervisorSystem()
        result = system.run("创建一个简单的网站访问计数器应用")
        
    elif orchestration_type == "manager_worker":
        system = ManagerWorkerSystem()
        result = system.run("分析2022年全球气候变化的主要趋势并创建数据可视化")
        
    elif orchestration_type == "plan_executor":
        system = PlanExecutorSystem()
        result = system.run("研究机器学习中的过拟合问题并提供3种解决方法")
        
    elif orchestration_type == "rag_agent":
        system = RAGAgentSystem()
        
        # 设置示例知识库
        knowledge = [
            \"\"\"
            LangChain是一个用于开发由语言模型驱动的应用程序的框架。它的主要组件包括:
            
            1. 模型I/O: 处理与语言模型的交互，包括提示模板和输出解析。
            2. 检索: 从外部数据源检索数据。
            3. 记忆: 在链的运行之间持久化状态。
            4. 链: 将多个组件组合成一个应用程序。
            5. 智能体: 允许LLM根据用户输入选择执行操作。
            
            LangChain表达式语言(LCEL)是一种声明式语言，用于组合这些组件。它使用管道操作符(|)连接组件，支持同步和异步操作，并提供错误处理和批处理功能。
            \"\"\",
            \"\"\"
            多智能体系统是一种由多个智能体组成的系统，这些智能体相互协作以解决复杂问题。在LangChain中，多智能体编排可以通过以下模式实现:
            
            1. 团队监督模式: 一个监督者智能体协调多个专家智能体的工作。
            2. 经理-员工模式: 一个经理智能体将任务分配给多个员工智能体。
            3. 计划执行模式: 一个规划者智能体制定计划，执行者智能体执行计划。
            
            这些模式可以根据任务的复杂性和需求进行选择和组合。
            \"\"\",
            \"\"\"
            检索增强生成(RAG)是将大型语言模型与外部知识库结合的技术。它的工作流程包括:
            
            1. 索引: 将文档分割成块并创建向量索引。
            2. 检索: 根据查询检索相关文档片段。
            3. 生成: 使用检索到的上下文增强LLM的回答。
            
            RAG可以与智能体结

SyntaxError: unterminated string literal (detected at line 1) (1350562727.py, line 1)

In [13]:
[reflection on previous impl]
In the previous implementation, I tried to create comprehensive code examples for both single agent and multi-agent orchestration patterns in LangChain. However, the code ended with a syntax error: "unterminated string literal". 

Looking closely at the error, I notice that I used triple quotes (`"""`) for the multi-line strings within the knowledge list but didn't properly close the RAG knowledge example. The last text block was cut off at "RAG可以与智能体结" without closing the triple quotes, which caused the syntax error.

Additionally, there appears to be an issue with how I'm escaping special characters in the embedded strings. In Python, when using triple-quoted strings inside a triple-quoted string, we need to ensure proper escaping to avoid syntax errors.

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directories for code examples
async def create_agent_examples():
    # Create directories
    await terminal.run("mkdir -p langchain_tutorial/code_examples/agent_examples")
    
    # Create single agent examples file
    single_agent_code = '''# LangChain 单智能体模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.tools.python.tool import PythonREPLTool
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key" 

# 1. ReAct 智能体示例 - 结合推理和行动的智能体
def create_react_agent():
    """创建基本的ReAct智能体"""
    # 定义工具集
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="用于查询维基百科文章的工具"
        ),
        Tool(
            name="Python解释器",
            func=python_repl.run,
            description="用于执行Python代码的工具，可以进行计算或数据分析"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0)
    
    # 创建记忆组件
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # 初始化ReAct智能体
    agent = initialize_agent(
        tools, 
        llm, 
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        memory=memory,
        handle_parsing_errors=True
    )
    
    return agent

# 2. OpenAI函数智能体示例 - 专为函数调用设计的智能体
def create_openai_functions_agent():
    """创建基于OpenAI函数调用的智能体"""
    # 定义工具集
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="Python执行器",
            func=python_repl.run,
            description="执行Python代码的工具，适合进行计算、数据处理"
        ),
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="搜索维基百科文章的工具，适合查询事实性信息"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0)
    
    # 初始化OpenAI函数智能体
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True
    )
    
    return agent

# 3. 自定义智能体工具示例
class WeatherTool(BaseTool):
    name = "天气查询"
    description = "查询指定城市的天气情况"
    
    def _run(self, city: str) -> str:
        # 模拟天气API调用
        return f"{city}的天气: 晴朗, 25°C, 湿度50%"
        
    async def _arun(self, city: str) -> str:
        return self._run(city)

class CalculatorTool(BaseTool):
    name = "计算器"
    description = "进行数学计算，输入应为数学表达式"
    
    def _run(self, expression: str) -> str:
        try:
            result = eval(expression)
            return f"计算结果: {result}"
        except Exception as e:
            return f"计算错误: {str(e)}"
            
    async def _arun(self, expression: str) -> str:
        return self._run(expression)

def create_custom_tool_agent():
    """创建带有自定义工具的智能体"""
    tools = [
        WeatherTool(),
        CalculatorTool(),
        PythonREPLTool()
    ]
    
    llm = ChatOpenAI(temperature=0)
    
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    return agent

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 单智能体模式示例 ===")
    
    # 选择要演示的智能体类型 
    agent_type = "react"  # 可选: "react", "openai_functions", "custom"
    
    if agent_type == "react":
        agent = create_react_agent()
        response = agent.run(
            "谁是阿尔伯特·爱因斯坦? 他出生于哪一年? 计算从他出生到现在过了多少年。"
        )
    elif agent_type == "openai_functions":
        agent = create_openai_functions_agent() 
        response = agent.run(
            "计算 2345 + 5678 的结果，并解释这两个数字的数学特性。"
        )
    elif agent_type == "custom":
        agent = create_custom_tool_agent()
        response = agent.run(
            "北京的天气如何？然后计算25乘以4的结果。"
        )
        
    print(f"\\n最终回答: {response}")
'''

    # Create multi-agent examples file
    multi_agent_code = '''# LangChain 多智能体编排模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.python.tool import PythonREPLTool

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 1. 团队监督者模式 (Team Supervisor)
class TeamSupervisorSystem:
    """团队监督者模式 - 一个监督者智能体协调多个专家智能体"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建专家智能体
        self.researcher = self._create_researcher_agent()
        self.coder = self._create_coder_agent()
        self.critic = self._create_critic_agent()
        
        # 创建监督者
        self.supervisor = self._create_supervisor_agent()
    
    def _create_researcher_agent(self):
        """创建研究员智能体"""
        wikipedia = WikipediaAPIWrapper()
        tools = [
            Tool(
                name="维基百科",
                func=wikipedia.run,
                description="用于查询事实性信息的维基百科工具"
            )
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_coder_agent(self):
        """创建编码员智能体"""
        tools = [PythonREPLTool()]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_critic_agent(self):
        """创建评论员智能体"""
        prompt = PromptTemplate(
            template="你是一位严格的评论家。请评估以下解决方案的质量、正确性和效率：\\n{solution}\\n请提供详细的改进建议。",
            input_variables=["solution"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt, verbose=True)
    
    def _create_supervisor_agent(self):
        """创建监督者智能体"""
        prompt = PromptTemplate(
            template="你是一个团队领导，负责协调其他专家完成任务。你需要将复杂问题分解成子任务，分配给适当的专家，并整合他们的工作。\\n\\n任务描述: {task}\\n\\n请制定一个计划，说明如何将这个任务分解并分配给团队成员（研究员、编码员、评论员）。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行团队监督者系统"""
        print("\\n===== 团队监督者模式 =====")
        print(f"任务: {task}")
        
        # 1. 监督者分配任务
        print("\\n1. 监督者制定计划")
        plan = self.supervisor.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 研究员收集信息
        print("\\n2. 研究员收集信息")
        research_query = f"请查询关于以下内容的事实性信息: {task}"
        research_result = self.researcher.run(research_query)
        print(f"\\n研究结果:\\n{research_result}")
        
        # 3. 编码员根据研究结果实现解决方案
        print("\\n3. 编码员实现解决方案")
        coding_task = f"根据以下研究结果，编写Python代码解决问题: {task}\\n\\n研究信息: {research_result}"
        code_solution = self.coder.run(coding_task)
        print(f"\\n代码解决方案:\\n{code_solution}")
        
        # 4. 评论员评估解决方案
        print("\\n4. 评论员评估解决方案")
        critique = self.critic.run(solution=code_solution)
        print(f"\\n评估意见:\\n{critique}")
        
        # 5. 监督者整合结果
        print("\\n5. 监督者整合最终结果")
        final_prompt = f"请整合以下信息，提供最终的解决方案和总结：\\n\\n原始任务: {task}\\n\\n研究结果: {research_result}\\n\\n代码解决方案: {code_solution}\\n\\n评估意见: {critique}"
        final_solution = self.llm.predict(final_prompt)
        print(f"\\n最终解决方案:\\n{final_solution}")
        
        return {
            "task": task,
            "plan": plan,
            "research": research_result,
            "code": code_solution,
            "critique": critique,
            "final_solution": final_solution
        }

# 2. 计划执行模式 (Plan Executor)
class PlanExecutorSystem:
    """计划执行模式 - 一个智能体制定计划，另一个智能体执行计划"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建规划者智能体
        self.planner = self._create_planner()
        
        # 创建执行者智能体
        self.executor = self._create_executor()
    
    def _create_planner(self):
        """创建规划者智能体"""
        prompt = PromptTemplate(
            template="你是一位专业的规划者。请为以下任务制定一个详细的、分步骤的计划：\\n\\n任务: {task}\\n\\n请提供一个清晰的、有序的计划，每个步骤应该具体且可执行。确保计划涵盖任务的所有方面，并考虑可能的障碍和解决方案。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def _create_executor(self):
        """创建执行者智能体"""
        tools = [
            Tool(
                name="维基百科",
                func=WikipediaAPIWrapper().run,
                description="用于查询信息的工具"
            ),
            PythonREPLTool()
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行计划执行系统"""
        print("\\n===== 计划执行模式 =====")
        print(f"任务: {task}")
        
        # 1. 制定计划
        print("\\n1. 规划者制定计划")
        plan = self.planner.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 执行计划
        print("\\n2. 执行者执行计划")
        execution_task = f"根据以下计划执行任务：\\n\\n任务: {task}\\n\\n计划:\\n{plan}\\n\\n执行每个步骤并报告结果。"
        execution_result = self.executor.run(execution_task)
        print(f"\\n执行结果:\\n{execution_result}")
        
        return {
            "task": task,
            "plan": plan,
            "execution_result": execution_result
        }

# 3. RAG与智能体结合示例
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

class RAGAgentSystem:
    """将RAG（检索增强生成）与智能体结合的系统"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.retriever = None
        self.rag_chain = None
        self.agent = None
        
    def setup_knowledge_base(self, texts):
        """设置知识库"""
        print("\\n设置RAG知识库...")
        
        # 创建临时文本文件
        import tempfile
        import os
        
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"document_{i}.txt")
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(text)
            file_paths.append(file_path)
        
        # 加载文档
        documents = []
        for file_path in file_paths:
            loader = TextLoader(file_path)
            documents.extend(loader.load())
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        chunks = text_splitter.split_documents(documents)
        
        # 创建检索器
        embeddings = OpenAIEmbeddings()
        self.vectordb = FAISS.from_documents(chunks, embeddings)
        self.retriever = self.vectordb.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # 创建RAG链
        self._create_rag_chain()
        
        # 创建智能体工具
        self._create_agent_with_rag_tool()
        
        print("知识库设置完成!")
        
    def _create_rag_chain(self):
        """创建RAG链"""
        # 创建RAG提示模板
        template = """使用以下检索的上下文来回答问题。如果你无法从上下文中找到答案，请说明你不知道。
        
上下文:
{context}

问题: {question}
"""
        prompt = PromptTemplate.from_template(template)
        
        # 定义文档格式化函数
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建RAG链
        self.rag_chain = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | self.llm
        )
    
    def _create_agent_with_rag_tool(self):
        """创建带有RAG工具的智能体"""
        # 定义RAG工具
        def rag_tool(query):
            """使用RAG系统查询知识库"""
            result = self.rag_chain.invoke(query)
            return result.content if hasattr(result, 'content') else str(result)
        
        tools = [
            Tool(
                name="知识库查询",
                func=rag_tool,
                description="当你需要查询特定领域知识时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        self.agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True
        )
    
    def run(self, query: str) -> str:
        """运行RAG智能体系统"""
        print("\\n===== RAG智能体系统 =====")
        print(f"查询: {query}")
        
        if not self.agent:
            raise ValueError("请先通过setup_knowledge_base方法设置知识库")
        
        result = self.agent.run(query)
        return result

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 多智能体编排模式示例 ===")
    
    # 选择要演示的编排模式
    orchestration_type = "team_supervisor"  # 可选: "team_supervisor", "plan_executor", "rag_agent"
    
    if orchestration_type == "team_supervisor":
        system = TeamSupervisorSystem()
        result = system.run("创建一个简单的网站访问计数器应用")
        
    elif orchestration_type == "plan_executor":
        system = PlanExecutorSystem()
        result = system.run("研究机器学习中的过拟合问题并提供3种解决方法")
        
    elif orchestration_type == "rag_agent":
        system = RAGAgentSystem()
        
        # 设置示例知识库
        knowledge = [
            """
            LangChain是一个用于开发由语言模型驱动的应用程序的框架。它的主要组件包括:
            
            1. 模型I/O: 处理与语言模型的交互，包括提示模板和输出解析。
            2. 检索: 从外部数据源检索数据。
            3. 记忆: 在链的运行之间持久化状态。
            4. 链: 将多个组件组合成一个应用程序。
            5. 智能体: 允许LLM根据用户输入选择执行操作。
            
            LangChain表达式语言(LCEL)是一种声明式语言，用于组合这些组件。
            """,
            """
            多智能体系统是一种由多个智能体组成的系统，这些智能体相互协作以解决复杂问题。在LangChain中，多智能体编排可以通过以下模式实现:
            
            1. 团队监督模式: 一个监督者智能体协调多个专家智能体的工作。
            2. 经理-员工模式: 一个经理智能体将任务分配给多个员工智能体。
            3. 计划执行模式: 一个规划者智能体制定计划，执行者智能体执行计划。
            """
        ]
        
        system.setup_knowledge_base(knowledge)
        result = system.run("解释LangChain中的多智能体编排模式有哪些")
'''

    # Create a RAG+Agent integration example
    rag_agent_code = '''# LangChain RAG与智能体结合示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class RAGAgentExample:
    """
    检索增强生成(RAG)与智能体结合示例
    展示如何将RAG系统集成到智能体工作流中
    """
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vector_store = None
        self.retriever = None
    
    def load_documents(self, texts: List[str]):
        """加载文档到系统中"""
        print("加载文档...")
        
        # 创建临时文件
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        # 将文本写入临时文件
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个文档")
    
    def process_documents(self):
        """处理文档：分块和创建向量存储"""
        if not self.documents:
            raise ValueError("请先加载文档")
        
        print("处理文档：分块和向量化...")
        
        # 分块
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        chunks = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vector_store = FAISS.from_documents(chunks, embeddings)
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        
        print(f"文档处理完成，共创建 {len(chunks)} 个文本块")
        
        return self.retriever
    
    def create_standalone_rag_chain(self):
        """创建独立的RAG链"""
        if not self.retriever:
            raise ValueError("请先处理文档")
        
        # 创建RAG提示模板
        rag_prompt = PromptTemplate.from_template("""
        请使用以下检索的信息回答问题。如果检索的信息中没有答案，请说明你不知道。

        检索的信息:
        {context}

        问题: {question}
        """)
        
        # 定义文档格式化函数
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建RAG链
        rag_chain = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | rag_prompt
            | self.llm
        )
        
        return rag_chain
    
    def create_rag_tool(self):
        """将RAG系统封装为智能体工具"""
        rag_chain = self.create_standalone_rag_chain()
        
        def query_knowledge_base(query: str) -> str:
            """查询知识库获取信息"""
            result = rag_chain.invoke(query)
            return result.content if hasattr(result, "content") else str(result)
        
        # 创建RAG工具
        rag_tool = Tool(
            name="知识库查询",
            func=query_knowledge_base,
            description="当你需要查询特定领域知识时使用此工具"
        )
        
        return rag_tool
    
    def create_agent_with_rag(self):
        """创建集成RAG的智能体"""
        # 获取RAG工具
        rag_tool = self.create_rag_tool()
        
        # 添加其他工具
        python_tool = PythonREPLTool()
        
        # 创建智能体
        agent = initialize_agent(
            tools=[rag_tool, python_tool],
            llm=self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
        
        return agent

# 1. 直接RAG问答示例
def run_rag_qa_example():
    """运行直接RAG问答示例"""
    print("\\n===== 直接RAG问答示例 =====")
    
    # 创建RAG系统
    rag_system = RAGAgentExample()
    
    # 加载文档
    documents = [
        """
        LangChain是一个用于开发由语言模型驱动的应用程序的框架。它的核心组件包括:
        
        1. 模型I/O (Models): 与各种语言模型提供商的集成接口。
        2. 提示管理 (Prompts): 优化和管理提示模板。 
        3. 记忆系统 (Memory): 维护对话状态和历史。
        4. 索引和检索 (Indexes): 使用向量存储等技术组织和检索信息。
        5. 链 (Chains): 将多个组件串联成管道。
        6. 智能体 (Agents): 允许LLM决定使用哪些工具来完成任务。
        """,
        """
        LangChain表达式语言(LCEL)是一种声明式语言，用于组合LangChain的组件。它具有以下特性:
        
        1. 链式API: 使用管道操作符(|)连接组件。
        2. 并行执行: 可以同时执行多个操作并合并结果。
        3. 错误处理: 内置重试和异常处理机制。
        4. 批处理: 一次处理多个输入以提高效率。
        5. 流式传输: 支持逐步生成和处理响应。
        
        示例: chain = prompt | llm | output_parser
        """
    ]
    rag_system.load_documents(documents)
    rag_system.process_documents()
    
    # 创建RAG链
    rag_chain = rag_system.create_standalone_rag_chain()
    
    # 查询
    queries = [
        "LangChain的核心组件有哪些?",
        "什么是LCEL，它有什么特点?"
    ]
    
    for query in queries:
        print(f"\\n问题: {query}")
        response = rag_chain.invoke(query)
        print(f"回答: {response.content if hasattr(response, 'content') else response}")

# 2. 智能体与RAG集成示例
def run_agent_with_rag_example():
    """运行智能体与RAG集成示例"""
    print("\\n===== 智能体与RAG集成示例 =====")
    
    # 创建RAG系统
    rag_system = RAGAgentExample()
    
    # 加载文档
    documents = [
        """
        LangChain中的智能体系统允许语言模型使用工具来解决复杂问题。主要智能体类型包括:

SyntaxError: invalid character '，' (U+FF0C) (1670564751.py, line 304)

In [14]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directories for code examples
async def main():
    # Create directories
    await terminal.run("mkdir -p langchain_tutorial/code_examples/agent_examples")
    
    # Create single agent example file
    await create_single_agent_file()
    
    # Create multi-agent example file
    await create_multi_agent_file()
    
    # Create RAG agent example file
    await create_rag_agent_file()
    
    # List created files
    files = await terminal.run("find langchain_tutorial/code_examples/agent_examples -type f | sort")
    print(f"Created example files:\n{files}")
    
    print("\n========== Completed ==========")
    print("Created comprehensive examples for:")
    print("1. Single agent implementation patterns")
    print("2. Multi-agent orchestration patterns")
    print("3. RAG integration with agents")

async def create_single_agent_file():
    """Create the single agent examples file"""
    single_agent_content = """# LangChain 单智能体模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.tools.python.tool import PythonREPLTool
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key" 

# 1. ReAct 智能体示例 - 结合推理和行动的智能体
def create_react_agent():
    """创建基本的ReAct智能体"""
    # 定义工具集
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="用于查询维基百科文章的工具"
        ),
        Tool(
            name="Python解释器",
            func=python_repl.run,
            description="用于执行Python代码的工具，可以进行计算或数据分析"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0)
    
    # 创建记忆组件
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # 初始化ReAct智能体
    agent = initialize_agent(
        tools, 
        llm, 
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        memory=memory,
        handle_parsing_errors=True
    )
    
    return agent

# 2. OpenAI函数智能体示例 - 专为函数调用设计的智能体
def create_openai_functions_agent():
    """创建基于OpenAI函数调用的智能体"""
    # 定义工具集
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="Python执行器",
            func=python_repl.run,
            description="执行Python代码的工具，适合进行计算、数据处理"
        ),
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="搜索维基百科文章的工具，适合查询事实性信息"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0)
    
    # 初始化OpenAI函数智能体
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True
    )
    
    return agent

# 3. 自定义智能体工具示例
class WeatherTool(BaseTool):
    name = "天气查询"
    description = "查询指定城市的天气情况"
    
    def _run(self, city: str) -> str:
        # 模拟天气API调用
        return f"{city}的天气: 晴朗, 25°C, 湿度50%"
        
    async def _arun(self, city: str) -> str:
        return self._run(city)

class CalculatorTool(BaseTool):
    name = "计算器"
    description = "进行数学计算，输入应为数学表达式"
    
    def _run(self, expression: str) -> str:
        try:
            result = eval(expression)
            return f"计算结果: {result}"
        except Exception as e:
            return f"计算错误: {str(e)}"
            
    async def _arun(self, expression: str) -> str:
        return self._run(expression)

def create_custom_tool_agent():
    """创建带有自定义工具的智能体"""
    tools = [
        WeatherTool(),
        CalculatorTool(),
        PythonREPLTool()
    ]
    
    llm = ChatOpenAI(temperature=0)
    
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    return agent

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 单智能体模式示例 ===")
    
    # 选择要演示的智能体类型 
    agent_type = "react"  # 可选: "react", "openai_functions", "custom"
    
    if agent_type == "react":
        agent = create_react_agent()
        response = agent.run(
            "谁是阿尔伯特·爱因斯坦? 他出生于哪一年? 计算从他出生到现在过了多少年。"
        )
    elif agent_type == "openai_functions":
        agent = create_openai_functions_agent() 
        response = agent.run(
            "计算 2345 + 5678 的结果，并解释这两个数字的数学特性。"
        )
    elif agent_type == "custom":
        agent = create_custom_tool_agent()
        response = agent.run(
            "北京的天气如何？然后计算25乘以4的结果。"
        )
        
    print(f"\\n最终回答: {response}")
"""
    
    # Write the file
    editor.write("langchain_tutorial/code_examples/agent_examples/single_agent_example.py", single_agent_content)
    print("Created single agent examples file")

async def create_multi_agent_file():
    """Create the multi-agent examples file"""
    multi_agent_content = """# LangChain 多智能体编排模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.python.tool import PythonREPLTool

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 1. 团队监督者模式 (Team Supervisor)
class TeamSupervisorSystem:
    """团队监督者模式 - 一个监督者智能体协调多个专家智能体"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建专家智能体
        self.researcher = self._create_researcher_agent()
        self.coder = self._create_coder_agent()
        self.critic = self._create_critic_agent()
        
        # 创建监督者
        self.supervisor = self._create_supervisor_agent()
    
    def _create_researcher_agent(self):
        """创建研究员智能体"""
        wikipedia = WikipediaAPIWrapper()
        tools = [
            Tool(
                name="维基百科",
                func=wikipedia.run,
                description="用于查询事实性信息的维基百科工具"
            )
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_coder_agent(self):
        """创建编码员智能体"""
        tools = [PythonREPLTool()]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_critic_agent(self):
        """创建评论员智能体"""
        prompt = PromptTemplate(
            template="你是一位严格的评论家。请评估以下解决方案的质量、正确性和效率：\\n{solution}\\n请提供详细的改进建议。",
            input_variables=["solution"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt, verbose=True)
    
    def _create_supervisor_agent(self):
        """创建监督者智能体"""
        prompt = PromptTemplate(
            template="你是一个团队领导，负责协调其他专家完成任务。你需要将复杂问题分解成子任务，分配给适当的专家，并整合他们的工作。\\n\\n任务描述: {task}\\n\\n请制定一个计划，说明如何将这个任务分解并分配给团队成员（研究员、编码员、评论员）。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行团队监督者系统"""
        print("\\n===== 团队监督者模式 =====")
        print(f"任务: {task}")
        
        # 1. 监督者分配任务
        print("\\n1. 监督者制定计划")
        plan = self.supervisor.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 研究员收集信息
        print("\\n2. 研究员收集信息")
        research_query = f"请查询关于以下内容的事实性信息: {task}"
        research_result = self.researcher.run(research_query)
        print(f"\\n研究结果:\\n{research_result}")
        
        # 3. 编码员根据研究结果实现解决方案
        print("\\n3. 编码员实现解决方案")
        coding_task = f"根据以下研究结果，编写Python代码解决问题: {task}\\n\\n研究信息: {research_result}"
        code_solution = self.coder.run(coding_task)
        print(f"\\n代码解决方案:\\n{code_solution}")
        
        # 4. 评论员评估解决方案
        print("\\n4. 评论员评估解决方案")
        critique = self.critic.run(solution=code_solution)
        print(f"\\n评估意见:\\n{critique}")
        
        # 5. 监督者整合结果
        print("\\n5. 监督者整合最终结果")
        final_prompt = f"请整合以下信息，提供最终的解决方案和总结：\\n\\n原始任务: {task}\\n\\n研究结果: {research_result}\\n\\n代码解决方案: {code_solution}\\n\\n评估意见: {critique}"
        final_solution = self.llm.predict(final_prompt)
        print(f"\\n最终解决方案:\\n{final_solution}")
        
        return {
            "task": task,
            "plan": plan,
            "research": research_result,
            "code": code_solution,
            "critique": critique,
            "final_solution": final_solution
        }

# 2. 计划执行模式 (Plan Executor)
class PlanExecutorSystem:
    """计划执行模式 - 一个智能体制定计划，另一个智能体执行计划"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建规划者智能体
        self.planner = self._create_planner()
        
        # 创建执行者智能体
        self.executor = self._create_executor()
    
    def _create_planner(self):
        """创建规划者智能体"""
        prompt = PromptTemplate(
            template="你是一位专业的规划者。请为以下任务制定一个详细的、分步骤的计划：\\n\\n任务: {task}\\n\\n请提供一个清晰的、有序的计划，每个步骤应该具体且可执行。确保计划涵盖任务的所有方面，并考虑可能的障碍和解决方案。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def _create_executor(self):
        """创建执行者智能体"""
        tools = [
            Tool(
                name="维基百科",
                func=WikipediaAPIWrapper().run,
                description="用于查询信息的工具"
            ),
            PythonREPLTool()
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def run(self, task: str) -> Dict[str, Any]:
        """运行计划执行系统"""
        print("\\n===== 计划执行模式 =====")
        print(f"任务: {task}")
        
        # 1. 制定计划
        print("\\n1. 规划者制定计划")
        plan = self.planner.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 执行计划
        print("\\n2. 执行者执行计划")
        execution_task = f"根据以下计划执行任务：\\n\\n任务: {task}\\n\\n计划:\\n{plan}\\n\\n执行每个步骤并报告结果。"
        execution_result = self.executor.run(execution_task)
        print(f"\\n执行结果:\\n{execution_result}")
        
        return {
            "task": task,
            "plan": plan,
            "execution_result": execution_result
        }

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 多智能体编排模式示例 ===")
    
    # 选择要演示的编排模式
    orchestration_type = "team_supervisor"  # 可选: "team_supervisor", "plan_executor"
    
    if orchestration_type == "team_supervisor":
        system = TeamSupervisorSystem()
        result = system.run("创建一个简单的网站访问计数器应用")
        
    elif orchestration_type == "plan_executor":
        system = PlanExecutorSystem()
        result = system.run("研究机器学习中的过拟合问题并提供3种解决方法")
"""
    
    # Write the file
    editor.write("langchain_tutorial/code_examples/agent_examples/multi_agent_example.py", multi_agent_content)
    print("Created multi-agent examples file")

async def create_rag_agent_file():
    """Create the RAG+Agent integration example file"""
    rag_agent_content = """# LangChain RAG与智能体结合示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class RAGAgentExample:
    """
    检索增强生成(RAG)与智能体结合示例
    展示如何将RAG系统集成到智能体工作流中
    """
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vector_store = None
        self.retriever = None
    
    def load_documents(self, texts: List[str]):
        """加载文档到系统中"""
        print("加载文档...")
        
        # 创建临时文件
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        # 将文本写入临时文件
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个文档")
    
    def process_documents(self):
        """处理文档：分块和创建向量存储"""
        if not self.documents:
            raise ValueError("请先加载文档")
        
        print("处理文档：分块和向量化...")
        
        # 分块
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        chunks = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vector_store = FAISS.from_documents(chunks, embeddings)
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        
        print(f"文档处理完成，共创建 {len(chunks)} 个文本块")
        
        return self.retriever
    
    def create_standalone_rag_chain(self):
        """创建独立的RAG链"""
        if not self.retriever:
            raise ValueError("请先处理文档")
        
        # 创建RAG提示模板
        rag_prompt = PromptTemplate.from_template(\"\"\"
        请使用以下检索的信息回答问题。如果检索的信息中没有答案，请说明你不知道。

        检索的信息:
        {context}

        问题: {question}
        \"\"\")
        
        # 定义文档格式化函数
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建RAG链
        rag_chain = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | rag_prompt
            | self.llm
        )
        
        return rag_chain
    
    def create_rag_tool(self):
        """将RAG系统封装为智能体工具"""
        rag_chain = self.create_standalone_rag_chain()
        
        def query_knowledge_base(query: str) -> str:
            """查询知识库获取信息"""
            result = rag_chain.invoke(query)
            return result.content if hasattr(result, "content") else str(result)
        
        # 创建RAG工具
        rag_tool = Tool(
            name="知识库查询",
            func=query_knowledge_base,
            description="当你需要查询特定领域知识时使用此工具"
        )
        
        return rag_tool
    
    def create_agent_with_rag(self):
        """创建集成RAG的智能体"""
        # 获取RAG工具
        rag_tool = self.create_rag_tool()
        
        # 添加其他工具
        python_tool = PythonREPLTool()
        
        # 创建智能体
        agent = initialize_agent(
            tools=[rag_tool, python_tool],
            llm=self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
        
        return agent

# 使用示例
if __name__ == "__main__":
    print("=== LangChain RAG与智能体结合示例 ===")
    
    # 1. 运行直接RAG问答示例
    print("\\n===== 直接RAG问答示例 =====")
    
    # 创建RAG系统
    rag_system = RAGAgentExample()
    
    # 加载文档
    documents = [
        \"\"\"
        LangChain是一个用于开发由语言模型驱动的应用程序的框架。它的核心组件包括:
        
        1. 模型I/O (Models): 与各种语言模型提供商的集成接口。
        2. 提示管理 (Prompts): 优化和管理提示模板。 
        3. 记忆系统 (Memory): 维护对话状态和历史。
        4. 索引和检索 (Indexes): 使用向量存储等技术组织和检索信息。
        5. 链 (Chains): 将多个组件串联成管道。
        6. 智能体 (Agents): 允许LLM决定使用哪些工具来完成任务。
        \"\"\",
        \"\"\"
        LangChain表达式语言(LCEL)是一种声明式语言，用于组合LangChain的组件。它具有以下特性:
        
        1. 链式API: 使用管道操作符(|)连接组件。
        2. 并行执行: 可以同时执行多个操作并合并结果。
        3. 错误处理: 内置重试和异常处理机制。
        4. 批处理: 一次处理多个输入以提高效率。
        5. 流式传输: 支持逐步生成和处理响应。
        
        示例: chain = prompt | llm | output_parser
        \"\"\"
    ]
    rag_system.load_documents(documents)
    rag_system.process_documents()
    
    # 创建RAG链
    rag_chain = rag_system.create_standalone_rag_chain()
    
    # 查询
    queries = [
        "LangChain的核心组件有哪些?",
        "什么是LCEL，它有什么特点?"
    ]
    
    for query in queries:
        print(f"\\n问题: {query}")
        response = rag_chain.invoke(query)
        print(f"回答: {response.content if hasattr(response, 'content') else response}")
    
    # 2. 运行智能体与RAG集成示例
    print("\\n===== 智能体与RAG集成示例 =====")
    
    # 创建智能体
    agent = rag_system.create_agent_with_rag()
    
    # 查询
    agent_queries = [
        "总结LangChain的核心组件并使用Python代码计算它们的总数",
        "请解释LCEL中的管道操作符是如何工作的"
    ]
    
    for query in agent_queries:
        print(f"\\n智能体查询: {query}")
        response = agent.run(query)
        print(f"智能体回答: {response}")
"""
    
    # Write the file
    editor.write("langchain_tutorial/code_examples/agent_examples/rag_agent_example.py", rag_agent_content)
    print("Created RAG agent integration example file")
    
    # Create a README file for the examples
    readme_content = """# LangChain 智能体和多智能体编排示例

本目录包含 LangChain 框架中单智能体和多智能体编排的实用示例代码。

## 文件说明

1. `single_agent_example.py` - 演示不同类型的单智能体实现
   - ReAct 智能体：结合推理和行动的智能体
   - OpenAI函数调用智能体：基于函数调用能力的智能体
   - 自定义工具智能体：使用自定义工具的智能体

2. `multi_agent_example.py` - 演示多智能体编排模式
   - 团队监督者模式：一个监督者智能体协调多个专家智能体工作
   - 计划执行模式：一个智能体制定计划，另一个智能体执行计划

3. `rag_agent_example.py` - 演示RAG与智能体结合的实现
   - 直接RAG问答：使用检索增强生成直接回答问题
   - 智能体与RAG集成：智能体可以主动使用RAG工具获取知识

## 使用说明

1. 确保设置必要的环境变量：
   ```python
   os.environ["OPENAI_API_KEY"] = "your-api-key"
   ```

2. 安装所需依赖：
   ```bash
   pip install langchain langchain_community faiss-cpu openai
   ```

3. 运行示例：
   ```bash
   python single_agent_example.py
   python multi_agent_example.py
   python rag_agent_example.py
   ```

## 注意事项

- 这些示例旨在展示核心概念，实际应用时可能需要进一步配置和优化
- 代码中的某些功能可能需要特定的API访问权限
- RAG示例中使用了临时文件来模拟文档加载，实际应用中可以直接从真实数据源加载
"""
    
    editor.write("langchain_tutorial/code_examples/agent_examples/README.md", readme_content)
    print("Created README file")

# Execute the main function
await main()

SyntaxError: invalid character '，' (U+FF0C) (4002838069.py, line 320)

In [15]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directories for code examples
async def main():
    # Create directories
    await terminal.run("mkdir -p langchain_tutorial/code_examples/agent_examples")
    
    # Create single agent example file
    single_agent_content = """# LangChain 单智能体模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.tools.python.tool import PythonREPLTool
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key" 

# 1. ReAct 智能体示例 - 结合推理和行动的智能体
def create_react_agent():
    \"\"\"创建基本的ReAct智能体\"\"\"
    # 定义工具集
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="用于查询维基百科文章的工具"
        ),
        Tool(
            name="Python解释器",
            func=python_repl.run,
            description="用于执行Python代码的工具，可以进行计算或数据分析"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0)
    
    # 创建记忆组件
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # 初始化ReAct智能体
    agent = initialize_agent(
        tools, 
        llm, 
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        memory=memory,
        handle_parsing_errors=True
    )
    
    return agent

# 2. OpenAI函数智能体示例 - 专为函数调用设计的智能体
def create_openai_functions_agent():
    \"\"\"创建基于OpenAI函数调用的智能体\"\"\"
    # 定义工具集
    wikipedia = WikipediaAPIWrapper()
    python_repl = PythonREPLTool()
    
    tools = [
        Tool(
            name="Python执行器",
            func=python_repl.run,
            description="执行Python代码的工具，适合进行计算、数据处理"
        ),
        Tool(
            name="维基百科",
            func=wikipedia.run,
            description="搜索维基百科文章的工具，适合查询事实性信息"
        )
    ]
    
    # 创建LLM
    llm = ChatOpenAI(temperature=0)
    
    # 初始化OpenAI函数智能体
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=True
    )
    
    return agent

# 3. 自定义智能体工具示例
class WeatherTool(BaseTool):
    name = "天气查询"
    description = "查询指定城市的天气情况"
    
    def _run(self, city: str) -> str:
        # 模拟天气API调用
        return f"{city}的天气: 晴朗, 25°C, 湿度50%"
        
    async def _arun(self, city: str) -> str:
        return self._run(city)

class CalculatorTool(BaseTool):
    name = "计算器"
    description = "进行数学计算，输入应为数学表达式"
    
    def _run(self, expression: str) -> str:
        try:
            result = eval(expression)
            return f"计算结果: {result}"
        except Exception as e:
            return f"计算错误: {str(e)}"
            
    async def _arun(self, expression: str) -> str:
        return self._run(expression)

def create_custom_tool_agent():
    \"\"\"创建带有自定义工具的智能体\"\"\"
    tools = [
        WeatherTool(),
        CalculatorTool(),
        PythonREPLTool()
    ]
    
    llm = ChatOpenAI(temperature=0)
    
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    return agent

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 单智能体模式示例 ===")
    
    # 选择要演示的智能体类型 
    agent_type = "react"  # 可选: "react", "openai_functions", "custom"
    
    if agent_type == "react":
        agent = create_react_agent()
        response = agent.run(
            "谁是阿尔伯特·爱因斯坦? 他出生于哪一年? 计算从他出生到现在过了多少年。"
        )
    elif agent_type == "openai_functions":
        agent = create_openai_functions_agent() 
        response = agent.run(
            "计算 2345 + 5678 的结果，并解释这两个数字的数学特性。"
        )
    elif agent_type == "custom":
        agent = create_custom_tool_agent()
        response = agent.run(
            "北京的天气如何？然后计算25乘以4的结果。"
        )
        
    print(f"\\n最终回答: {response}")
"""
    
    # Write the file
    editor.write("langchain_tutorial/code_examples/agent_examples/single_agent_example.py", single_agent_content)
    print("Created single agent examples file")

    # Create the multi-agent examples file
    multi_agent_content = """# LangChain 多智能体编排模式示例

import os
from typing import List, Dict, Any
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.python.tool import PythonREPLTool

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 1. 团队监督者模式 (Team Supervisor)
class TeamSupervisorSystem:
    \"\"\"团队监督者模式 - 一个监督者智能体协调多个专家智能体\"\"\"
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建专家智能体
        self.researcher = self._create_researcher_agent()
        self.coder = self._create_coder_agent()
        self.critic = self._create_critic_agent()
        
        # 创建监督者
        self.supervisor = self._create_supervisor_agent()
    
    def _create_researcher_agent(self):
        \"\"\"创建研究员智能体\"\"\"
        wikipedia = WikipediaAPIWrapper()
        tools = [
            Tool(
                name="维基百科",
                func=wikipedia.run,
                description="用于查询事实性信息的维基百科工具"
            )
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_coder_agent(self):
        \"\"\"创建编码员智能体\"\"\"
        tools = [PythonREPLTool()]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def _create_critic_agent(self):
        \"\"\"创建评论员智能体\"\"\"
        prompt = PromptTemplate(
            template="你是一位严格的评论家。请评估以下解决方案的质量、正确性和效率：\\n{solution}\\n请提供详细的改进建议。",
            input_variables=["solution"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt, verbose=True)
    
    def _create_supervisor_agent(self):
        \"\"\"创建监督者智能体\"\"\"
        prompt = PromptTemplate(
            template="你是一个团队领导，负责协调其他专家完成任务。你需要将复杂问题分解成子任务，分配给适当的专家，并整合他们的工作。\\n\\n任务描述: {task}\\n\\n请制定一个计划，说明如何将这个任务分解并分配给团队成员（研究员、编码员、评论员）。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def run(self, task: str) -> Dict[str, Any]:
        \"\"\"运行团队监督者系统\"\"\"
        print("\\n===== 团队监督者模式 =====")
        print(f"任务: {task}")
        
        # 1. 监督者分配任务
        print("\\n1. 监督者制定计划")
        plan = self.supervisor.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 研究员收集信息
        print("\\n2. 研究员收集信息")
        research_query = f"请查询关于以下内容的事实性信息: {task}"
        research_result = self.researcher.run(research_query)
        print(f"\\n研究结果:\\n{research_result}")
        
        # 3. 编码员根据研究结果实现解决方案
        print("\\n3. 编码员实现解决方案")
        coding_task = f"根据以下研究结果，编写Python代码解决问题: {task}\\n\\n研究信息: {research_result}"
        code_solution = self.coder.run(coding_task)
        print(f"\\n代码解决方案:\\n{code_solution}")
        
        # 4. 评论员评估解决方案
        print("\\n4. 评论员评估解决方案")
        critique = self.critic.run(solution=code_solution)
        print(f"\\n评估意见:\\n{critique}")
        
        # 5. 监督者整合结果
        print("\\n5. 监督者整合最终结果")
        final_prompt = f"请整合以下信息，提供最终的解决方案和总结：\\n\\n原始任务: {task}\\n\\n研究结果: {research_result}\\n\\n代码解决方案: {code_solution}\\n\\n评估意见: {critique}"
        final_solution = self.llm.predict(final_prompt)
        print(f"\\n最终解决方案:\\n{final_solution}")
        
        return {
            "task": task,
            "plan": plan,
            "research": research_result,
            "code": code_solution,
            "critique": critique,
            "final_solution": final_solution
        }

# 2. 计划执行模式 (Plan Executor)
class PlanExecutorSystem:
    \"\"\"计划执行模式 - 一个智能体制定计划，另一个智能体执行计划\"\"\"
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        
        # 创建规划者智能体
        self.planner = self._create_planner()
        
        # 创建执行者智能体
        self.executor = self._create_executor()
    
    def _create_planner(self):
        \"\"\"创建规划者智能体\"\"\"
        prompt = PromptTemplate(
            template="你是一位专业的规划者。请为以下任务制定一个详细的、分步骤的计划：\\n\\n任务: {task}\\n\\n请提供一个清晰的、有序的计划，每个步骤应该具体且可执行。确保计划涵盖任务的所有方面，并考虑可能的障碍和解决方案。",
            input_variables=["task"]
        )
        
        return LLMChain(llm=self.llm, prompt=prompt)
    
    def _create_executor(self):
        \"\"\"创建执行者智能体\"\"\"
        tools = [
            Tool(
                name="维基百科",
                func=WikipediaAPIWrapper().run,
                description="用于查询信息的工具"
            ),
            PythonREPLTool()
        ]
        
        return initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
    
    def run(self, task: str) -> Dict[str, Any]:
        \"\"\"运行计划执行系统\"\"\"
        print("\\n===== 计划执行模式 =====")
        print(f"任务: {task}")
        
        # 1. 制定计划
        print("\\n1. 规划者制定计划")
        plan = self.planner.run(task=task)
        print(f"\\n计划:\\n{plan}")
        
        # 2. 执行计划
        print("\\n2. 执行者执行计划")
        execution_task = f"根据以下计划执行任务：\\n\\n任务: {task}\\n\\n计划:\\n{plan}\\n\\n执行每个步骤并报告结果。"
        execution_result = self.executor.run(execution_task)
        print(f"\\n执行结果:\\n{execution_result}")
        
        return {
            "task": task,
            "plan": plan,
            "execution_result": execution_result
        }

# 使用示例
if __name__ == "__main__":
    print("=== LangChain 多智能体编排模式示例 ===")
    
    # 选择要演示的编排模式
    orchestration_type = "team_supervisor"  # 可选: "team_supervisor", "plan_executor"
    
    if orchestration_type == "team_supervisor":
        system = TeamSupervisorSystem()
        result = system.run("创建一个简单的网站访问计数器应用")
        
    elif orchestration_type == "plan_executor":
        system = PlanExecutorSystem()
        result = system.run("研究机器学习中的过拟合问题并提供3种解决方法")
"""
    
    # Write the file
    editor.write("langchain_tutorial/code_examples/agent_examples/multi_agent_example.py", multi_agent_content)
    print("Created multi-agent examples file")

    # Create the RAG+Agent integration example file
    rag_agent_content = """# LangChain RAG与智能体结合示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class RAGAgentExample:
    \"\"\"
    检索增强生成(RAG)与智能体结合示例
    展示如何将RAG系统集成到智能体工作流中
    \"\"\"
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vector_store = None
        self.retriever = None
    
    def load_documents(self, texts: List[str]):
        \"\"\"加载文档到系统中\"\"\"
        print("加载文档...")
        
        # 创建临时文件
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        # 将文本写入临时文件
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个文档")
    
    def process_documents(self):
        \"\"\"处理文档：分块和创建向量存储\"\"\"
        if not self.documents:
            raise ValueError("请先加载文档")
        
        print("处理文档：分块和向量化...")
        
        # 分块
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        chunks = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vector_store = FAISS.from_documents(chunks, embeddings)
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        
        print(f"文档处理完成，共创建 {len(chunks)} 个文本块")
        
        return self.retriever
    
    def create_standalone_rag_chain(self):
        \"\"\"创建独立的RAG链\"\"\"
        if not self.retriever:
            raise ValueError("请先处理文档")
        
        # 创建RAG提示模板
        rag_prompt = PromptTemplate.from_template(\"\"\"
        请使用以下检索的信息回答问题。如果检索的信息中没有答案，请说明你不知道。

        检索的信息:
        {context}

        问题: {question}
        \"\"\")
        
        # 定义文档格式化函数
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建RAG链
        rag_chain = (
            {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
            | rag_prompt
            | self.llm
        )
        
        return rag_chain
    
    def create_rag_tool(self):
        \"\"\"将RAG系统封装为智能体工具\"\"\"
        rag_chain = self.create_standalone_rag_chain()
        
        def query_knowledge_base(query: str) -> str:
            \"\"\"查询知识库获取信息\"\"\"
            result = rag_chain.invoke(query)
            return result.content if hasattr(result, "content") else str(result)
        
        # 创建RAG工具
        rag_tool = Tool(
            name="知识库查询",
            func=query_knowledge_base,
            description="当你需要查询特定领域知识时使用此工具"
        )
        
        return rag_tool
    
    def create_agent_with_rag(self):
        \"\"\"创建集成RAG的智能体\"\"\"
        # 获取RAG工具
        rag_tool = self.create_rag_tool()
        
        # 添加其他工具
        python_tool = PythonREPLTool()
        
        # 创建智能体
        agent = initialize_agent(
            tools=[rag_tool, python_tool],
            llm=self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
        
        return agent

# 使用示例
if __name__ == "__main__":
    print("=== LangChain RAG与智能体结合示例 ===")
    
    # 1. 运行直接RAG问答示例
    print("\\n===== 直接RAG问答示例 =====")
    
    # 创建RAG系统
    rag_system = RAGAgentExample()
    
    # 加载文档
    documents = [
        \"\"\"
        LangChain是一个用于开发由语言模型驱动的应用程序的框架。它的核心组件包括:
        
        1. 模型I/O (Models): 与各种语言模型提供商的集成接口。
        2. 提示管理 (Prompts): 优化和管理提示模板。 
        3. 记忆系统 (Memory): 维护对话状态和历史。
        4. 索引和检索 (Indexes): 使用向量存储等技术组织和检索信息。
        5. 链 (Chains): 将多个组件串联成管道。
        6. 智能体 (Agents): 允许LLM决定使用哪些工具来完成任务。
        \"\"\",
        \"\"\"
        LangChain表达式语言(LCEL)是一种声明式语言，用于组合LangChain的组件。它具有以下特性:
        
        1. 链式API: 使用管道操作符(|)连接组件。
        2. 并行执行: 可以同时执行多个操作并合并结果。
        3. 错误处理: 内置重试和异常处理机制。
        4. 批处理: 一次处理多个输入以提高效率。
        5. 流式传输: 支持逐步生成和处理响应。
        
        示例: chain = prompt | llm | output_parser
        \"\"\"
    ]
    rag_system.load_documents(documents)
    rag_system.process_documents()
    
    # 创建RAG链
    rag_chain = rag_system.create_standalone_rag_chain()
    
    # 查询
    queries = [
        "LangChain的核心组件有哪些?",
        "什么是LCEL，它有什么特点?"
    ]
    
    for query in queries:
        print(f"\\n问题: {query}")
        response = rag_chain.invoke(query)
        print(f"回答: {response.content if hasattr(response, 'content') else response}")
    
    # 2. 运行智能体与RAG集成示例
    print("\\n===== 智能体与RAG集成示例 =====")
    
    # 创建智能体
    agent = rag_system.create_agent_with_rag()
    
    # 查询
    agent_queries = [
        "总结LangChain的核心组件并使用Python代码计算它们的总数",
        "请解释LCEL中的管道操作符是如何工作的"
    ]
    
    for query in agent_queries:
        print(f"\\n智能体查询: {query}")
        response = agent.run(query)
        print(f"智能体回答: {response}")
"""
    
    # Write the file
    editor.write("langchain_tutorial/code_examples/agent_examples/rag_agent_example.py", rag_agent_content)
    print("Created RAG agent integration example file")
    
    # Create a README file for the examples
    readme_content = """# LangChain 智能体和多智能体编排示例

本目录包含 LangChain 框架中单智能体和多智能体编排的实用示例代码。

## 文件说明

1. `single_agent_example.py` - 演示不同类型的单智能体实现
   - ReAct 智能体：结合推理和行动的智能体
   - OpenAI函数调用智能体：基于函数调用能力的智能体
   - 自定义工具智能体：使用自定义工具的智能体

2. `multi_agent_example.py` - 演示多智能体编排模式
   - 团队监督者模式：一个监督者智能体协调多个专家智能体工作
   - 计划执行模式：一个智能体制定计划，另一个智能体执行计划

3. `rag_agent_example.py` - 演示RAG与智能体结合的实现
   - 直接RAG问答：使用检索增强生成直接回答问题
   - 智能体与RAG集成：智能体可以主动使用RAG工具获取知识

## 使用说明

1. 确保设置必要的环境变量：
   ```python
   os.environ["OPENAI_API_KEY"] = "your-api-key"
   ```

2. 安装所需依赖：
   ```bash
   pip install langchain langchain_community faiss-cpu openai
   ```

3. 运行示例：
   ```bash
   python single_agent_example.py
   python multi_agent_example.py
   python rag_agent_example.py
   ```

## 注意事项

- 这些示例旨在展示核心概念，实际应用时可能需要进一步配置和优化
- 代码中的某些功能可能需要特定的API访问权限
- RAG示例中使用了临时文件来模拟文档加载，实际应用中可以直接从真实数据源加载
"""
    
    editor.write("langchain_tutorial/code_examples/agent_examples/README.md", readme_content)
    print("Created README file")
    
    # Show the created files
    files = await terminal.run("find langchain_tutorial/code_examples/agent_examples -type f | sort")
    print(f"\nCreated example files:\n{files}")
    
    print("\n========== Code Examples Creation Complete ==========")
    print("Created comprehensive examples for:")
    print("1. Single agent implementation patterns")
    print("2. Multi-agent orchestration patterns")
    print("3. RAG integration with agents")

# Execute the main function
await main()

Created single agent examples file
Created multi-agent examples file
Created RAG agent integration example file
Created README file

Created example files:

langchain_tutorial/code_examples/agent_examples/README.md
langchain_tutorial/code_examples/agent_examples/multi_agent_example.py
langchain_tutorial/code_examples/agent_examples/rag_agent_example.py
langchain_tutorial/code_examples/agent_examples/single_agent_example.py


========== Code Examples Creation Complete ==========
Created comprehensive examples for:
1. Single agent implementation patterns
2. Multi-agent orchestration patterns
3. RAG integration with agents


In [16]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create a structured summary document that organizes the information
# on LangChain's single and multi-agent orchestration patterns
summary_content = """# LangChain 单智能体与多智能体编排模式总结

## 1. 智能体架构概述

LangChain的智能体系统是一个强大的框架，允许语言模型（LLMs）通过工具与外部环境交互，从而完成复杂任务。

### 1.1 智能体系统的核心组件

- **智能体 (Agent)**: 负责决策和推理的语言模型
- **工具 (Tools)**: 智能体可以使用的函数或API
- **执行器 (Executor)**: 协调智能体与工具之间的交互
- **记忆 (Memory)**: 存储对话或执行历史
- **观察 (Observation)**: 工具执行的结果反馈

## 2. 单智能体实现模式

### 2.1 ReAct 智能体

ReAct（Reasoning + Acting）智能体结合了推理和行动的能力，是最常用的智能体类型之一。

**核心特性:**
- 结合推理（思考）和行动的能力
- 提供中间推理步骤
- 支持"链式思考"过程
- 适用于需要复杂逻辑和推理的任务

**实现代码示例:**

SyntaxError: incomplete input (2040116023.py, line 10)

In [17]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create a structured summary document that organizes the information
# on LangChain's single and multi-agent orchestration patterns
summary_content = """# LangChain 单智能体与多智能体编排模式总结

## 1. 智能体架构概述

LangChain的智能体系统是一个强大的框架，允许语言模型（LLMs）通过工具与外部环境交互，从而完成复杂任务。

### 1.1 智能体系统的核心组件

- **智能体 (Agent)**: 负责决策和推理的语言模型
- **工具 (Tools)**: 智能体可以使用的函数或API
- **执行器 (Executor)**: 协调智能体与工具之间的交互
- **记忆 (Memory)**: 存储对话或执行历史
- **观察 (Observation)**: 工具执行的结果反馈

## 2. 单智能体实现模式

### 2.1 ReAct 智能体

ReAct（Reasoning + Acting）智能体结合了推理和行动的能力，是最常用的智能体类型之一。

**核心特性:**
- 结合推理（思考）和行动的能力
- 提供中间推理步骤
- 支持"链式思考"过程
- 适用于需要复杂逻辑和推理的任务

**实现示例:**

SyntaxError: incomplete input (479763984.py, line 10)

In [18]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create a structured summary document that organizes the information
# on LangChain's single and multi-agent orchestration patterns
async def create_langchain_agents_summary():
    summary_content = """# LangChain 单智能体与多智能体编排模式总结

## 1. 智能体架构概述

LangChain的智能体系统是一个强大的框架，允许语言模型（LLMs）通过工具与外部环境交互，从而完成复杂任务。

### 1.1 智能体系统的核心组件

- **智能体 (Agent)**: 负责决策和推理的语言模型
- **工具 (Tools)**: 智能体可以使用的函数或API
- **执行器 (Executor)**: 协调智能体与工具之间的交互
- **记忆 (Memory)**: 存储对话或执行历史
- **观察 (Observation)**: 工具执行的结果反馈

## 2. 单智能体实现模式

### 2.1 ReAct 智能体

ReAct（Reasoning + Acting）智能体结合了推理和行动的能力，是最常用的智能体类型之一。

**核心特性:**
- 结合推理（思考）和行动的能力
- 提供中间推理步骤
- 支持"链式思考"过程
- 适用于需要复杂逻辑和推理的任务

**实现示例:**

SyntaxError: incomplete input (980260477.py, line 11)

In [19]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create a comprehensive structured summary document
async def create_langchain_agents_summary():
    summary_content = """# LangChain 单智能体与多智能体编排模式总结

## 1. 智能体架构概述

LangChain的智能体系统是一个强大的框架，允许语言模型（LLMs）通过工具与外部环境交互，从而完成复杂任务。

### 1.1 智能体系统的核心组件

- **智能体 (Agent)**: 负责决策和推理的语言模型
- **工具 (Tools)**: 智能体可以使用的函数或API
- **执行器 (Executor)**: 协调智能体与工具之间的交互
- **记忆 (Memory)**: 存储对话或执行历史
- **观察 (Observation)**: 工具执行的结果反馈

## 2. 单智能体实现模式

### 2.1 ReAct 智能体

ReAct（Reasoning + Acting）智能体结合了推理和行动的能力，是最常用的智能体类型之一。

**核心特性:**
- 结合推理（思考）和行动的能力
- 提供中间推理步骤
- 支持"链式思考"过程
- 适用于需要复杂逻辑和推理的任务

**实现示例:**

SyntaxError: incomplete input (242544012.py, line 10)

In [20]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create a comprehensive structured summary document
async def create_langchain_agents_summary():
    summary_content = """# LangChain 单智能体与多智能体编排模式总结

## 1. 智能体架构概述

LangChain的智能体系统是一个强大的框架，允许语言模型（LLMs）通过工具与外部环境交互，从而完成复杂任务。

### 1.1 智能体系统的核心组件

- **智能体 (Agent)**: 负责决策和推理的语言模型
- **工具 (Tools)**: 智能体可以使用的函数或API
- **执行器 (Executor)**: 协调智能体与工具之间的交互
- **记忆 (Memory)**: 存储对话或执行历史
- **观察 (Observation)**: 工具执行的结果反馈

## 2. 单智能体实现模式

### 2.1 ReAct 智能体

ReAct（Reasoning + Acting）智能体结合了推理和行动的能力，是最常用的智能体类型之一。

**核心特性:**
- 结合推理（思考）和行动的能力
- 提供中间推理步骤
- 支持"链式思考"过程
- 适用于需要复杂逻辑和推理的任务

**实现示例:**

SyntaxError: incomplete input (242544012.py, line 10)

In [21]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

# Create a comprehensive structured summary document
async def create_langchain_agents_summary():
    summary_content = """# LangChain 单智能体与多智能体编排模式总结

## 1. 智能体架构概述

LangChain的智能体系统是一个强大的框架，允许语言模型（LLMs）通过工具与外部环境交互，从而完成复杂任务。

### 1.1 智能体系统的核心组件

- **智能体 (Agent)**: 负责决策和推理的语言模型
- **工具 (Tools)**: 智能体可以使用的函数或API
- **执行器 (Executor)**: 协调智能体与工具之间的交互
- **记忆 (Memory)**: 存储对话或执行历史
- **观察 (Observation)**: 工具执行的结果反馈

## 2. 单智能体实现模式

### 2.1 ReAct 智能体

ReAct（Reasoning + Acting）智能体结合了推理和行动的能力，是最常用的智能体类型之一。

**核心特性:**
- 结合推理（思考）和行动的能力
- 提供中间推理步骤
- 支持"链式思考"过程
- 适用于需要复杂逻辑和推理的任务

**实现方式:**

SyntaxError: incomplete input (1339704324.py, line 10)

In [22]:
from metagpt.tools.libs.web_scraping import view_page_element_to_scrape
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import asyncio
from bs4 import BeautifulSoup
import os

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for search and agent integration
await terminal.run("mkdir -p langchain_tutorial/docs/search_agent_integration")

# URLs for search and agent integration documentation
search_agent_urls = [
    ("search_tools", "https://python.langchain.com/docs/modules/agents/tools/"),
    ("search_toolkit", "https://python.langchain.com/docs/modules/agents/toolkits/retrieval"),
    ("vector_stores", "https://python.langchain.com/docs/integrations/vectorstores/"),
    ("retrievers", "https://python.langchain.com/docs/modules/data_connection/retrievers/"),
    ("agent_with_retrieval_tool", "https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db")
]

# Function to extract content
async def extract_search_agent_content(name, url):
    print(f"Fetching {name} documentation from: {url}")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"Extract all information about how search/retrieval capabilities can be integrated with agents",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    content = ""
    if main_content:
        title = main_content.find('h1')
        if title:
            content += f"# {title.get_text(strip=True)}\n\n"
        
        for element in main_content.find_all(['h2', 'h3', 'h4', 'p', 'pre', 'ul', 'ol', 'li', 'div']):
            if element.name in ['h2', 'h3', 'h4']:
                level = int(element.name[1])
                content += f"{'#' * level} {element.get_text(strip=True)}\n\n"
            elif element.name == 'p':
                content += f"{element.get_text(strip=True)}\n\n"
            elif element.name == 'pre':
                code = element.find('code')
                if code:
                    language = "python" if "python" in code.get('class', [''])[0] else ""
                    content += f"```{language}\n{code.get_text()}\n```\n\n"
                else:
                    content += f"```\n{element.get_text()}\n```\n\n"
            elif element.name in ['ul', 'ol']:
                for li in element.find_all('li', recursive=False):
                    content += f"- {li.get_text(strip=True)}\n"
                content += "\n"
    
    return content

# Collect documentation on search and agent integration
for name, url in search_agent_urls:
    content = await extract_search_agent_content(name, url)
    file_path = f"langchain_tutorial/docs/search_agent_integration/{name}.md"
    editor.write(file_path, content + f"\n\nSource: {url}")

# Create a code example demonstrating search agent integration
example_code = """# LangChain 搜索与智能体集成示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class SearchAgentIntegration:
    """搜索与智能体集成的不同方法示例"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vectorstore = None
        self.retriever = None
    
    def load_sample_documents(self):
        """加载示例文档"""
        # 创建示例文档
        texts = [
            """
            LangChain是一个用于构建基于语言模型的应用程序的框架。
            它的核心特点包括:
            1. 组件化: 提供模块化的接口和组件
            2. 可定制性: 可以灵活组合和调整不同组件
            3. 调试能力: 可以轻松追踪应用中的操作
            4. 端到端开发: 从原型到生产的完整解决方案
            """,
            """
            LangChain的检索增强生成(RAG)模式是将外部知识与LLM结合的强大方法。
            RAG工作流程包括:
            1. 索引阶段: 将文档转换为向量表示并存储
            2. 检索阶段: 查询相关文档片段
            3. 生成阶段: 使用检索到的上下文增强LLM的回答
            这种方法减少了幻觉，提高了回答准确性。
            """,
            """
            LangChain的智能体(Agents)系统允许LLM决定使用哪些工具来解决问题。
            智能体可以:
            1. 拆解复杂问题
            2. 选择并使用合适的工具
            3. 根据工具输出调整策略
            4. 迭代直到达成目标
            智能体与工具的结合使LLM能够与现实世界交互。
            """
        ]
        
        # 加载文档
        import tempfile
        self.documents = []
        temp_dir = tempfile.mkdtemp()
        
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个示例文档")
    
    def process_documents(self):
        """处理文档并创建向量存储"""
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_documents(splits, embeddings)
        self.retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}  # 获取最相关的2个文档块
        )
        
        print(f"文档已处理，共创建 {len(splits)} 个文档块")
    
    def method1_retrieval_qa_chain(self, query: str) -> str:
        """方法1: 使用检索QA链 - 最基本的检索增强问答"""
        print("\n===== 方法1: 使用检索QA链 =====")
        
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",  # 将所有文档合并到提示中
            retriever=self.retriever,
            verbose=True
        )
        
        result = qa_chain({"query": query})
        return result["result"]
    
    def method2_retrieval_tool_for_agent(self, query: str) -> str:
        """方法2: 将检索器作为智能体工具 - 智能体可以主动决定何时搜索信息"""
        print("\n===== 方法2: 将检索器作为智能体工具 =====")
        
        # 创建检索工具
        def search_docs(query: str) -> str:
            docs = self.retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 定义工具
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当你需要查询关于LangChain的信息时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        # 执行查询
        result = agent.run(query)
        return result
    
    def method3_contextual_agent(self, query: str) -> str:
        """方法3: 基于上下文的智能体 - 先检索相关文档，然后让智能体基于上下文回答"""
        print("\n===== 方法3: 基于上下文的智能体 =====")
        
        # 首先获取相关文档
        docs = self.retriever.get_relevant_documents(query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建带上下文的工具
        python_tool = PythonREPLTool()
        
        tools = [
            Tool(
                name="Python执行器",
                func=python_tool.run,
                description="执行Python代码，适用于计算、数据处理等"
            )
        ]
        
        # 创建基于上下文的提示
        contextual_prompt = f"你有以下参考信息:\\n\\n{context}\\n\\n基于这些信息回答问题: {query}"
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        # 执行具有上下文的查询
        result = agent.run(contextual_prompt)
        return result

# 主执行函数
def main():
    print("=== LangChain 搜索与智能体集成示例 ===")
    
    # 创建示例
    example = SearchAgentIntegration()
    
    # 加载和处理文档
    example.load_sample_documents()
    example.process_documents()
    
    # 测试不同的集成方法
    queries = [
        "LangChain的核心特点是什么?",
        "解释RAG模式是如何工作的，并计算它的步骤数量",
        "智能体如何与外部工具交互?"
    ]
    
    for i, query in enumerate(queries):
        print(f"\\n\\n问题 {i+1}: {query}")
        
        # 根据问题类型选择不同的方法
        if i == 0:
            # 简单的事实查询 - 使用基本检索QA
            answer = example.method1_retrieval_qa_chain(query)
        elif i == 1:
            # 需要计算的查询 - 使用检索工具的智能体
            answer = example.method2_retrieval_tool_for_agent(query)
        else:
            # 需要更多上下文的复杂查询 - 使用上下文智能体
            answer = example.method3_contextual_agent(query)
        
        print(f"\\n回答: {answer}")
        print("\\n" + "-"*50)

if __name__ == "__main__":
    main()
"""

# Write the example code
await terminal.run("mkdir -p langchain_tutorial/code_examples/search_agent_integration")
editor.write("langchain_tutorial/code_examples/search_agent_integration/search_agent_methods.py", example_code)

# Create an advanced hybrid search agent example
hybrid_search_agent_code = """# LangChain 高级混合检索智能体示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor, create_react_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools.python.tool import PythonREPLTool
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.schema import Document
from langchain.prompts import PromptTemplate
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class HybridSearchTool(Tool):
    """混合搜索工具 - 结合关键词和向量搜索"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        # 创建基本检索器
        self.basic_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        
        # 创建上下文压缩检索器
        compressor = LLMChainExtractor.from_llm(llm)
        self.compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=self.basic_retriever
        )
        
        super().__init__(
            name="混合知识库搜索",
            func=self.search,
            description="当你需要查询文档中的特定信息时使用此工具，输入应为明确的问题或信息需求"
        )
    
    def search(self, query: str) -> str:
        """执行混合搜索"""
        # 获取向量检索结果
        vector_docs = self.basic_retriever.get_relevant_documents(query)
        
        # 使用LLM提取最相关的信息
        compressed_docs = self.compression_retriever.get_relevant_documents(query)
        
        # 合并并去重结果
        all_docs = []
        seen_content = set()
        
        for doc in vector_docs + compressed_docs:
            if doc.page_content not in seen_content:
                all_docs.append(doc)
                seen_content.add(doc.page_content)
        
        # 格式化结果
        if not all_docs:
            return "没有找到相关信息"
        
        return "\\n\\n".join([f"文档 {i+1}:\\n{doc.page_content}" for i, doc in enumerate(all_docs)])


class AdvancedSearchAgent:
    """高级搜索智能体 - 结合多种检索方法并使用对话记忆"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vectorstore = None
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent_executor = None
    
    def load_documents(self, file_paths: Optional[List[str]] = None):
        """加载文档"""
        self.documents = []
        
        # 如果没有提供文件路径，使用示例文档
        if not file_paths:
            temp_dir = tempfile.mkdtemp()
            file_paths = []
            
            # 创建示例文档
            example_texts = [
                """
                # 搜索系统与智能体集成的方法
                
                搜索系统可以通过多种方式与LangChain智能体集成:
                
                1. **作为工具提供**: 封装为工具API供智能体调用
                2. **预检索增强**: 在启动智能体前检索信息
                3. **混合架构**: 两种方法的结合，根据需要使用
                
                每种方法都有其优缺点，适用于不同场景。
                """,
                """
                # 检索增强生成(RAG)架构
                
                RAG架构是结合检索系统和LLM的有效方法，通常包含以下组件:
                
                - 文档加载器: 加载各种格式的文档
                - 文档转换器: 分割文档为可管理的块
                - 文本嵌入模型: 将文本转换为向量表示
                - 向量存储: 存储和索引嵌入向量
                - 检索器: 检索相关信息
                - 生成模型: 基于检索内容生成答案
                
                RAG过程包括索引、检索和生成三个阶段。
                """,
                """
                # 智能体工具类型
                
                LangChain提供多种工具类型可以与智能体结合:
                
                - 检索工具: 从文档中检索信息
                - API工具: 与外部服务交互
                - Python工具: 执行Python代码
                - 人类工具: 获取人类反馈
                - Shell工具: 执行系统命令
                
                智能体可以根据任务需求选择合适的工具。
                """,
                """
                # 上下文压缩技术
                
                上下文压缩技术可以优化检索结果:
                
                1. 嵌入过滤: 基于嵌入相似度过滤
                2. 语义过滤: 基于语义相关性过滤
                3. LLM抽取: 使用LLM提取最相关部分
                4. 重排序: 重新排序检索到的文档
                
                这些技术可以帮助减少噪声，提高检索质量。
                """
            ]
            
            for i, text in enumerate(example_texts):
                file_path = os.path.join(temp_dir, f"doc_{i}.txt")
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(text)
                file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            try:
                loader = TextLoader(file_path)
                self.documents.extend(loader.load())
            except Exception as e:
                print(f"加载文档 {file_path} 时出错: {str(e)}")
        
        print(f"已加载 {len(self.documents)} 个文档")
    
    def process_documents(self):
        """处理文档并创建向量存储"""
        if not self.documents:
            raise ValueError("请先加载文档")
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(splits, embeddings)
        
        print(f"文档已处理，共创建 {len(splits)} 个文档块")
    
    def setup_agent(self):
        """设置智能体"""
        if not self.vectorstore:
            raise ValueError("请先处理文档")
        
        # 创建混合搜索工具
        hybrid_search_tool = HybridSearchTool(self.vectorstore, self.llm)
        
        # 创建Python工具
        python_tool = PythonREPLTool()
        
        # 创建检索QA链作为独立工具
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever()
        )
        
        def qa_tool(query: str) -> str:
            """直接QA问答工具"""
            return qa_chain.run(query)
        
        # 定义工具集
        tools = [
            hybrid_search_tool,
            Tool(
                name="Python执行器",
                func=python_tool.run,
                description="执行Python代码，适用于计算、数据处理等"
            ),
            Tool(
                name="直接问答",
                func=qa_tool,
                description="当你需要直接回答问题而不进行推理时使用此工具"
            )
        ]
        
        # 创建带有记忆的智能体
        # 使用React提示创建智能体
        prompt = PromptTemplate.from_template(
            """你是一个高级搜索智能体，能够访问知识库并执行代码来回答问题。
            
            {chat_history}
            
            你可以使用以下工具:
            {tools}
            
            针对人类的问题，先考虑是否需要搜索信息或执行代码，然后使用合适的工具。
            如果信息已在聊天历史中，可以直接回答而不需要重新搜索。
            
            人类问题: {input}
            让我们一步步思考:"""
        )
        
        agent = create_react_agent(self.llm, tools, prompt)
        
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=5
        )
    
    def ask(self, query: str) -> str:
        """向智能体提问"""
        if not self.agent_executor:
            raise ValueError("请先设置智能体")
        
        result = self.agent_executor.run(query)
        return result


# 主执行函数
def main():
    print("=== LangChain 高级混合检索智能体示例 ===")
    
    # 创建高级搜索智能体
    agent = AdvancedSearchAgent()
    
    # 加载和处理文档
    agent.load_documents()
    agent.process_documents()
    
    # 设置智能体
    agent.setup_agent()
    
    # 测试智能体
    questions = [
        "智能体可以通过哪些方式与搜索系统集成?",
        "什么是RAG架构，它有哪些组件?",
        "基于之前的问题，计算RAG架构中的组件数量",
        "什么是上下文压缩技术，它如何改进检索结果?"
    ]
    
    for i, question in enumerate(questions):
        print(f"\\n\\n问题 {i+1}: {question}")
        answer = agent.ask(question)
        print(f"\\n回答: {answer}")
        print("\\n" + "-"*50)


if __name__ == "__main__":
    main()
"""

# Write the advanced hybrid search agent example
editor.write("langchain_tutorial/code_examples/search_agent_integration/hybrid_search_agent.py", hybrid_search_agent_code)

# Create a comprehensive summary document about search and agent integration
summary_content = """# LangChain 搜索与智能体集成技术总结

## 1. 集成方法概述

LangChain 框架提供了多种将搜索/检索功能与智能体集成的方法，主要包括以下几种模式：

### 1.1 检索工具模式

在这种模式下，检索系统被封装为智能体可以使用的工具。智能体可以根据需要决定何时调用检索工具来获取相关信息。

**优势：**
- 智能体可以决定何时使用检索功能
- 更灵活的交互模式
- 可以结合其他工具使用

**实现方式：**

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 499)

In [24]:
from metagpt.tools.libs.web_scraping import view_page_element_to_scrape
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import asyncio
from bs4 import BeautifulSoup
import os

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for search and agent integration
await terminal.run("mkdir -p langchain_tutorial/docs/search_agent_integration")

# URLs for search and agent integration documentation
search_agent_urls = [
    ("search_tools", "https://python.langchain.com/docs/modules/agents/tools/"),
    ("search_toolkit", "https://python.langchain.com/docs/modules/agents/toolkits/retrieval"),
    ("vector_stores", "https://python.langchain.com/docs/integrations/vectorstores/"),
    ("retrievers", "https://python.langchain.com/docs/modules/data_connection/retrievers/"),
    ("agent_with_retrieval_tool", "https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db")
]

# Function to extract content
async def extract_search_agent_content(name, url):
    print(f"Fetching {name} documentation from: {url}")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"Extract all information about how search/retrieval capabilities can be integrated with agents",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    content = ""
    if main_content:
        title = main_content.find('h1')
        if title:
            content += f"# {title.get_text(strip=True)}\n\n"
        
        for element in main_content.find_all(['h2', 'h3', 'h4', 'p', 'pre', 'ul', 'ol', 'li', 'div']):
            if element.name in ['h2', 'h3', 'h4']:
                level = int(element.name[1])
                content += f"{'#' * level} {element.get_text(strip=True)}\n\n"
            elif element.name == 'p':
                content += f"{element.get_text(strip=True)}\n\n"
            elif element.name == 'pre':
                code = element.find('code')
                if code:
                    language = "python" if "python" in code.get('class', [''])[0] else ""
                    content += f"```{language}\n{code.get_text()}\n```\n\n"
                else:
                    content += f"```\n{element.get_text()}\n```\n\n"
            elif element.name in ['ul', 'ol']:
                for li in element.find_all('li', recursive=False):
                    content += f"- {li.get_text(strip=True)}\n"
                content += "\n"
    
    return content

# Collect documentation on search and agent integration
for name, url in search_agent_urls:
    content = await extract_search_agent_content(name, url)
    file_path = f"langchain_tutorial/docs/search_agent_integration/{name}.md"
    editor.write(file_path, content + f"\n\nSource: {url}")

# Create a code example demonstrating search agent integration
example_code = """# LangChain 搜索与智能体集成示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class SearchAgentIntegration:
    """搜索与智能体集成的不同方法示例"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vectorstore = None
        self.retriever = None
    
    def load_sample_documents(self):
        """加载示例文档"""
        # 创建示例文档
        texts = [
            """
            LangChain是一个用于构建基于语言模型的应用程序的框架。
            它的核心特点包括:
            1. 组件化: 提供模块化的接口和组件
            2. 可定制性: 可以灵活组合和调整不同组件
            3. 调试能力: 可以轻松追踪应用中的操作
            4. 端到端开发: 从原型到生产的完整解决方案
            """,
            """
            LangChain的检索增强生成(RAG)模式是将外部知识与LLM结合的强大方法。
            RAG工作流程包括:
            1. 索引阶段: 将文档转换为向量表示并存储
            2. 检索阶段: 查询相关文档片段
            3. 生成阶段: 使用检索到的上下文增强LLM的回答
            这种方法减少了幻觉，提高了回答准确性。
            """,
            """
            LangChain的智能体(Agents)系统允许LLM决定使用哪些工具来解决问题。
            智能体可以:
            1. 拆解复杂问题
            2. 选择并使用合适的工具
            3. 根据工具输出调整策略
            4. 迭代直到达成目标
            智能体与工具的结合使LLM能够与现实世界交互。
            """
        ]
        
        # 加载文档
        import tempfile
        self.documents = []
        temp_dir = tempfile.mkdtemp()
        
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个示例文档")
    
    def process_documents(self):
        """处理文档并创建向量存储"""
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_documents(splits, embeddings)
        self.retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}  # 获取最相关的2个文档块
        )
        
        print(f"文档已处理，共创建 {len(splits)} 个文档块")
    
    def method1_retrieval_qa_chain(self, query: str) -> str:
        """方法1: 使用检索QA链 - 最基本的检索增强问答"""
        print("\\n===== 方法1: 使用检索QA链 =====")
        
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",  # 将所有文档合并到提示中
            retriever=self.retriever,
            verbose=True
        )
        
        result = qa_chain({"query": query})
        return result["result"]
    
    def method2_retrieval_tool_for_agent(self, query: str) -> str:
        """方法2: 将检索器作为智能体工具 - 智能体可以主动决定何时搜索信息"""
        print("\\n===== 方法2: 将检索器作为智能体工具 =====")
        
        # 创建检索工具
        def search_docs(query: str) -> str:
            docs = self.retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 定义工具
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当你需要查询关于LangChain的信息时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        # 执行查询
        result = agent.run(query)
        return result
    
    def method3_contextual_agent(self, query: str) -> str:
        """方法3: 基于上下文的智能体 - 先检索相关文档，然后让智能体基于上下文回答"""
        print("\\n===== 方法3: 基于上下文的智能体 =====")
        
        # 首先获取相关文档
        docs = self.retriever.get_relevant_documents(query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建带上下文的工具
        python_tool = PythonREPLTool()
        
        tools = [
            Tool(
                name="Python执行器",
                func=python_tool.run,
                description="执行Python代码，适用于计算、数据处理等"
            )
        ]
        
        # 创建基于上下文的提示
        contextual_prompt = f"你有以下参考信息:\\n\\n{context}\\n\\n基于这些信息回答问题: {query}"
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        # 执行具有上下文的查询
        result = agent.run(contextual_prompt)
        return result

# 主执行函数
def main():
    print("=== LangChain 搜索与智能体集成示例 ===")
    
    # 创建示例
    example = SearchAgentIntegration()
    
    # 加载和处理文档
    example.load_sample_documents()
    example.process_documents()
    
    # 测试不同的集成方法
    queries = [
        "LangChain的核心特点是什么?",
        "解释RAG模式是如何工作的，并计算它的步骤数量",
        "智能体如何与外部工具交互?"
    ]
    
    for i, query in enumerate(queries):
        print(f"\\n\\n问题 {i+1}: {query}")
        
        # 根据问题类型选择不同的方法
        if i == 0:
            # 简单的事实查询 - 使用基本检索QA
            answer = example.method1_retrieval_qa_chain(query)
        elif i == 1:
            # 需要计算的查询 - 使用检索工具的智能体
            answer = example.method2_retrieval_tool_for_agent(query)
        else:
            # 需要更多上下文的复杂查询 - 使用上下文智能体
            answer = example.method3_contextual_agent(query)
        
        print(f"\\n回答: {answer}")
        print("\\n" + "-"*50)

if __name__ == "__main__":
    main()
"""

# Write the example code
await terminal.run("mkdir -p langchain_tutorial/code_examples/search_agent_integration")
editor.write("langchain_tutorial/code_examples/search_agent_integration/search_agent_methods.py", example_code)

# Create an advanced hybrid search agent example - Fixed indentation issue in the template string
hybrid_search_agent_code = """# LangChain 高级混合检索智能体示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor, create_react_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools.python.tool import PythonREPLTool
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.schema import Document
from langchain.prompts import PromptTemplate
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class HybridSearchTool(Tool):
    """混合搜索工具 - 结合关键词和向量搜索"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        # 创建基本检索器
        self.basic_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        
        # 创建上下文压缩检索器
        compressor = LLMChainExtractor.from_llm(llm)
        self.compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=self.basic_retriever
        )
        
        super().__init__(
            name="混合知识库搜索",
            func=self.search,
            description="当你需要查询文档中的特定信息时使用此工具，输入应为明确的问题或信息需求"
        )
    
    def search(self, query: str) -> str:
        """执行混合搜索"""
        # 获取向量检索结果
        vector_docs = self.basic_retriever.get_relevant_documents(query)
        
        # 使用LLM提取最相关的信息
        compressed_docs = self.compression_retriever.get_relevant_documents(query)
        
        # 合并并去重结果
        all_docs = []
        seen_content = set()
        
        for doc in vector_docs + compressed_docs:
            if doc.page_content not in seen_content:
                all_docs.append(doc)
                seen_content.add(doc.page_content)
        
        # 格式化结果
        if not all_docs:
            return "没有找到相关信息"
        
        return "\\n\\n".join([f"文档 {i+1}:\\n{doc.page_content}" for i, doc in enumerate(all_docs)])


class AdvancedSearchAgent:
    """高级搜索智能体 - 结合多种检索方法并使用对话记忆"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vectorstore = None
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent_executor = None
    
    def load_documents(self, file_paths: Optional[List[str]] = None):
        """加载文档"""
        self.documents = []
        
        # 如果没有提供文件路径，使用示例文档
        if not file_paths:
            temp_dir = tempfile.mkdtemp()
            file_paths = []
            
            # 创建示例文档
            example_texts = [
                """
                # 搜索系统与智能体集成的方法
                
                搜索系统可以通过多种方式与LangChain智能体集成:
                
                1. **作为工具提供**: 封装为工具API供智能体调用
                2. **预检索增强**: 在启动智能体前检索信息
                3. **混合架构**: 两种方法的结合，根据需要使用
                
                每种方法都有其优缺点，适用于不同场景。
                """,
                """
                # 检索增强生成(RAG)架构
                
                RAG架构是结合检索系统和LLM的有效方法，通常包含以下组件:
                
                - 文档加载器: 加载各种格式的文档
                - 文档转换器: 分割文档为可管理的块
                - 文本嵌入模型: 将文本转换为向量表示
                - 向量存储: 存储和索引嵌入向量
                - 检索器: 检索相关信息
                - 生成模型: 基于检索内容生成答案
                
                RAG过程包括索引、检索和生成三个阶段。
                """,
                """
                # 智能体工具类型
                
                LangChain提供多种工具类型可以与智能体结合:
                
                - 检索工具: 从文档中检索信息
                - API工具: 与外部服务交互
                - Python工具: 执行Python代码
                - 人类工具: 获取人类反馈
                - Shell工具: 执行系统命令
                
                智能体可以根据任务需求选择合适的工具。
                """,
                """
                # 上下文压缩技术
                
                上下文压缩技术可以优化检索结果:
                
                1. 嵌入过滤: 基于嵌入相似度过滤
                2. 语义过滤: 基于语义相关性过滤
                3. LLM抽取: 使用LLM提取最相关部分
                4. 重排序: 重新排序检索到的文档
                
                这些技术可以帮助减少噪声，提高检索质量。
                """
            ]
            
            for i, text in enumerate(example_texts):
                file_path = os.path.join(temp_dir, f"doc_{i}.txt")
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(text)
                file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            try:
                loader = TextLoader(file_path)
                self.documents.extend(loader.load())
            except Exception as e:
                print(f"加载文档 {file_path} 时出错: {str(e)}")
        
        print(f"已加载 {len(self.documents)} 个文档")
    
    def process_documents(self):
        """处理文档并创建向量存储"""
        if not self.documents:
            raise ValueError("请先加载文档")
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(splits, embeddings)
        
        print(f"文档已处理，共创建 {len(splits)} 个文档块")
    
    def setup_agent(self):
        """设置智能体"""
        if not self.vectorstore:
            raise ValueError("请先处理文档")
        
        # 创建混合搜索工具
        hybrid_search_tool = HybridSearchTool(self.vectorstore, self.llm)
        
        # 创建Python工具
        python_tool = PythonREPLTool()
        
        # 创建检索QA链作为独立工具
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever()
        )
        
        def qa_tool(query: str) -> str:
            """直接QA问答工具"""
            return qa_chain.run(query)
        
        # 定义工具集
        tools = [
            hybrid_search_tool,
            Tool(
                name="Python执行器",
                func=python_tool.run,
                description="执行Python代码，适用于计算、数据处理等"
            ),
            Tool(
                name="直接问答",
                func=qa_tool,
                description="当你需要直接回答问题而不进行推理时使用此工具"
            )
        ]
        
        # 创建带有记忆的智能体
        # 使用React提示创建智能体
        # Fixed the indentation in the template string by using triple quotes without indentation
        prompt_template = """你是一个高级搜索智能体，能够访问知识库并执行代码来回答问题。

{chat_history}

你可以使用以下工具:
{tools}

针对人类的问题，先考虑是否需要搜索信息或执行代码，然后使用合适的工具。
如果信息已在聊天历史中，可以直接回答而不需要重新搜索。

人类问题: {input}
让我们一步步思考:"""

        prompt = PromptTemplate.from_template(prompt_template)
        
        agent = create_react_agent(self.llm, tools, prompt)
        
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=5
        )
    
    def ask(self, query: str) -> str:
        """向智能体提问"""
        if not self.agent_executor:
            raise ValueError("请先设置智能体")
        
        result = self.agent_executor.run(query)
        return result


# 主执行函数
def main():
    print("=== LangChain 高级混合检索智能体示例 ===")
    
    # 创建高级搜索智能体
    agent = AdvancedSearchAgent()
    
    # 加载和处理文档
    agent.load_documents()
    agent.process_documents()
    
    # 设置智能体
    agent.setup_agent()
    
    # 测试智能体
    questions = [
        "智能体可以通过哪些方式与搜索系统集成?",
        "什么是RAG架构，它有哪些组件?",
        "基于之前的问题，计算RAG架构中的组件数量",
        "什么是上下文压缩技术，它如何改进检索结果?"
    ]
    
    for i, question in enumerate(questions):
        print(f"\\n\\n问题 {i+1}: {question}")
        answer = agent.ask(question)
        print(f"\\n回答: {answer}")
        print("\\n" + "-"*50)


if __name__ == "__main__":
    main()
"""

# Write the advanced hybrid search agent example
editor.write("langchain_tutorial/code_examples/search_agent_integration/hybrid_search_agent.py", hybrid_search_agent_code)

# Create a comprehensive summary document about search and agent integration
summary_content = """# LangChain 搜索与智能体集成技术总结

## 1. 集成方法概述

LangChain 框架提供了多种将搜索/检索功能与智能体集成的方法，主要包括以下几种模式：

### 1.1 检索工具模式

在这种模式下，检索系统被封装为智能体可以使用的工具。智能体可以根据需要决定何时调用检索工具来获取相关信息。

**优势：**
- 智能体可以决定何时使用检索功能
- 更灵活的交互模式
- 可以结合其他工具使用

**实现方式：**

SyntaxError: invalid character '。' (U+3002) (2485267529.py, line 100)

In [25]:
from metagpt.tools.libs.web_scraping import view_page_element_to_scrape
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import asyncio
from bs4 import BeautifulSoup
import os

# Setup tools
terminal = Terminal()
editor = Editor()

# Create directory for search and agent integration
await terminal.run("mkdir -p langchain_tutorial/docs/search_agent_integration")

# URLs for search and agent integration documentation
search_agent_urls = [
    ("search_tools", "https://python.langchain.com/docs/modules/agents/tools/"),
    ("search_toolkit", "https://python.langchain.com/docs/modules/agents/toolkits/retrieval"),
    ("vector_stores", "https://python.langchain.com/docs/integrations/vectorstores/"),
    ("retrievers", "https://python.langchain.com/docs/modules/data_connection/retrievers/"),
    ("agent_with_retrieval_tool", "https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db")
]

# Function to extract content
async def extract_search_agent_content(name, url):
    print(f"Fetching {name} documentation from: {url}")
    html_content = await view_page_element_to_scrape(
        url=url,
        requirement=f"Extract all information about how search/retrieval capabilities can be integrated with agents",
        keep_links=True
    )
    
    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find('main')
    
    content = ""
    if main_content:
        title = main_content.find('h1')
        if title:
            content += f"# {title.get_text(strip=True)}\n\n"
        
        for element in main_content.find_all(['h2', 'h3', 'h4', 'p', 'pre', 'ul', 'ol', 'li', 'div']):
            if element.name in ['h2', 'h3', 'h4']:
                level = int(element.name[1])
                content += f"{'#' * level} {element.get_text(strip=True)}\n\n"
            elif element.name == 'p':
                content += f"{element.get_text(strip=True)}\n\n"
            elif element.name == 'pre':
                code = element.find('code')
                if code:
                    language = "python" if "python" in code.get('class', [''])[0] else ""
                    content += f"```{language}\n{code.get_text()}\n```\n\n"
                else:
                    content += f"```\n{element.get_text()}\n```\n\n"
            elif element.name in ['ul', 'ol']:
                for li in element.find_all('li', recursive=False):
                    content += f"- {li.get_text(strip=True)}\n"
                content += "\n"
    
    return content

# Collect documentation on search and agent integration
for name, url in search_agent_urls:
    content = await extract_search_agent_content(name, url)
    file_path = f"langchain_tutorial/docs/search_agent_integration/{name}.md"
    editor.write(file_path, content + f"\n\nSource: {url}")

# Create a code example demonstrating search agent integration
# Fix: Replace Chinese periods with ASCII periods in text content
example_code = """# LangChain 搜索与智能体集成示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class SearchAgentIntegration:
    """搜索与智能体集成的不同方法示例"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vectorstore = None
        self.retriever = None
    
    def load_sample_documents(self):
        """加载示例文档"""
        # 创建示例文档
        texts = [
            """
            LangChain是一个用于构建基于语言模型的应用程序的框架.
            它的核心特点包括:
            1. 组件化: 提供模块化的接口和组件
            2. 可定制性: 可以灵活组合和调整不同组件
            3. 调试能力: 可以轻松追踪应用中的操作
            4. 端到端开发: 从原型到生产的完整解决方案
            """,
            """
            LangChain的检索增强生成(RAG)模式是将外部知识与LLM结合的强大方法.
            RAG工作流程包括:
            1. 索引阶段: 将文档转换为向量表示并存储
            2. 检索阶段: 查询相关文档片段
            3. 生成阶段: 使用检索到的上下文增强LLM的回答
            这种方法减少了幻觉，提高了回答准确性.
            """,
            """
            LangChain的智能体(Agents)系统允许LLM决定使用哪些工具来解决问题.
            智能体可以:
            1. 拆解复杂问题
            2. 选择并使用合适的工具
            3. 根据工具输出调整策略
            4. 迭代直到达成目标
            智能体与工具的结合使LLM能够与现实世界交互.
            """
        ]
        
        # 加载文档
        import tempfile
        self.documents = []
        temp_dir = tempfile.mkdtemp()
        
        for i, text in enumerate(texts):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个示例文档")
    
    def process_documents(self):
        """处理文档并创建向量存储"""
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_documents(splits, embeddings)
        self.retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}  # 获取最相关的2个文档块
        )
        
        print(f"文档已处理，共创建 {len(splits)} 个文档块")
    
    def method1_retrieval_qa_chain(self, query: str) -> str:
        """方法1: 使用检索QA链 - 最基本的检索增强问答"""
        print("\\n===== 方法1: 使用检索QA链 =====")
        
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",  # 将所有文档合并到提示中
            retriever=self.retriever,
            verbose=True
        )
        
        result = qa_chain({"query": query})
        return result["result"]
    
    def method2_retrieval_tool_for_agent(self, query: str) -> str:
        """方法2: 将检索器作为智能体工具 - 智能体可以主动决定何时搜索信息"""
        print("\\n===== 方法2: 将检索器作为智能体工具 =====")
        
        # 创建检索工具
        def search_docs(query: str) -> str:
            docs = self.retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 定义工具
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当你需要查询关于LangChain的信息时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        # 执行查询
        result = agent.run(query)
        return result
    
    def method3_contextual_agent(self, query: str) -> str:
        """方法3: 基于上下文的智能体 - 先检索相关文档，然后让智能体基于上下文回答"""
        print("\\n===== 方法3: 基于上下文的智能体 =====")
        
        # 首先获取相关文档
        docs = self.retriever.get_relevant_documents(query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建带上下文的工具
        python_tool = PythonREPLTool()
        
        tools = [
            Tool(
                name="Python执行器",
                func=python_tool.run,
                description="执行Python代码，适用于计算、数据处理等"
            )
        ]
        
        # 创建基于上下文的提示
        contextual_prompt = f"你有以下参考信息:\\n\\n{context}\\n\\n基于这些信息回答问题: {query}"
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True
        )
        
        # 执行具有上下文的查询
        result = agent.run(contextual_prompt)
        return result

# 主执行函数
def main():
    print("=== LangChain 搜索与智能体集成示例 ===")
    
    # 创建示例
    example = SearchAgentIntegration()
    
    # 加载和处理文档
    example.load_sample_documents()
    example.process_documents()
    
    # 测试不同的集成方法
    queries = [
        "LangChain的核心特点是什么?",
        "解释RAG模式是如何工作的，并计算它的步骤数量",
        "智能体如何与外部工具交互?"
    ]
    
    for i, query in enumerate(queries):
        print(f"\\n\\n问题 {i+1}: {query}")
        
        # 根据问题类型选择不同的方法
        if i == 0:
            # 简单的事实查询 - 使用基本检索QA
            answer = example.method1_retrieval_qa_chain(query)
        elif i == 1:
            # 需要计算的查询 - 使用检索工具的智能体
            answer = example.method2_retrieval_tool_for_agent(query)
        else:
            # 需要更多上下文的复杂查询 - 使用上下文智能体
            answer = example.method3_contextual_agent(query)
        
        print(f"\\n回答: {answer}")
        print("\\n" + "-"*50)

if __name__ == "__main__":
    main()
"""

# Write the example code
await terminal.run("mkdir -p langchain_tutorial/code_examples/search_agent_integration")
editor.write("langchain_tutorial/code_examples/search_agent_integration/search_agent_methods.py", example_code)

# Create an advanced hybrid search agent example - Fixed indentation issue in the template string
hybrid_search_agent_code = """# LangChain 高级混合检索智能体示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor, create_react_agent
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.tools.python.tool import PythonREPLTool
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.schema import Document
from langchain.prompts import PromptTemplate
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class HybridSearchTool(Tool):
    """混合搜索工具 - 结合关键词和向量搜索"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        # 创建基本检索器
        self.basic_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        
        # 创建上下文压缩检索器
        compressor = LLMChainExtractor.from_llm(llm)
        self.compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=self.basic_retriever
        )
        
        super().__init__(
            name="混合知识库搜索",
            func=self.search,
            description="当你需要查询文档中的特定信息时使用此工具，输入应为明确的问题或信息需求"
        )
    
    def search(self, query: str) -> str:
        """执行混合搜索"""
        # 获取向量检索结果
        vector_docs = self.basic_retriever.get_relevant_documents(query)
        
        # 使用LLM提取最相关的信息
        compressed_docs = self.compression_retriever.get_relevant_documents(query)
        
        # 合并并去重结果
        all_docs = []
        seen_content = set()
        
        for doc in vector_docs + compressed_docs:
            if doc.page_content not in seen_content:
                all_docs.append(doc)
                seen_content.add(doc.page_content)
        
        # 格式化结果
        if not all_docs:
            return "没有找到相关信息"
        
        return "\\n\\n".join([f"文档 {i+1}:\\n{doc.page_content}" for i, doc in enumerate(all_docs)])


class AdvancedSearchAgent:
    """高级搜索智能体 - 结合多种检索方法并使用对话记忆"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.documents = []
        self.vectorstore = None
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent_executor = None
    
    def load_documents(self, file_paths: Optional[List[str]] = None):
        """加载文档"""
        self.documents = []
        
        # 如果没有提供文件路径，使用示例文档
        if not file_paths:
            temp_dir = tempfile.mkdtemp()
            file_paths = []
            
            # 创建示例文档 - Fix: Replace Chinese punctuation with ASCII
            example_texts = [
                """
                # 搜索系统与智能体集成的方法
                
                搜索系统可以通过多种方式与LangChain智能体集成:
                
                1. **作为工具提供**: 封装为工具API供智能体调用
                2. **预检索增强**: 在启动智能体前检索信息
                3. **混合架构**: 两种方法的结合，根据需要使用
                
                每种方法都有其优缺点，适用于不同场景.
                """,
                """
                # 检索增强生成(RAG)架构
                
                RAG架构是结合检索系统和LLM的有效方法，通常包含以下组件:
                
                - 文档加载器: 加载各种格式的文档
                - 文档转换器: 分割文档为可管理的块
                - 文本嵌入模型: 将文本转换为向量表示
                - 向量存储: 存储和索引嵌入向量
                - 检索器: 检索相关信息
                - 生成模型: 基于检索内容生成答案
                
                RAG过程包括索引、检索和生成三个阶段.
                """,
                """
                # 智能体工具类型
                
                LangChain提供多种工具类型可以与智能体结合:
                
                - 检索工具: 从文档中检索信息
                - API工具: 与外部服务交互
                - Python工具: 执行Python代码
                - 人类工具: 获取人类反馈
                - Shell工具: 执行系统命令
                
                智能体可以根据任务需求选择合适的工具.
                """,
                """
                # 上下文压缩技术
                
                上下文压缩技术可以优化检索结果:
                
                1. 嵌入过滤: 基于嵌入相似度过滤
                2. 语义过滤: 基于语义相关性过滤
                3. LLM抽取: 使用LLM提取最相关部分
                4. 重排序: 重新排序检索到的文档
                
                这些技术可以帮助减少噪声，提高检索质量.
                """
            ]
            
            for i, text in enumerate(example_texts):
                file_path = os.path.join(temp_dir, f"doc_{i}.txt")
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(text)
                file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            try:
                loader = TextLoader(file_path)
                self.documents.extend(loader.load())
            except Exception as e:
                print(f"加载文档 {file_path} 时出错: {str(e)}")
        
        print(f"已加载 {len(self.documents)} 个文档")
    
    def process_documents(self):
        """处理文档并创建向量存储"""
        if not self.documents:
            raise ValueError("请先加载文档")
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(splits, embeddings)
        
        print(f"文档已处理，共创建 {len(splits)} 个文档块")
    
    def setup_agent(self):
        """设置智能体"""
        if not self.vectorstore:
            raise ValueError("请先处理文档")
        
        # 创建混合搜索工具
        hybrid_search_tool = HybridSearchTool(self.vectorstore, self.llm)
        
        # 创建Python工具
        python_tool = PythonREPLTool()
        
        # 创建检索QA链作为独立工具
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever()
        )
        
        def qa_tool(query: str) -> str:
            """直接QA问答工具"""
            return qa_chain.run(query)
        
        # 定义工具集
        tools = [
            hybrid_search_tool,
            Tool(
                name="Python执行器",
                func=python_tool.run,
                description="执行Python代码，适用于计算、数据处理等"
            ),
            Tool(
                name="直接问答",
                func=qa_tool,
                description="当你需要直接回答问题而不进行推理时使用此工具"
            )
        ]
        
        # 使用不带缩进的原始字符串来避免缩进问题
        prompt_template = """你是一个高级搜索智能体，能够访问知识库并执行代码来回答问题。

{chat_history}

你可以使用以下工具:
{tools}

针对人类的问题，先考虑是否需要搜索信息或执行代码，然后使用合适的工具。
如果信息已在聊天历史中，可以直接回答而不需要重新搜索。

人类问题: {input}
让我们一步步思考:"""

        prompt = PromptTemplate.from_template(prompt_template)
        
        agent = create_react_agent(self.llm, tools, prompt)
        
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=5
        )
    
    def ask(self, query: str) -> str:
        """向智能体提问"""
        if not self.agent_executor:
            raise ValueError("请先设置智能体")
        
        result = self.agent_executor.run(query)
        return result


# 主执行函数
def main():
    print("=== LangChain 高级混合检索智能体示例 ===")
    
    # 创建高级搜索智能体
    agent = AdvancedSearchAgent()
    
    # 加载和处理文档
    agent.load_documents()
    agent.process_documents()
    
    # 设置智能体
    agent.setup_agent()
    
    # 测试智能体
    questions = [
        "智能体可以通过哪些方式与搜索系统集成?",
        "什么是RAG架构，它有哪些组件?",
        "基于之前的问题，计算RAG架构中的组件数量",
        "什么是上下文压缩技术，它如何改进检索结果?"
    ]
    
    for i, question in enumerate(questions):
        print(f"\\n\\n问题 {i+1}: {question}")
        answer = agent.ask(question)
        print(f"\\n回答: {answer}")
        print("\\n" + "-"*50)


if __name__ == "__main__":
    main()
"""

# Write the advanced hybrid search agent example
editor.write("langchain_tutorial/code_examples/search_agent_integration/hybrid_search_agent.py", hybrid_search_agent_code)

# Create a comprehensive summary document about search and agent integration
summary_content = """# LangChain 搜索与智能体集成技术总结

## 1. 集成方法概述

LangChain 框架提供了多种将搜索/检索功能与智能体集成的方法，主要包括以下几种模式:

### 1.1 检索工具模式

在这种模式下，检索系统被封装为智能体可以使用的工具。智能体可以根据需要决定何时调用检索工具来获取相关信息。

**优势:**
- 智能体可以决定何时使用检索功能
- 更灵活的交互模式
- 可以结合其他工具使用

**实现方式:**

SyntaxError: invalid character '，' (U+FF0C) (4171685850.py, line 114)

In [26]:
I need to fix the encoding issues in our previous code examples and create additional examples for search-enhanced agents with LangChain. Let me focus on creating a new example that demonstrates different ways to integrate search capabilities with agents.

```python
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

# Set up tools
terminal = Terminal()
editor = Editor()

# Create directory for search-enhanced agent examples
async def create_search_agent_examples():
    print("Creating search-enhanced agent examples...")
    
    # Make sure directory exists
    await terminal.run("mkdir -p langchain_tutorial/code_examples/search_agent_integration")
    
    # Create a comprehensive example demonstrating different ways to integrate search with agents
    search_enhanced_agent_code = """# LangChain 搜索与智能体结合综合示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class SearchEnhancedAgent:
    """搜索增强智能体 - 展示搜索与智能体结合的不同方式"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.vectorstore = None
        self.documents = []
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
    
    def load_sample_documents(self):
        """加载示例文档"""
        print("正在加载示例文档...")
        
        # 创建示例文本
        documents_text = [
            """
            # LangChain搜索集成
            
            LangChain提供多种将搜索功能集成到智能体的方法:
            
            1. 检索工具（Retrieval Tools）: 将检索器包装为工具供智能体使用
            2. 上下文增强（Contextual Augmentation）: 在智能体处理前添加相关上下文
            3. 检索增强生成（RAG）: 结合检索和生成能力来提供更准确的回答
            4. 自主检索（Self-querying）: 让智能体自己决定如何查询知识库
            
            这些方法可以根据具体需求组合使用。
            """,
            """
            # 向量检索优化
            
            向量检索可以通过以下技术优化:
            
            1. 查询修改: 重写或扩展用户查询以提高检索质量
            2. 过滤: 使用元数据过滤相关文档
            3. 重排序: 使用额外标准对检索结果重新排序
            4. 压缩: 提取最相关的内容片段
            5. 混合检索: 结合语义和关键词搜索
            
            优化后的检索系统能够显著提高智能体的响应质量。
            """,
            """
            # 智能体工具选择
            
            智能体需要智能地选择合适的工具:
            
            1. 基于查询分析选择搜索或计算工具
            2. 考虑任务的信息需求和复杂度
            3. 在多个信息源之间平衡权衡
            4. 评估工具执行结果并决定后续行动
            
            有效的工具选择使智能体能够更高效地完成任务。
            """
        ]
        
        # 使用临时文件存储文档
        import tempfile
        self.documents = []
        temp_dir = tempfile.mkdtemp()
        
        # 写入和加载文档
        for i, text in enumerate(documents_text):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            
            # 加载文档
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个示例文档")
    
    def create_vectorstore(self):
        """处理文档并创建向量存储"""
        if not self.documents:
            raise ValueError("请先加载文档")
        
        print("正在创建向量存储...")
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_documents(splits, embeddings)
        
        print(f"向量存储已创建，共 {len(splits)} 个文本块")
        return self.vectorstore
    
    def method1_retrieval_qa_chain(self):
        """方法1: 基础检索QA - 使用检索QA链直接回答问题"""
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\n=== 方法1: 基础检索QA ===")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}
        )
        
        # 创建检索QA链
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=retriever,
            verbose=True
        )
        
        # 执行查询
        query = "LangChain提供哪些方法将搜索功能集成到智能体中?"
        result = qa_chain.invoke({"query": query})
        print(f"\n问题: {query}")
        print(f"回答: {result['result']}")
        
        return qa_chain
    
    def method2_retrieval_tool(self):
        """方法2: 检索工具 - 将检索器包装为智能体工具"""
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\n=== 方法2: 检索工具 ===")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}
        )
        
        # 创建检索工具
        def search_docs(query: str) -> str:
            docs = retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建工具列表
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当需要查询有关LangChain搜索集成或向量检索技术的信息时使用"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
        
        # 执行查询
        query = "向量检索可以通过哪些技术优化? 列出这些技术，并计算总数。"
        result = agent.invoke(query)
        print(f"\n问题: {query}")
        print(f"回答: {result}")
        
        return agent
    
    def method3_context_augmented_agent(self):
        """方法3: 上下文增强智能体 - 先检索相关文档作为上下文，再使用智能体处理"""
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\n=== 方法3: 上下文增强智能体 ===")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # 创建工具
        tools = [
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True
        )
        
        # 执行查询
        query = "智能体如何选择合适的工具?"
        
        # 先检索相关文档
        docs = retriever.get_relevant_documents(query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 添加上下文后处理
        augmented_query = f"基于以下信息回答问题。如果信息不足，可以使用自己的知识。\\n\\n信息: {context}\\n\\n问题: {query}"
        
        result = agent.invoke(augmented_query)
        print(f"\n问题: {query}")
        print(f"回答: {result}")
        
        return agent
    
    def method4_advanced_rag(self):
        """方法4: 高级RAG - 使用上下文压缩和混合检索增强智能体"""
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\n=== 方法4: 高级RAG ===")
        
        # 创建基本检索器
        base_retriever = self.vectorstore.as_retriever()
        
        # 创建压缩器
        compressor = LLMChainExtractor.from_llm(self.llm)
        
        # 创建压缩检索器
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=base_retriever
        )
        
        # 创建RAG提示模板
        rag_prompt = PromptTemplate.from_template(\"\"\"
        你是一个专业助手，请根据以下检索到的信息回答问题。
        如果检索的信息不足以回答问题，请说明信息不足，并尝试基于你的知识提供相关见解。
        
        检索到的信息:
        {context}
        
        问题: {question}
        
        请提供详细且结构化的回答:
        \"\"\")
        
        # 构建RAG链
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        rag_chain = (
            {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
            | rag_prompt
            | self.llm
        )
        
        # 执行查询
        query = "比较不同搜索集成方法的优缺点"
        result = rag_chain.invoke(query)
        print(f"\n问题: {query}")
        print(f"回答: {result.content if hasattr(result, 'content') else result}")
        
        return rag_chain
    
    def method5_query_transformation(self):
        """方法5: 查询转换 - 使用LLM改进查询后再检索"""
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\n=== 方法5: 查询转换 ===")
        
        # 创建查询改写链
        query_transformer_prompt = PromptTemplate.from_template(\"\"\"
        给定以下原始查询，请将其重写为更有效的向量数据库检索查询。
        添加可能相关的关键词，使查询更完整，但保持简洁。
        
        原始查询: {query}
        
        重写的查询:
        \"\"\")
        
        query_transformer_chain = query_transformer_prompt | self.llm
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(search_kwargs={"k": 2})
        
        # 执行查询
        original_query = "智能体使用工具的最佳实践"
        
        # 1. 改写查询
        transformed_query_result = query_transformer_chain.invoke({"query": original_query})
        transformed_query = transformed_query_result.content if hasattr(transformed_query_result, 'content') else str(transformed_query_result)
        
        print(f"\n原始查询: {original_query}")
        print(f"改写后查询: {transformed_query}")
        
        # 2. 使用改写后的查询进行检索
        docs = retriever.get_relevant_documents(transformed_query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 3. 使用LLM生成最终答案
        final_prompt = f"基于以下信息回答问题: '{original_query}'\\n\\n检索到的信息:\\n{context}"
        final_result = self.llm.invoke(final_prompt)
        
        print(f"回答: {final_result.content if hasattr(final_result, 'content') else final_result}")
        
        return {
            "original_query": original_query,
            "transformed_query": transformed_query,
            "retrieved_context": context,
            "final_result": final_result
        }
    
    def run_all_methods(self):
        """运行所有方法进行演示"""
        # 准备数据
        self.load_sample_documents()
        self.create_vectorstore()
        
        # 运行各种方法
        self.method1_retrieval_qa_chain()
        self.method2_retrieval_tool()
        self.method3_context_augmented_agent()
        self.method4_advanced_rag()
        self.method5_query_transformation()
        
        print("\n所有方法演示完成！")


# 多步骤搜索智能体示例
class MultiStepSearchAgent:
    """多步骤搜索智能体 - 实现迭代搜索和信息整合"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.vectorstore = None
    
    def setup_from_existing(self, vectorstore):
        """使用现有向量存储设置智能体"""
        self.vectorstore = vectorstore
    
    def create_search_planning_agent(self):
        """创建搜索规划智能体"""
        if not self.vectorstore:
            raise ValueError("请先设置向量存储")
        
        # 创建检索工具
        retriever = self.vectorstore.as_retriever(search_kwargs={"k": 2})
        
        def search_docs(query: str) -> str:
            docs = retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建工具列表
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当需要查询与任务相关的信息时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 使用自定义提示创建智能体
        planning_prompt = \"\"\"你是一个专业的研究助手，可以通过分解复杂问题并使用搜索工具找到相关信息来解决问题。
        
        对于复杂问题，请先思考如何将其分解为多个子问题，然后逐一解决这些子问题。
        检索到的信息可能不完整，需要进行多次搜索并整合信息。
        始终以结构化的方式整理你的思考过程和发现。
        
        当前可用工具：
        {tools}
        
        任务: {input}
        {agent_scratchpad}
        """
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            handle_parsing_errors=True,
            max_iterations=8
        )
        
        return agent
    
    def process_complex_query(self, query: str):
        """处理需要多步骤搜索的复杂查询"""
        if not self.vectorstore:
            raise ValueError("请先设置向量存储")
        
        print(f"\n=== 处理复杂查询: {query} ===")
        
        # 创建搜索规划智能体
        agent = self.create_search_planning_agent()
        
        # 执行多步骤搜索
        result = agent.invoke(f"请通过多步骤搜索解决以下问题，确保全面回答所有相关方面: {query}")
        
        print(f"\n最终回答: {result}")
        return result


# 运行示例
def main():
    print("=== LangChain 搜索与智能体结合综合示例 ===")
    
    # 创建并运行基本搜索增强智能体
    print("\n1. 运行基本搜索增强智能体示例")
    search_agent = SearchEnhancedAgent()
    search_agent.load_sample_documents()
    search_agent.create_vectorstore()
    
    # 运行各类方法
    search_agent.method1_retrieval_qa_chain()
    search_agent.method2_retrieval_tool()
    search_agent.method3_context_augmented_agent()
    search_agent.method4_advanced_rag()
    
    # 创建并运行多步骤搜索智能体
    print("\n2. 运行多步骤搜索智能体示例")
    multi_step_agent = MultiStepSearchAgent()
    multi_step_agent.setup_from_existing(search_agent.vectorstore)
    
    # 执行复杂查询
    complex_query = "比较不同搜索集成方法的优缺点，并推荐适合不同应用场景的最佳选择"
    multi_step_agent.process_complex_query(complex_query)
    
    print("\n=== 示例运行完成 ===")


if __name__ == "__main__":
    main()
"""
    
    # Create a real-world example of search-enhanced agent integration
    application_example_code = """# LangChain 搜索增强智能体实际应用示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.schema import SystemMessage
import tempfile
import json

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class ProductSupport:
    """产品支持智能体 - 结合知识库搜索的技术支持助手"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.2)
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.documents = []
        self.vectorstore = None
        self.agent = None
    
    def setup_knowledge_base(self):
        """设置示例产品知识库"""
        print("设置产品知识库...")
        
        # 创建示例知识库文档
        product_docs = [
            {
                "title": "产品概述",
                "content": """
                智能家居控制系统 v2.0
                
                我们的智能家居控制系统允许用户通过移动应用、语音或自动化规则控制家中的所有联网设备。
                系统支持灯光、恒温器、安全摄像头、门锁、窗帘和家电等设备。
                主要功能包括:
                - 远程控制: 从任何地方控制家中设备
                - 语音控制: 通过Alexa、Google Assistant集成
                - 自动化: 基于时间、位置或传感器触发的规则
                - 能源监控: 跟踪能源使用并提供优化建议
                - 安全警报: 异常活动通知
                """
            },
            {
                "title": "安装指南",
                "content": """
                智能家居控制系统安装指南
                
                1. 硬件要求:
                   - 中央控制枢纽(包含在包装中)
                   - WiFi网络(推荐5GHz)
                   - 兼容的智能设备
                   - iOS或Android手机(用于移动应用)
                
                2. 初始设置:
                   a. 将中央控制枢纽连接到电源
                   b. 将枢纽连接到您的WiFi网络
                   c. 在手机上下载并安装"智能家居"应用
                   d. 创建账户并按照应用中的指导进行操作
                
                3. 设备配对:
                   a. 在应用中点击"添加设备"
                   b. 选择设备类型
                   c. 将设备设置为配对模式(参考设备说明)
                   d. 按照应用中的特定说明完成配对
                
                常见问题:
                - 如果枢纽无法连接WiFi，请确保路由器固件已更新
                - 蓝灯闪烁表示配对模式，红灯表示连接错误
                """
            },
            {
                "title": "故障排除",
                "content": """
                智能家居系统故障排除指南
                
                1. 设备离线:
                   - 检查设备电源
                   - 确认WiFi连接正常
                   - 重启设备(断电10秒后重新连接)
                   - 重启中央控制枢纽
                   - 确保设备在WiFi范围内
                
                2. 应用程序无法连接:
                   - 检查手机网络连接
                   - 确认使用正确的账户凭据
                   - 清除应用缓存并重新启动
                   - 检查应用是否需要更新
                
                3. 自动化规则失效:
                   - 验证触发条件是否满足
                   - 检查所有相关设备是否在线
                   - 查看规则设置中的时区设置
                   - 删除并重新创建规则
                
                4. 系统响应缓慢:
                   - 检查internet连接速度
                   - 减少连接到枢纽的设备数量
                   - 重启中央控制枢纽
                   - 检查固件更新
                
                5. 错误代码:
                   - E01: 网络连接错误
                   - E02: 设备认证失败
                   - E03: 固件更新失败
                   - E04: 存储空间不足
                   - E05: 系统过载
                """
            }
        ]
        
        # 创建临时文件存储文档
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        for doc in product_docs:
            file_path = os.path.join(temp_dir, f"{doc['title'].replace(' ', '_')}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(f"# {doc['title']}\\n\\n{doc['content']}")
            file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        # 处理文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(splits, embeddings)
        
        print(f"知识库设置完成，包含 {len(splits)} 个文本块")
    
    def setup_agent(self):
        """设置产品支持智能体"""
        if not self.vectorstore:
            raise ValueError("请先设置知识库")
        
        print("设置产品支持智能体...")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # 创建上下文压缩检索器
        compressor = LLMChainExtractor.from_llm(self.llm)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=retriever
        )
        
        # 创建检索工具
        def search_product_knowledge(query: str) -> str:
            """搜索产品知识库"""
            docs = compression_retriever.get_relevant_documents(query)
            if not docs:
                return "知识库中没有找到相关信息。"
            return "\\n\\n".join([f"来源: {doc.metadata.get('source', '未知')}\\n内容: {doc.page_content}" for doc in docs])
        
        # 创建故障诊断工具
        def diagnose_problem(description: str) -> str:
            """诊断产品问题"""
            # 创建诊断链
            diagnosis_prompt = PromptTemplate(
                template="你是智能家居系统专家。请分析以下问题描述并提供可能的原因和解决步骤：\\n\\n{problem_description}",
                input_variables=["problem_description"]
            )
            diagnosis_chain = LLMChain(llm=self.llm, prompt=diagnosis_prompt)
            
            # 执行诊断
            result = diagnosis_chain.run(description)
            return result
        
        # 定义工具
        tools = [
            Tool(
                name="产品知识库",
                func=search_product_knowledge,
                description="当需要查询产品信息、安装指南或常见问题时使用"
            ),
            Tool(
                name="故障诊断",
                func=diagnose_problem,
                description="当用户描述具体问题需要诊断时使用"
            ),
            PythonREPLTool()
        ]
        
        # 创建产品支持智能体
        system_message = SystemMessage(
            content=\"\"\"你是智能家居控制系统的客户支持专家。你的任务是帮助用户解决与产品相关的问题。
            
            遵循以下指南:
            1. 首先理解用户的问题
            2. 使用产品知识库工具查询相关信息
            3. 如果是具体的技术问题，使用故障诊断工具
            4. 以友好、专业的方式回应用户
            5. 给出清晰、分步骤的解决方案
            6. 如果知识库中没有相关信息，承认这一点并提供一般性建议
            
            确保回答准确、实用且易于理解。
            \"\"\"
        )
        
        prompt = ChatPromptTemplate.from_messages([
            system_message,
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        self.agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            handle_parsing_errors=True,
            agent_kwargs={"system_message": system_message}
        )
        
        print("产品支持智能体设置完成")
    
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 413)

In [28]:
[reflection on previous impl]
In my previous implementation, I was trying to create a code example that demonstrates different ways to integrate search capabilities with LangChain agents. The error is an indentation error at line 413, specifically at the `agent = initialize_agent(` line in the `ProductSupport.setup_agent` method. This indicates that there's an inconsistent indentation level in the code.

Looking more carefully at the code structure, I see that the indentation for the `setup_agent` method is inconsistent with the rest of the class methods. This is especially problematic in Python where indentation is significant for code structure.

Additionally, the code is quite long and was cut off in the middle of the `ProductSupport` class implementation, without showing the complete methods and their corresponding indentation levels. I need to ensure consistent indentation throughout the class and complete any unfinished methods.

[improved impl]:
```python
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Set up tools
terminal = Terminal()
editor = Editor()

# Create directory for search-enhanced agent examples
async def create_search_agent_examples():
    print("Creating search-enhanced agent examples...")
    
    # Make sure directory exists
    await terminal.run("mkdir -p langchain_tutorial/code_examples/search_agent_integration")
    
    # Create a comprehensive example demonstrating different ways to integrate search with agents
    search_enhanced_agent_code = """# LangChain 搜索与智能体结合综合示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class SearchEnhancedAgent:
    \"\"\"搜索增强智能体 - 展示搜索与智能体结合的不同方式\"\"\"
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.vectorstore = None
        self.documents = []
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
    
    def load_sample_documents(self):
        \"\"\"加载示例文档\"\"\"
        print("正在加载示例文档...")
        
        # 创建示例文本
        documents_text = [
            \"\"\"
            # LangChain搜索集成
            
            LangChain提供多种将搜索功能集成到智能体的方法:
            
            1. 检索工具（Retrieval Tools）: 将检索器包装为工具供智能体使用
            2. 上下文增强（Contextual Augmentation）: 在智能体处理前添加相关上下文
            3. 检索增强生成（RAG）: 结合检索和生成能力来提供更准确的回答
            4. 自主检索（Self-querying）: 让智能体自己决定如何查询知识库
            
            这些方法可以根据具体需求组合使用。
            \"\"\",
            \"\"\"
            # 向量检索优化
            
            向量检索可以通过以下技术优化:
            
            1. 查询修改: 重写或扩展用户查询以提高检索质量
            2. 过滤: 使用元数据过滤相关文档
            3. 重排序: 使用额外标准对检索结果重新排序
            4. 压缩: 提取最相关的内容片段
            5. 混合检索: 结合语义和关键词搜索
            
            优化后的检索系统能够显著提高智能体的响应质量。
            \"\"\",
            \"\"\"
            # 智能体工具选择
            
            智能体需要智能地选择合适的工具:
            
            1. 基于查询分析选择搜索或计算工具
            2. 考虑任务的信息需求和复杂度
            3. 在多个信息源之间平衡权衡
            4. 评估工具执行结果并决定后续行动
            
            有效的工具选择使智能体能够更高效地完成任务。
            \"\"\"
        ]
        
        # 使用临时文件存储文档
        import tempfile
        self.documents = []
        temp_dir = tempfile.mkdtemp()
        
        # 写入和加载文档
        for i, text in enumerate(documents_text):
            file_path = os.path.join(temp_dir, f"doc_{i}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)
            
            # 加载文档
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        print(f"已加载 {len(self.documents)} 个示例文档")
    
    def create_vectorstore(self):
        \"\"\"处理文档并创建向量存储\"\"\"
        if not self.documents:
            raise ValueError("请先加载文档")
        
        print("正在创建向量存储...")
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = Chroma.from_documents(splits, embeddings)
        
        print(f"向量存储已创建，共 {len(splits)} 个文本块")
        return self.vectorstore
    
    def method1_retrieval_qa_chain(self):
        \"\"\"方法1: 基础检索QA - 使用检索QA链直接回答问题\"\"\"
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\\n=== 方法1: 基础检索QA ===")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}
        )
        
        # 创建检索QA链
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=retriever,
            verbose=True
        )
        
        # 执行查询
        query = "LangChain提供哪些方法将搜索功能集成到智能体中?"
        result = qa_chain.invoke({"query": query})
        print(f"\\n问题: {query}")
        print(f"回答: {result['result']}")
        
        return qa_chain
    
    def method2_retrieval_tool(self):
        \"\"\"方法2: 检索工具 - 将检索器包装为智能体工具\"\"\"
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\\n=== 方法2: 检索工具 ===")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 2}
        )
        
        # 创建检索工具
        def search_docs(query: str) -> str:
            docs = retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建工具列表
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当需要查询有关LangChain搜索集成或向量检索技术的信息时使用"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True
        )
        
        # 执行查询
        query = "向量检索可以通过哪些技术优化? 列出这些技术，并计算总数。"
        result = agent.invoke(query)
        print(f"\\n问题: {query}")
        print(f"回答: {result}")
        
        return agent
    
    def method3_context_augmented_agent(self):
        \"\"\"方法3: 上下文增强智能体 - 先检索相关文档作为上下文，再使用智能体处理\"\"\"
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\\n=== 方法3: 上下文增强智能体 ===")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # 创建工具
        tools = [
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True
        )
        
        # 执行查询
        query = "智能体如何选择合适的工具?"
        
        # 先检索相关文档
        docs = retriever.get_relevant_documents(query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 添加上下文后处理
        augmented_query = f"基于以下信息回答问题。如果信息不足，可以使用自己的知识。\\n\\n信息: {context}\\n\\n问题: {query}"
        
        result = agent.invoke(augmented_query)
        print(f"\\n问题: {query}")
        print(f"回答: {result}")
        
        return agent
    
    def method4_advanced_rag(self):
        \"\"\"方法4: 高级RAG - 使用上下文压缩和混合检索增强智能体\"\"\"
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\\n=== 方法4: 高级RAG ===")
        
        # 创建基本检索器
        base_retriever = self.vectorstore.as_retriever()
        
        # 创建压缩器
        compressor = LLMChainExtractor.from_llm(self.llm)
        
        # 创建压缩检索器
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=base_retriever
        )
        
        # 创建RAG提示模板
        rag_prompt = PromptTemplate.from_template(\"\"\"
        你是一个专业助手，请根据以下检索到的信息回答问题。
        如果检索的信息不足以回答问题，请说明信息不足，并尝试基于你的知识提供相关见解。
        
        检索到的信息:
        {context}
        
        问题: {question}
        
        请提供详细且结构化的回答:
        \"\"\")
        
        # 构建RAG链
        def format_docs(docs):
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        rag_chain = (
            {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
            | rag_prompt
            | self.llm
        )
        
        # 执行查询
        query = "比较不同搜索集成方法的优缺点"
        result = rag_chain.invoke(query)
        print(f"\\n问题: {query}")
        print(f"回答: {result.content if hasattr(result, 'content') else result}")
        
        return rag_chain
    
    def method5_query_transformation(self):
        \"\"\"方法5: 查询转换 - 使用LLM改进查询后再检索\"\"\"
        if not self.vectorstore:
            raise ValueError("请先创建向量存储")
        
        print("\\n=== 方法5: 查询转换 ===")
        
        # 创建查询改写链
        query_transformer_prompt = PromptTemplate.from_template(\"\"\"
        给定以下原始查询，请将其重写为更有效的向量数据库检索查询。
        添加可能相关的关键词，使查询更完整，但保持简洁。
        
        原始查询: {query}
        
        重写的查询:
        \"\"\")
        
        query_transformer_chain = query_transformer_prompt | self.llm
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(search_kwargs={"k": 2})
        
        # 执行查询
        original_query = "智能体使用工具的最佳实践"
        
        # 1. 改写查询
        transformed_query_result = query_transformer_chain.invoke({"query": original_query})
        transformed_query = transformed_query_result.content if hasattr(transformed_query_result, 'content') else str(transformed_query_result)
        
        print(f"\\n原始查询: {original_query}")
        print(f"改写后查询: {transformed_query}")
        
        # 2. 使用改写后的查询进行检索
        docs = retriever.get_relevant_documents(transformed_query)
        context = "\\n\\n".join(doc.page_content for doc in docs)
        
        # 3. 使用LLM生成最终答案
        final_prompt = f"基于以下信息回答问题: '{original_query}'\\n\\n检索到的信息:\\n{context}"
        final_result = self.llm.invoke(final_prompt)
        
        print(f"回答: {final_result.content if hasattr(final_result, 'content') else final_result}")
        
        return {
            "original_query": original_query,
            "transformed_query": transformed_query,
            "retrieved_context": context,
            "final_result": final_result
        }
    
    def run_all_methods(self):
        \"\"\"运行所有方法进行演示\"\"\"
        # 准备数据
        self.load_sample_documents()
        self.create_vectorstore()
        
        # 运行各种方法
        self.method1_retrieval_qa_chain()
        self.method2_retrieval_tool()
        self.method3_context_augmented_agent()
        self.method4_advanced_rag()
        self.method5_query_transformation()
        
        print("\\n所有方法演示完成！")


# 多步骤搜索智能体示例
class MultiStepSearchAgent:
    \"\"\"多步骤搜索智能体 - 实现迭代搜索和信息整合\"\"\"
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.vectorstore = None
    
    def setup_from_existing(self, vectorstore):
        \"\"\"使用现有向量存储设置智能体\"\"\"
        self.vectorstore = vectorstore
    
    def create_search_planning_agent(self):
        \"\"\"创建搜索规划智能体\"\"\"
        if not self.vectorstore:
            raise ValueError("请先设置向量存储")
        
        # 创建检索工具
        retriever = self.vectorstore.as_retriever(search_kwargs={"k": 2})
        
        def search_docs(query: str) -> str:
            docs = retriever.get_relevant_documents(query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建工具列表
        tools = [
            Tool(
                name="文档搜索",
                func=search_docs,
                description="当需要查询与任务相关的信息时使用此工具"
            ),
            PythonREPLTool()
        ]
        
        # 创建智能体
        agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=self.memory,
            handle_parsing_errors=True,
            max_iterations=8
        )
        
        return agent
    
    def process_complex_query(self, query: str):
        \"\"\"处理需要多步骤搜索的复杂查询\"\"\"
        if not self.vectorstore:
            raise ValueError("请先设置向量存储")
        
        print(f"\\n=== 处理复杂查询: {query} ===")
        
        # 创建搜索规划智能体
        agent = self.create_search_planning_agent()
        
        # 执行多步骤搜索
        result = agent.invoke(f"请通过多步骤搜索解决以下问题，确保全面回答所有相关方面: {query}")
        
        print(f"\\n最终回答: {result}")
        return result


# 运行示例
def main():
    print("=== LangChain 搜索与智能体结合综合示例 ===")
    
    # 创建并运行基本搜索增强智能体
    print("\\n1. 运行基本搜索增强智能体示例")
    search_agent = SearchEnhancedAgent()
    search_agent.load_sample_documents()
    search_agent.create_vectorstore()
    
    # 运行各类方法
    search_agent.method1_retrieval_qa_chain()
    search_agent.method2_retrieval_tool()
    search_agent.method3_context_augmented_agent()
    search_agent.method4_advanced_rag()
    
    # 创建并运行多步骤搜索智能体
    print("\\n2. 运行多步骤搜索智能体示例")
    multi_step_agent = MultiStepSearchAgent()
    multi_step_agent.setup_from_existing(search_agent.vectorstore)
    
    # 执行复杂查询
    complex_query = "比较不同搜索集成方法的优缺点，并推荐适合不同应用场景的最佳选择"
    multi_step_agent.process_complex_query(complex_query)
    
    print("\\n=== 示例运行完成 ===")


if __name__ == "__main__":
    main()
"""
    
    # Create a real-world example of search-enhanced agent integration
    application_example_code = """# LangChain 搜索增强智能体实际应用示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.schema import SystemMessage
import tempfile
import json

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class ProductSupport:
    \"\"\"产品支持智能体 - 结合知识库搜索的技术支持助手\"\"\"
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.2)
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.documents = []
        self.vectorstore = None
        self.agent = None
    
    def setup_knowledge_base(self):
        \"\"\"设置示例产品知识库\"\"\"
        print("设置产品知识库...")
        
        # 创建示例知识库文档
        product_docs = [
            {
                "title": "产品概述",
                "content": \"\"\"
                智能家居控制系统 v2.0
                
                我们的智能家居控制系统允许用户通过移动应用、语音或自动化规则控制家中的所有联网设备。
                系统支持灯光、恒温器、安全摄像头、门锁、窗帘和家电等设备。
                主要功能包括:
                - 远程控制: 从任何地方控制家中设备
                - 语音控制: 通过Alexa、Google Assistant集成
                - 自动化: 基于时间、位置或传感器触发的规则
                - 能源监控: 跟踪能源使用并提供优化建议
                - 安全警报: 异常活动通知
                \"\"\"
            },
            {
                "title": "安装指南",
                "content": \"\"\"
                智能家居控制系统安装指南
                
                1. 硬件要求:
                   - 中央控制枢纽(包含在包装中)
                   - WiFi网络(推荐5GHz)
                   - 兼容的智能设备
                   - iOS或Android手机(用于移动应用)
                
                2. 初始设置:
                   a. 将中央控制枢纽连接到电源
                   b. 将枢纽连接到您的WiFi网络
                   c. 在手机上下载并安装"智能家居"应用
                   d. 创建账户并按照应用中的指导进行操作
                
                3. 设备配对:
                   a. 在应用中点击"添加设备"
                   b. 选择设备类型
                   c. 将设备设置为配对模式(参考设备说明)
                   d. 按照应用中的特定说明完成配对
                
                常见问题:
                - 如果枢纽无法连接WiFi，请确保路由器固件已更新
                - 蓝灯闪烁表示配对模式，红灯表示连接错误
                \"\"\"
            },
            {
                "title": "故障排除",
                "content": \"\"\"
                智能家居系统故障排除指南
                
                1. 设备离线:
                   - 检查设备电源
                   - 确认WiFi连接正常
                   - 重启设备(断电10秒后重新连接)
                   - 重启中央控制枢纽
                   - 确保设备在WiFi范围内
                
                2. 应用程序无法连接:
                   - 检查手机网络连接
                   - 确认使用正确的账户凭据
                   - 清除应用缓存并重新启动
                   - 检查应用是否需要更新
                
                3. 自动化规则失效:
                   - 验证触发条件是否满足
                   - 检查所有相关设备是否在线
                   - 查看规则设置中的时区设置
                   - 删除并重新创建规则
                
                4. 系统响应缓慢:
                   - 检查internet连接速度
                   - 减少连接到枢纽的设备数量
                   - 重启中央控制枢纽
                   - 检查固件更新
                
                5. 错误代码:
                   - E01: 网络连接错误
                   - E02: 设备认证失败
                   - E03: 固件更新失败
                   - E04: 存储空间不足
                   - E05: 系统过载
                \"\"\"
            }
        ]
        
        # 创建临时文件存储文档
        temp_dir = tempfile.mkdtemp()
        file_paths = []
        
        for doc in product_docs:
            file_path = os.path.join(temp_dir, f"{doc['title'].replace(' ', '_')}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(f"# {doc['title']}\\n\\n{doc['content']}")
            file_paths.append(file_path)
        
        # 加载文档
        for file_path in file_paths:
            loader = TextLoader(file_path)
            self.documents.extend(loader.load())
        
        # 处理文档
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        splits = text_splitter.split_documents(self.documents)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(splits, embeddings)
        
        print(f"知识库设置完成，包含 {len(splits)} 个文本块")
    
    def setup_agent(self):
        \"\"\"设置产品支持智能体\"\"\"
        if not self.vectorstore:
            raise ValueError("请先设置知识库")
        
        print("设置产品支持智能体...")
        
        # 创建检索器
        retriever = self.vectorstore.as_retriever(
            search_kwargs={"k": 3}
        )
        
        # 创建上下文压缩检索器
        compressor = LLMChainExtractor.from_llm(self.llm)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=retriever
        )
        
        # 创建检索工具
        def search_product_knowledge(query: str) -> str:
            \"\"\"搜索产品知识库\"\"\"
            docs = compression_retriever.get_relevant_documents(query)
            if not docs:
                return "知识库中没有找到相关信息。"
            return "\\n\\n".join([f"来源: {doc.metadata.get('source', '未知')}\\n内容: {doc.page_content}" for doc in docs])
        
        # 创建故障诊断工具
        def diagnose_problem(description: str) -> str:
            \"\"\"诊断产品问题\"\"\"
            # 创建诊断链
            diagnosis_prompt = PromptTemplate(
                template="你是智能家居系统专家。请分析以下问题描述并提供可能的原因和解决步骤：\\n\\n{problem_description}",
                input_variables=["problem_description"]
            )
            diagnosis_chain = LLMChain(llm=self.llm, prompt=diagnosis_prompt)
            
            # 执行诊断
            result = diagnosis_chain.run(description)
            return result
        
        # 定义工具
        tools = [
            Tool(
                name="产品知识库",
                func=search_product_knowledge,
                description="当需要查询产品信息、安装指南或常见问题时使用"
            ),
            Tool(
                name="故障诊断",
                func=diagnose_problem,
                description="当用户描述具体问题需要诊断时使用"
            ),
            PythonREPLTool()
        ]
        
        # 创建产品支持智能体
        system_message = SystemMessage(
            content=\"\"\"你是智能家居控制系统的客户支持专家。你的任务是帮助用户解决与产品相关的问题。
            
            遵循以下指南:
            1. 首先理解用户的问题
            2. 使用产品知识库工具查询相关信息
            3. 如果是具体的技术问题，使用故障诊断工具
            4. 以友好、专业的方式回应用户
            5. 给出清晰、分步骤的解决方案
            6. 如果知识库中没有相关信息，承认这一点并提供一般性建议
            
            确保回答准确、实用且易于理解。
            \"\"\"
        )
        
        prompt = ChatPromptTemplate.from_messages([
            system_message,
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        # 创建智能体
        self.agent = initialize_agent(
            tools,
            self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_

SyntaxError: unterminated string literal (detected at line 2) (2088735988.py, line 2)

In [29]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor

# Setup tools
terminal = Terminal()
editor = Editor()

async def create_search_agent_examples():
    """Create search-agent integration examples for LangChain"""
    
    # Create directory for examples
    await terminal.run("mkdir -p langchain_tutorial/code_examples/search_agent_integration")
    
    # Create basic search agent example
    basic_search_agent = """# LangChain 搜索与智能体集成基础示例

import os
from typing import List, Dict, Any
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

def create_demo_knowledge_base():
    """创建示例知识库"""
    print("创建示例知识库...")
    
    # 创建临时目录存储示例文档
    temp_dir = tempfile.mkdtemp()
    
    # 示例文档内容
    documents = [
        {
            "title": "搜索与智能体集成方法",
            "content": """
            LangChain提供多种将搜索功能集成到智能体的方法:
            
            1. 检索工具（Retrieval Tools）: 将检索器包装为工具供智能体使用
            2. 上下文增强（Contextual Augmentation）: 在智能体处理前添加相关上下文
            3. 检索增强生成（RAG）: 结合检索和生成能力来提供更准确的回答
            4. 自主检索（Self-querying）: 让智能体自己决定如何查询知识库
            
            这些方法可以根据具体需求组合使用。
            """
        },
        {
            "title": "向量检索优化",
            "content": """
            向量检索可以通过以下技术优化:
            
            1. 查询修改: 重写或扩展用户查询以提高检索质量
            2. 过滤: 使用元数据过滤相关文档
            3. 重排序: 使用额外标准对检索结果重新排序
            4. 压缩: 提取最相关的内容片段
            5. 混合检索: 结合语义和关键词搜索
            
            优化后的检索系统能够显著提高智能体的响应质量。
            """
        },
        {
            "title": "智能体工具选择",
            "content": """
            智能体需要智能地选择合适的工具:
            
            1. 基于查询分析选择搜索或计算工具
            2. 考虑任务的信息需求和复杂度
            3. 在多个信息源之间平衡权衡
            4. 评估工具执行结果并决定后续行动
            
            有效的工具选择使智能体能够更高效地完成任务。
            """
        }
    ]
    
    # 写入文档到临时文件并加载
    all_docs = []
    for i, doc in enumerate(documents):
        file_path = os.path.join(temp_dir, f"{i}_{doc['title'].replace(' ', '_')}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(f"# {doc['title']}\\n\\n{doc['content']}")
        
        # 加载文档
        loader = TextLoader(file_path)
        all_docs.extend(loader.load())
    
    # 分割文档
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(all_docs)
    
    # 创建向量存储
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(splits, embeddings)
    
    print(f"知识库创建完成，包含 {len(splits)} 个文本块")
    return vectorstore

def method1_retrieval_qa_direct():
    """方法1: 直接检索 - 使用检索QA链直接回答问题"""
    print("\\n=== 方法1: 直接检索 ===")
    
    # 创建知识库
    vectorstore = create_demo_knowledge_base()
    retriever = vectorstore.as_retriever()
    
    # 创建检索QA链
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(),
        chain_type="stuff",
        retriever=retriever,
        verbose=True
    )
    
    # 执行查询
    query = "LangChain中的搜索集成方法有哪些?"
    result = qa_chain.invoke({"query": query})
    print(f"\\n问题: {query}")
    print(f"回答: {result['result']}")
    
    return vectorstore, qa_chain

def method2_agent_with_retrieval_tool(vectorstore):
    """方法2: 检索工具 - 将检索器封装为智能体工具"""
    print("\\n=== 方法2: 检索工具 ===")
    
    # 创建检索工具
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    
    def search_knowledge_base(query: str) -> str:
        """搜索知识库工具"""
        docs = retriever.get_relevant_documents(query)
        return "\\n\\n".join(doc.page_content for doc in docs)
    
    # 创建工具列表
    tools = [
        Tool(
            name="知识库搜索",
            func=search_knowledge_base,
            description="当需要查询关于LangChain搜索方法或向量检索技术的信息时使用"
        ),
        PythonREPLTool()
    ]
    
    # 创建智能体
    agent = initialize_agent(
        tools,
        ChatOpenAI(temperature=0),
        agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )
    
    # 执行查询
    query = "向量检索有哪些优化技术? 计算它们的数量。"
    result = agent.invoke(query)
    print(f"\\n问题: {query}")
    print(f"回答: {result}")
    
    return agent

def method3_context_enhanced_agent(vectorstore):
    """方法3: 上下文增强 - 在处理前检索相关信息并增强上下文"""
    print("\\n=== 方法3: 上下文增强 ===")
    
    # 创建检索器
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    
    # 创建智能体工具
    tools = [PythonREPLTool()]
    
    # 创建智能体
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    agent = initialize_agent(
        tools,
        ChatOpenAI(temperature=0),
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True
    )
    
    # 先检索相关文档
    query = "智能体如何选择合适的工具?"
    docs = retriever.get_relevant_documents(query)
    context = "\\n\\n".join(doc.page_content for doc in docs)
    
    # 增强查询
    enhanced_query = f"基于以下信息回答问题，如果需要可以使用Python代码演示：\\n\\n信息：{context}\\n\\n问题：{query}"
    
    # 执行查询
    result = agent.invoke(enhanced_query)
    print(f"\\n问题: {query}")
    print(f"回答: {result}")
    
    return agent

def main():
    print("=== LangChain 搜索与智能体集成示例 ===")
    
    # 方法1: 直接检索QA
    vectorstore, qa_chain = method1_retrieval_qa_direct()
    
    # 方法2: 检索作为智能体工具
    agent_with_tool = method2_agent_with_retrieval_tool(vectorstore)
    
    # 方法3: 上下文增强智能体
    contextual_agent = method3_context_enhanced_agent(vectorstore)
    
    print("\\n=== 所有示例执行完成 ===")

if __name__ == "__main__":
    main()
"""
    
    # Create advanced search agent example
    advanced_search_agent = """# LangChain 高级搜索智能体示例

import os
from typing import List, Dict, Any, Optional
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents import AgentType, initialize_agent, Tool, create_react_agent, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.tools.python.tool import PythonREPLTool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import SystemMessage
import tempfile

# 确保设置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"

class AdvancedSearchAgent:
    """高级搜索智能体 - 结合多种搜索增强技术的智能体"""
    
    def __init__(self):
        self.llm = ChatOpenAI(temperature=0)
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.vectorstore = None
        self.agent = None
    
    def setup_knowledge_base(self):
        """设置示例知识库"""
        print("设置知识库...")
        
        # 创建临时目录存储示例文档
        temp_dir = tempfile.mkdtemp()
        
        # 示例文档内容
        documents = [
            {
                "title": "搜索集成架构",
                "content": """
                搜索与智能体集成的三种主要架构:
                
                1. 工具型集成: 将搜索/检索系统作为工具提供给智能体使用
                   - 智能体可以决定何时使用搜索
                   - 更灵活但可能需要更多提示工程
                   - 适合复杂任务和开放域问答
                
                2. 预检索增强: 在智能体处理前先进行检索并作为上下文提供
                   - 智能体始终能看到相关信息
                   - 减少不必要的API调用
                   - 适合知识密集型任务
                
                3. 混合架构: 结合以上两种方法
                   - 提供基础上下文，同时允许主动搜索
                   - 更全面但复杂度更高
                   - 适合多轮对话和复杂推理任务
                """
            },
            {
                "title": "高级检索技术",
                "content": """
                提高检索质量的高级技术:
                
                1. 查询重构与扩展
                   - 使用LLM重写用户查询
                   - 添加相关关键词扩展查询
                   - 分解复杂查询为多个简单查询
                
                2. 上下文压缩
                   - 提取检索文档中最相关的部分
                   - 减少上下文长度，集中于关键信息
                   - 使用LLM或启发式方法进行筛选
                
                3. 混合检索
                   - 结合多种检索策略(语义、关键词等)
                   - 整合多个数据源的结果
                   - 使用加权方法排序最终结果
                """
            },
            {
                "title": "行业应用场景",
                "content": """
                搜索增强智能体的行业应用:
                
                1. 客户支持
                   - 检索产品文档回答用户问题
                   - 访问内部知识库解决技术问题
                   - 提供个性化的故障排除建议
                
                2. 法律研究
                   - 搜索相关法规和判例
                   - 分析法律文件并提供见解
                   - 辅助起草法律文档
                
                3. 医疗咨询
                   - 检索医学文献和临床指南
                   - 提供基于证据的治疗建议
                   - 支持医生的诊断决策
                
                4. 教育辅助
                   - 个性化学习材料推荐
                   - 回答学生的专业领域问题
                   - 生成定制的练习和测试
                """
            }
        ]
        
        # 写入文档到临时文件并加载
        all_docs = []
        for i, doc in enumerate(documents):
            file_path = os.path.join(temp_dir, f"{i}_{doc['title'].replace(' ', '_')}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(f"# {doc['title']}\\n\\n{doc['content']}")
            
            # 加载文档
            loader = TextLoader(file_path)
            all_docs.extend(loader.load())
        
        # 分割文档
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        splits = text_splitter.split_documents(all_docs)
        
        # 创建向量存储
        embeddings = OpenAIEmbeddings()
        self.vectorstore = FAISS.from_documents(splits, embeddings)
        
        print(f"知识库设置完成，包含 {len(splits)} 个文本块")
    
    def setup_advanced_search_tools(self):
        """设置高级搜索工具"""
        if not self.vectorstore:
            raise ValueError("请先设置知识库")
        
        # 基本检索器
        base_retriever = self.vectorstore.as_retriever(search_kwargs={"k": 2})
        
        # 1. 创建上下文压缩检索器
        compressor = LLMChainExtractor.from_llm(self.llm)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=base_retriever
        )
        
        # 2. 创建查询转换器
        def transform_query(query: str) -> str:
            """使用LLM改写查询以提高检索效果"""
            transform_prompt = PromptTemplate.from_template(
                "请将以下用户查询改写成更适合向量数据库检索的形式，添加可能相关的关键词，但保持简洁。\\n\\n用户查询: {query}\\n\\n改写后的查询:"
            )
            chain = transform_prompt | self.llm
            result = chain.invoke({"query": query})
            return result.content if hasattr(result, "content") else str(result)
        
        # 3. 定义搜索函数
        def enhanced_search(query: str) -> str:
            """增强搜索函数，包括查询转换和上下文压缩"""
            # 转换查询
            transformed_query = transform_query(query)
            print(f"原始查询: {query}")
            print(f"改写查询: {transformed_query}")
            
            # 使用压缩检索器获取结果
            docs = compression_retriever.get_relevant_documents(transformed_query)
            return "\\n\\n".join(doc.page_content for doc in docs)
        
        # 创建工具集
        tools = [
            Tool(
                name="增强知识搜索",
                func=enhanced_search,
                description="当需要查询关于搜索技术、智能体集成方法或应用场景的信息时使用"
            ),
            PythonREPLTool()
        ]
        
        return tools
    
    def setup_agent(self):
        """设置高级搜索智能体"""
        if not self.vectorstore:
            self.setup_knowledge_base()
        
        # 获取高级搜索工具
        tools = self.setup_advanced_search_tools()
        
        # 创建系统提示
        system_message = SystemMessage(
            content="""你是一个先进的搜索增强智能体，专门回答关于搜索技术与智能体集成的问题。

遵循以下步骤:
1. 理解用户的查询意图
2. 使用增强知识搜索工具获取相关信息
3. 基于检索到的信息提供全面的回答
4. 如果需要进行计算或示例，使用Python工具
5. 以清晰、结构化的方式组织你的回答

如果检索的信息不完整，可以基于你的知识进行补充，但要明确区分来源。
            """
        )
        
        # 创建提示模板
        prompt = ChatPromptTemplate.from_messages([
            system_message,
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        
        # 创建智能体
        agent = create_react_agent(self.llm, tools, prompt)
        
        # 创建执行器
        self.agent = AgentExecutor.from_agent_and_tools(
            agent=agent,
            tools=tools,
            memory=self.memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=5
        )
        
        return self.agent
    
    def run_query(self, query: str) -> str:
        """执行查询"""
        if not self.agent:
            self.setup_agent()
        
        print(f"\\n执行查询: {query}")
        result = self.agent.invoke(query)
        return result

def main():
    print("=== LangChain 高级搜索智能体示例 ===")
    
    # 创建高级搜索智能体
    agent = AdvancedSearchAgent()
    agent.setup_knowledge_base()
    agent.setup_agent()
    
    # 执行多个查询示例
    queries = [
        "比较不同搜索与智能体集成架构的优缺点",
        "有哪些高级检索技术可以提高搜索质量?",
        "搜索增强智能体在哪些行业有应用，并计算行业数量"
    ]
    
    for i, query in enumerate(queries):
        print(f"\\n--- 查询 {i+1} ---")
        result = agent.run_query(query)
        print(f"\\n最终回答: {result}")
    
    print("\\n=== 示例执行完成 ===")

if __name__ == "__main__":
    main()
"""

    # Write the examples to files
    editor.write("langchain_tutorial/code_examples/search_agent_integration/basic_search_agent.py", basic_search_agent)
    print("Created basic search agent example file")
    
    editor.write("langchain_tutorial/code_examples/search_agent_integration/advanced_search_agent.py", advanced_search_agent)
    print("Created advanced search agent example file")
    
    # Create README file
    readme_content = """# LangChain 搜索与智能体集成示例

本目录包含展示LangChain框架中搜索功能与智能体集成的各种实现方式的代码示例。

## 文件说明

1. `basic_search_agent.py` - 基础搜索智能体示例
   - 演示三种基本的搜索-智能体集成模式：
     - 直接检索QA链
     - 检索作为智能体工具
     - 上下文增强智能体

2. `advanced_search_agent.py` - 高级搜索智能体示例
   - 演示更复杂的搜索增强技术：
     - 查询转换/重写
     - 上下文压缩
     - 多轮对话记忆

## 集成方法概述

1. **直接检索模式**
   - 使用检索QA链直接查询向量存储
   - 最简单的实现方式
   - 适用于单轮、直接的问答场景

2. **检索工具模式**
   - 将检索器包装为智能体工具
   - 让智能体决定何时搜索信息
   - 适用于复杂任务和开放域问答

3. **上下文增强模式**
   - 在处理前检索相关信息
   - 作为上下文提供给智能体
   - 适用于知识密集型任务

## 使用说明

1. 确保安装所需依赖：
   ```bash
   pip install langchain langchain_community faiss-cpu openai
   ```

2. 设置OpenAI API密钥：
   ```python
   import os
   os.environ["OPENAI_API_KEY"] = "your-api-key"
   ```

3. 运行示例：
   ```bash
   python basic_search_agent.py
   python advanced_search_agent.py
   ```
"""
    
    editor.write("langchain_tutorial/code_examples/search_agent_integration/README.md", readme_content)
    print("Created README file")
    
    # List files
    files = await terminal.run("find langchain_tutorial/code_examples/search_agent_integration -type f | sort")
    print(f"\nCreated files in search agent integration directory:\n{files}")
    
    print("\n=== Search Agent Example Creation Complete ===")
    print("Created comprehensive examples for:")
    print("1. Basic search-agent integration patterns")
    print("2. Advanced search techniques with LangChain agents")
    print("3. Documentation and usage instructions")

# Run the main function
await create_search_agent_examples()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 486)

In [31]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

terminal = Terminal()
editor = Editor()

async def create_comprehensive_langchain_guide():
    # Create directory for documentation
    await terminal.run("mkdir -p langchain_tutorial/documentation")
    
    # Compile the comprehensive guide integrating all collected information
    comprehensive_guide = """# LangChain 框架全面指南

## 1. LangChain 框架概述

LangChain 是一个用于开发由大型语言模型（LLMs）驱动的应用程序的框架，它提供了一系列组件和工具，使开发者能够创建复杂的、交互式的、基于语言模型的应用。

### 1.1 框架核心理念

LangChain 的设计理念围绕以下几个核心原则：

- **组件化设计**：提供模块化的组件，可以独立使用或组合成复杂的系统
- **与语言模型的无缝集成**：优化与各种语言模型的交互方式
- **链式处理**：允许将多个组件组合成处理管道
- **状态管理**：提供记忆组件以维护对话历史和状态
- **工具集成**：允许语言模型与外部工具和系统交互

### 1.2 LangChain 表达式语言 (LCEL)

LangChain 表达式语言是一种声明式语言，用于组合 LangChain 的各种组件，具有以下特点：

- 使用管道操作符 (`|`) 连接组件
- 支持同步和异步操作
- 内置错误处理和重试机制
- 支持流式传输和批处理
- 简化复杂链的构建过程

示例：

SyntaxError: incomplete input (2884206134.py, line 13)

In [32]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

terminal = Terminal()
editor = Editor()

async def create_comprehensive_langchain_guide():
    # Create directory for documentation
    await terminal.run("mkdir -p langchain_tutorial/documentation")
    
    # Compile the comprehensive guide integrating all collected information
    comprehensive_guide = """# LangChain 框架全面指南

## 1. LangChain 框架概述

LangChain 是一个用于开发由大型语言模型（LLMs）驱动的应用程序的框架，它提供了一系列组件和工具，使开发者能够创建复杂的、交互式的、基于语言模型的应用。

### 1.1 框架核心理念

LangChain 的设计理念围绕以下几个核心原则：

- **组件化设计**：提供模块化的组件，可以独立使用或组合成复杂的系统
- **与语言模型的无缝集成**：优化与各种语言模型的交互方式
- **链式处理**：允许将多个组件组合成处理管道
- **状态管理**：提供记忆组件以维护对话历史和状态
- **工具集成**：允许语言模型与外部工具和系统交互

### 1.2 LangChain 表达式语言 (LCEL)

LangChain 表达式语言是一种声明式语言，用于组合 LangChain 的各种组件，具有以下特点：

- 使用管道操作符 (`|`) 连接组件
- 支持同步和异步操作
- 内置错误处理和重试机制
- 支持流式传输和批处理
- 简化复杂链的构建过程

示例：

SyntaxError: incomplete input (2884206134.py, line 13)

In [33]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

terminal = Terminal()
editor = Editor()

async def create_comprehensive_langchain_guide():
    # Create directory for documentation
    await terminal.run("mkdir -p langchain_tutorial/documentation")
    
    # Compile the comprehensive guide integrating all collected information
    comprehensive_guide = """# LangChain 框架全面指南

## 1. LangChain 框架概述

LangChain 是一个用于开发由大型语言模型（LLMs）驱动的应用程序的框架，它提供了一系列组件和工具，使开发者能够创建复杂的、交互式的、基于语言模型的应用。

### 1.1 框架核心理念

LangChain 的设计理念围绕以下几个核心原则：

- **组件化设计**：提供模块化的组件，可以独立使用或组合成复杂的系统
- **与语言模型的无缝集成**：优化与各种语言模型的交互方式
- **链式处理**：允许将多个组件组合成处理管道
- **状态管理**：提供记忆组件以维护对话历史和状态
- **工具集成**：允许语言模型与外部工具和系统交互

### 1.2 LangChain 表达式语言 (LCEL)

LangChain 表达式语言是一种声明式语言，用于组合 LangChain 的各种组件，具有以下特点：

- 使用管道操作符 (`|`) 连接组件
- 支持同步和异步操作
- 内置错误处理和重试机制
- 支持流式传输和批处理
- 简化复杂链的构建过程

示例：

SyntaxError: incomplete input (2884206134.py, line 13)

In [34]:
from metagpt.tools.libs.terminal import Terminal
from metagpt.tools.libs.editor import Editor
import os

terminal = Terminal()
editor = Editor()

async def create_final_document_structure():
    # Create directory for final documentation
    await terminal.run("mkdir -p langchain_tutorial/documentation")
    
    # Read the two parts of the comprehensive guide
    part1_path = "langchain_tutorial/documentation/langchain_comprehensive_guide.md"
    part2_path = "langchain_tutorial/documentation/langchain_comprehensive_guide_part2.md"
    
    part1_content = await editor.read(part1_path)
    part2_content = await editor.read(part2_path)
    
    # Combine them into a single document
    combined_content = part1_content.content + "\n\n" + part2_content.content
    
    # Write the combined guide
    editor.write("langchain_tutorial/documentation/langchain_complete_guide.md", combined_content)
    print("Combined comprehensive guide created successfully")
    
    # Create the final index document that links to all resources
    index_content = """# LangChain 框架学习资源索引

## 项目概述

本项目收集并整理了 LangChain 框架的相关资源，包括框架组件说明、智能体实现方法、搜索集成和多智能体工作流的设计与实现。所有资料基于 LangChain 官方文档（[https://python.langchain.com/docs/introduction/](https://python.langchain.com/docs/introduction/)）整理而成。

## 综合指南

- [LangChain 框架完整指南](langchain_complete_guide.md) - 全面介绍 LangChain 的架构、组件和使用方法

## 文档结构

### 基础文档

- [文档结构](../docs/document_structure.md) - LangChain 文档的整体结构
- [框架组件](../docs/components.md) - LangChain 的核心组件概述
- [智能体](../docs/agents.md) - 智能体系统概览
- [框架总览](../docs/overview.md) - LangChain 框架的总体架构

### LangChain 表达式语言 (LCEL)

- [LCEL 介绍](../docs/lcel_intro.md) - LangChain 表达式语言基础
- [LCEL 接口](../docs/lcel_interface.md) - LCEL 接口说明
- [LCEL 食谱](../docs/lcel_cookbook.md) - LCEL 使用示例

### 组件详解

- [模型输入输出](../docs/components/model_io.md) - 语言模型接口
- [记忆系统](../docs/components/memory.md) - 对话历史管理
- [提示模板](../docs/components/prompt_templates.md) - 提示工程
- [输出解析器](../docs/components/output_parsers.md) - 结构化输出处理
- [链](../docs/components/chains.md) - 组件组合
- [检索系统](../docs/components/retrieval.md) - 数据连接和检索

### 单智能体实现

- [智能体执行器](../docs/single_agent/agent_executor.md) - 智能体执行管理
- [ReAct 智能体](../docs/single_agent/react_agent.md) - 结合推理和行动的智能体
- [OpenAI 函数智能体](../docs/single_agent/openai_functions.md) - 基于函数调用的智能体
- [结构化聊天智能体](../docs/single_agent/structured_chat.md) - 结构化对话智能体
- [计划执行智能体](../docs/single_agent/plan_execute.md) - 规划与执行智能体

### 多智能体编排

- [智能体监督](../docs/multi_agent/agent_supervision.md) - 多智能体监督框架
- [团队监督者模式](../docs/multi_agent/team_supervisor.md) - 团队协作模式
- [经理-工人模式](../docs/multi_agent/manager_worker.md) - 层级协作模式
- [计划执行模式](../docs/multi_agent/plan_executor.md) - 规划与执行分离模式
- [CrewAI 集成](../docs/multi_agent/crewai_integration.md) - CrewAI 多智能体框架集成

### 搜索与智能体集成

- [搜索与智能体集成总结](../docs/search_agent_integration/search_agent_summary.md) - 综合概述
- [搜索工具](../docs/search_agent_integration/search_tools.md) - 智能体可用的搜索工具
- [搜索工具包](../docs/search_agent_integration/search_toolkit.md) - 搜索工具集成包
- [向量存储](../docs/search_agent_integration/vector_stores.md) - 向量数据库集成
- [检索器](../docs/search_agent_integration/retrievers.md) - 检索增强系统
- [向量数据库聊天](../docs/search_agent_integration/agent_with_retrieval_tool.md) - 集成检索工具的聊天智能体

### 智能体编排指南

- [智能体编排模式](../docs/agent_patterns_summary.md) - 单智能体和多智能体编排模式总结
- [智能体编排指南](../docs/agent_orchestration_guide.md) - 智能体协作系统设计指南

## 代码示例

### 智能体示例

- [单智能体示例](../code_examples/agent_examples/single_agent_example.py) - 不同类型单智能体实现
- [多智能体示例](../code_examples/agent_examples/multi_agent_example.py) - 多智能体协作模式实现
- [RAG 智能体示例](../code_examples/agent_examples/rag_agent_example.py) - 检索增强智能体实现
- [README](../code_examples/agent_examples/README.md) - 示例代码使用说明

### 搜索智能体集成示例

- [基础搜索智能体](../code_examples/search_agent_integration/basic_search_agent.py) - 基本搜索集成模式
- [高级搜索智能体](../code_examples/search_agent_integration/advanced_search_agent.py) - 高级搜索技术
- [README](../code_examples/search_agent_integration/README.md) - 搜索集成示例使用说明

### 多智能体工作流示例

- [多智能体工作流](../code_examples/multi_agent_demo/multi_agent_workflow.py) - 完整多智能体系统示例
- [专家智能体](../code_examples/multi_agent_demo/agents/specialist_agents.py) - 专业领域智能体实现
- [自定义工具](../code_examples/multi_agent_demo/tools/custom_tools.py) - 智能体工具扩展
- [LCEL 示例](../code_examples/multi_agent_demo/lcel_example.py) - LangChain 表达式语言示例
- [README](../code_examples/multi_agent_demo/README.md) - 多智能体演示使用说明

## 资源来源

所有资料基于 LangChain 官方文档：[https://python.langchain.com/docs/introduction/](https://python.langchain.com/docs/introduction/)
"""
    
    # Write the index file
    editor.write("langchain_tutorial/documentation/index.md", index_content)
    print("Created index document successfully")
    
    # Clean up - delete the separate parts since we have combined them
    await terminal.run("rm langchain_tutorial/documentation/langchain_comprehensive_guide_part2.md")
    print("Cleaned up temporary files")
    
    # List all documentation files
    doc_files = await terminal.run("find langchain_tutorial/documentation -type f | sort")
    print("\nFinal documentation files:")
    print(doc_files)
    
    # Create a simple README for the project root
    project_readme = """# LangChain 框架学习项目

## 项目简介

本项目收集并整理了 LangChain 框架的核心组件、智能体系统、搜索集成和多智能体工作流设计的相关资料和代码示例，旨在帮助开发者快速上手和深入理解 LangChain 框架。

## 资源导航

请查看 [资源索引](documentation/index.md) 获取完整的资源列表和导航。

## 主要内容

- **文档资料**：框架组件解析、智能体系统详解、搜索集成方法、多智能体工作流设计
- **代码示例**：单智能体实现、多智能体编排、RAG系统集成、LCEL表达式语言使用

## 数据来源

所有资料基于 LangChain 官方文档（[https://python.langchain.com/docs/introduction/](https://python.langchain.com/docs/introduction/)）整理而成。

## 使用方法

1. 浏览 [完整指南](documentation/langchain_complete_guide.md) 获取 LangChain 框架的全面介绍
2. 参考 [代码示例](code_examples/) 了解实际实现方法
3. 查阅特定主题的详细文档深入学习
"""
    
    editor.write("langchain_tutorial/README.md", project_readme)
    print("Created project README file")
    
    print("\n=== Document Organization Complete ===")
    print("Successfully created:")
    print("1. Combined comprehensive guide")
    print("2. Structured index document with links to all resources")
    print("3. Project README file")
    print("\nAll information has been organized into a coherent structure.")

# Execute the function to create the final document structure
await create_final_document_structure()

Combined comprehensive guide created successfully
Created index document successfully
Cleaned up temporary files

Final documentation files:

langchain_tutorial/documentation/index.md
langchain_tutorial/documentation/langchain_complete_guide.md
langchain_tutorial/documentation/langchain_comprehensive_guide.md

Created project README file

=== Document Organization Complete ===
Successfully created:
1. Combined comprehensive guide
2. Structured index document with links to all resources
3. Project README file

All information has been organized into a coherent structure.
